In [1]:
import os

data_path = "/kaggle/input/rsna-intracranial-aneurysm-detection"
print(os.listdir(data_path))


['train_localizers.csv', 'segmentations', 'series', 'train.csv', 'kaggle_evaluation']


!pip -q install python-gdcm

In [2]:
import os, importlib.util, pydicom
# Importing the handler registers it
import pydicom.pixel_data_handlers.gdcm_handler as gdcm_handler

print("GDCM handler available:", gdcm_handler.is_available())

# Optional: prove it decodes a random slice quickly
root = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"
some_uid = next(d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d)))
dcm_path = os.path.join(root, some_uid, os.listdir(os.path.join(root, some_uid))[0])

ds = pydicom.dcmread(dcm_path)
arr = ds.pixel_array  # should succeed quickly if GDCM is active
print("Decoded OK. shape:", arr.shape, "dtype:", arr.dtype)

GDCM handler available: False
Decoded OK. shape: (448, 392) dtype: int16


In [3]:
import pandas as pd

labels = pd.read_csv(os.path.join(data_path, "train.csv"))
print(labels.head())


                                   SeriesInstanceUID  PatientAge PatientSex  \
0  1.2.826.0.1.3680043.8.498.10004044428023505108...          64     Female   
1  1.2.826.0.1.3680043.8.498.10004684224894397679...          76     Female   
2  1.2.826.0.1.3680043.8.498.10005158603912009425...          58       Male   
3  1.2.826.0.1.3680043.8.498.10009383108068795488...          71       Male   
4  1.2.826.0.1.3680043.8.498.10012790035410518400...          48     Female   

  Modality  Left Infraclinoid Internal Carotid Artery  \
0      MRA                                          0   
1      MRA                                          0   
2      CTA                                          0   
3      MRA                                          0   
4      MRA                                          0   

   Right Infraclinoid Internal Carotid Artery  \
0                                           0   
1                                           0   
2                                  

In [4]:
import os, glob, numpy as np, pandas as pd
import pydicom
from pydicom.filereader import dcmread
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

DATA_DIR = "/kaggle/input/rsna-intracranial-aneurysm-detection"
SERIES_DIR = os.path.join(DATA_DIR, "series")
labels = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
print(labels.shape, labels.columns.tolist()[:8], '...')


(4348, 18) ['SeriesInstanceUID', 'PatientAge', 'PatientSex', 'Modality', 'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery', 'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery'] ...


In [5]:
import os, glob, numpy as np, pandas as pd
import pydicom
from pydicom.filereader import dcmread
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

DATA_DIR = "/kaggle/input/rsna-intracranial-aneurysm-detection"
SERIES_DIR = os.path.join(DATA_DIR, "series")
labels = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
print(labels.shape, labels.columns.tolist()[:8], '...')


(4348, 18) ['SeriesInstanceUID', 'PatientAge', 'PatientSex', 'Modality', 'Left Infraclinoid Internal Carotid Artery', 'Right Infraclinoid Internal Carotid Artery', 'Left Supraclinoid Internal Carotid Artery', 'Right Supraclinoid Internal Carotid Artery'] ...


# **TEST 1** 

**MIP** didnt work as expecte mostly i the coronal and the segittal planes goona fix theat and goona explore ways to improve on the code and on making th outpus shorter

In [ ]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Tri-planar MIP Baseline
Complete implementation in a single file for Kaggle notebook execution.

CRITICAL IMPROVEMENTS:
- TRUE on-the-fly MIP reduction (no RAM buildup)
- Completion order processing for better responsiveness
- Normalize once after reduction (not per slice)
- Conservative defaults for stability
- Proper DataLoader gating
- CuDNN benchmark option
- All decoder optimizations
"""

import os
import sys
import argparse
import random
import warnings
import time
import hashlib
from pathlib import Path
from typing import List, Tuple, Dict, Optional, Union
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing as mp

import numpy as np
import pandas as pd
import cv2
import pydicom
# Fix import path for apply_voi_lut
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except ImportError:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except ImportError:
        # Fallback for older pydicom versions
        def apply_voi_lut(pixel_array, ds):
            return pixel_array
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Initialize DICOM decoders at startup for optimal performance
def initialize_dicom_decoders():
    """Initialize pylibjpeg decoders at startup."""
    try:
        import pylibjpeg
        import pylibjpeg_libjpeg
        import pylibjpeg_openjpeg
        import pylibjpeg_rle
        # Enable external codecs
        pydicom.config.decode_external_data = True
        return True
    except ImportError:
        print("⚠️  Some DICOM decoders missing. Install with:")
        print("   pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg pylibjpeg-rle")
        return False

# Initialize decoders on import
DECODERS_AVAILABLE = initialize_dicom_decoders()

# =============================================================================
# CONFIGURATION
# =============================================================================

class Config:
    """Configuration class for the RSNA IA detection model."""
    
    # Data paths (Kaggle default)
    DATA_DIR = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV = f"{DATA_DIR}/train.csv"
    
    # Model parameters
    IMG_SIZE = 384
    BATCH_SIZE = 4
    EPOCHS = 3
    LR = 2e-4
    NUM_WORKERS = 0  # Start with 0, can bump to 2 later
    DEFAULT_WORKERS = 2  # Conservative default for stability
    SEED = 42
    
    # Performance optimization settings
    CACHE_DIR = "/kaggle/working/mip_cache"  # Cache preprocessed MIPs
    USE_CACHE = True  # Enable MIP caching
    MAX_WORKERS = min(4, mp.cpu_count())  # Max workers for parallel processing
    PREFETCH_FACTOR = 2  # DataLoader prefetch factor
    
    # DICOM decoder settings
    FORCE_PYLIBJPEG = True  # Force pylibjpeg for better performance
    
    # Cache version for breaking changes
    CACHE_VERSION = "v2.1"  # Increment when changing processing logic
    
    # Memory threshold for switching to streaming mode
    LARGE_SERIES_THRESHOLD = 200  # Switch to streaming for series > 200 slices
    
    # Target columns (13 anatomical locations + Aneurysm Present)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

def make_seed(seed: int, benchmark_mode: bool = False) -> None:
    """
    Set random seeds for reproducibility.
    
    Args:
        seed: Random seed value
        benchmark_mode: If True, enable cuDNN benchmark for speed over reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    if benchmark_mode:
        # Speed over reproducibility
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
        print("⚡ CuDNN benchmark mode enabled (speed over reproducibility)")
    else:
        # Reproducibility over speed
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def device() -> str:
    """Get the device to use for training/inference."""
    return "cuda" if torch.cuda.is_available() else "cpu"

# =============================================================================
# DICOM I/O AND MIP GENERATION (OPTIMIZED)
# =============================================================================

def get_mip_cache_path(series_path: str, out_size: int, modality_hint: str = None) -> str:
    """
    Generate cache file path for a series MIP with robust cache key.
    
    Args:
        series_path: Path to series folder
        out_size: Output image size
        modality_hint: Modality hint for cache key
        
    Returns:
        Cache file path
    """
    # Create a robust cache key including all relevant parameters
    series_uid = os.path.basename(series_path)
    modality = modality_hint or "unknown"
    
    # Include cache version, modality, and image size in key
    cache_key = f"{Config.CACHE_VERSION}_{series_uid}_{modality}_{out_size}"
    cache_hash = hashlib.md5(cache_key.encode()).hexdigest()[:16]
    
    cache_dir = Config.CACHE_DIR
    os.makedirs(cache_dir, exist_ok=True)
    
    # Use .npy instead of .pkl for better performance and safety
    return os.path.join(cache_dir, f"mip_{cache_hash}.npy")

def save_mip_to_cache(mips: np.ndarray, cache_path: str) -> None:
    """Save MIPs to cache file using uint8 format for 4x smaller files."""
    try:
        # Convert float32 [0,1] to uint8 [0,255] for 4x smaller cache files
        mips_uint8 = (mips * 255).astype(np.uint8)
        np.save(cache_path, mips_uint8, allow_pickle=False)
    except Exception as e:
        print(f"Warning: Failed to save MIP cache: {e}")

def load_mip_from_cache(cache_path: str) -> Optional[np.ndarray]:
    """Load MIPs from cache file and convert uint8 back to float32 [0,1]."""
    try:
        if os.path.exists(cache_path):
            # Load uint8 [0,255] and convert back to float32 [0,1]
            mips_uint8 = np.load(cache_path, allow_pickle=False)
            mips_float = mips_uint8.astype(np.float32) / 255.0
            return mips_float
    except Exception as e:
        print(f"Warning: Failed to load MIP cache: {e}")
    return None

def process_dicom_slice_optimized(file_path: str) -> Optional[np.ndarray]:
    """
    Optimized DICOM slice processing for parallel execution.
    """
    try:
        ds = pydicom.dcmread(file_path)
        img = read_pixel(ds)
        
        # Handle photometric interpretation
        if hasattr(ds, 'PhotometricInterpretation'):
            photometric = ds.PhotometricInterpretation
            if photometric == 'MONOCHROME1':
                img = np.max(img) - img
            elif photometric == 'RGB':
                if len(img.shape) == 3 and img.shape[-1] == 3:
                    img = np.mean(img, axis=-1)
        return img
    except Exception:
        return None

def iter_dicom_files(series_path: str) -> List[str]:
    """Return sorted DICOM files in the series folder."""
    series_path = Path(series_path)
    dcm_files = list(series_path.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in series_path.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    file_info = []
    for file_path in dcm_files:
        try:
            ds = pydicom.dcmread(str(file_path), stop_before_pixels=True)
            instance_num = getattr(ds, 'InstanceNumber', 0)
            file_info.append((instance_num, str(file_path)))
        except:
            file_info.append((0, str(file_path)))
    file_info.sort(key=lambda x: (x[0], x[1]))
    return [f[1] for f in file_info]

def read_pixel(ds) -> np.ndarray:
    """Robustly decode DICOM pixel data to numpy array."""
    try:
        pixel_array = ds.pixel_array
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            pixel_array = apply_voi_lut(pixel_array, ds)
        if hasattr(ds, 'RescaleSlope') and hasattr(ds, 'RescaleIntercept'):
            pixel_array = pixel_array.astype(np.float32)
            pixel_array = pixel_array * ds.RescaleSlope + ds.RescaleIntercept
        return pixel_array
    except Exception as e:
        raise RuntimeError(f"Failed to decode pixel data: {e}")

def window_ct(img: np.ndarray, level: int = 200, width: int = 500) -> np.ndarray:
    """Apply CT windowing to image -> [0,1]."""
    img = img.astype(np.float32)
    min_val = level - width // 2
    max_val = level + width // 2
    img = np.clip(img, min_val, max_val)
    img = (img - min_val) / (max_val - min_val)
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    """Percentile clip (1-99) then min-max -> [0,1]."""
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# ===================== FIXED: ALWAYS 2D CORONAL/SAGITTAL =====================

def make_triplanar_mips_parallel(series_path: str, modality_hint: Optional[str] = None, 
                                 out_size: int = 384, use_cache: bool = True) -> np.ndarray:
    """
    Create tri-planar MIPs using TRUE on-the-fly parallel processing.
    Ensures coronal/sagittal are always 2D even for single-slice series.
    """
    # Cache
    cache_path = get_mip_cache_path(series_path, out_size, modality_hint)
    if use_cache and Config.USE_CACHE:
        cached_mips = load_mip_from_cache(cache_path)
        if cached_mips is not None:
            return cached_mips
    
    dicom_files = iter_dicom_files(series_path)
    if not dicom_files:
        raise RuntimeError(f"No DICOM files found in {series_path}")
    total_slices = len(dicom_files)
    
    # Fallback to streaming for very large series
    if total_slices > Config.LARGE_SERIES_THRESHOLD:
        print(f"   📊 Large series detected ({total_slices} slices) - using streaming mode...")
        return make_triplanar_mips_stream(series_path, modality_hint, out_size)
    
    if total_slices > 50:
        print(f"   📊 Processing {total_slices} DICOM slices (parallel on-the-fly mode)...")
    
    axial_mip = None
    coronal_mips = []   # list of (W,)
    sagittal_mips = []  # list of (H,)
    successful_count = 0
    failed_decodes = 0
    
    max_workers = min(Config.MAX_WORKERS, total_slices)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_dicom_slice_optimized, f): f for f in dicom_files}
        completed_count = 0
        for future in as_completed(future_to_file):
            completed_count += 1
            try:
                img = future.result()
                if img is not None:
                    if img.ndim == 3:
                        if img.shape[2] == 3:  # RGB
                            img = np.mean(img, axis=2)
                        else:  # multi-channel
                            img = img.max(axis=2)
                    if axial_mip is None:
                        axial_mip = img.copy()
                    else:
                        axial_mip = np.maximum(axial_mip, img)
                    coronal_mips.append(img.max(axis=0))   # (W,)
                    sagittal_mips.append(img.max(axis=1))  # (H,)
                    successful_count += 1
                else:
                    failed_decodes += 1
                if total_slices > 100 and completed_count % 50 == 0:
                    print(f"      📈 Processed {completed_count}/{total_slices} slices ({successful_count} successful, {failed_decodes} failed)")
            except Exception:
                failed_decodes += 1
                continue
    
    if successful_count == 0:
        raise RuntimeError(f"No decodable slices in {series_path} (failed={failed_decodes})")
    if failed_decodes > 0:
        print(f"   ⚠️  Warning: {failed_decodes}/{total_slices} slices failed to decode in {os.path.basename(series_path)}")
    if total_slices > 50:
        print(f"   ✅ Parallel MIP generation complete: {successful_count} successful, {failed_decodes} failed")
    
    # Normalize axial once
    if modality_hint and modality_hint.upper() in ['CT', 'CTA']:
        axial_mip = window_ct(axial_mip)
    else:
        axial_mip = normalize_mr(axial_mip)
    H, W = axial_mip.shape  # base dimensions
    
    # CORONAL: build 2D (H, W) from a 1D row (W,)
    if coronal_mips:
        coronal_vec = coronal_mips[0] if len(coronal_mips) == 1 else np.max(np.stack(coronal_mips, axis=0), axis=0)
        if coronal_vec.ndim != 1:
            coronal_vec = coronal_vec.flatten()
        # resize vector to length W if needed
        if coronal_vec.shape[0] != W:
            coronal_vec = cv2.resize(coronal_vec[np.newaxis, :], (W, 1), interpolation=cv2.INTER_AREA).squeeze()
        coronal_mip = np.tile(coronal_vec[np.newaxis, :], (H, 1)).astype(np.float32)
        coronal_mip = window_ct(coronal_mip) if (modality_hint and modality_hint.upper() in ['CT','CTA']) else normalize_mr(coronal_mip)
        assert coronal_mip.ndim == 2, f"Coronal MIP should be 2D, got {coronal_mip.shape}"
    else:
        coronal_mip = np.zeros((H, W), dtype=np.float32)
    
    # SAGITTAL: build 2D (H, W) from a 1D col (H,)
    if sagittal_mips:
        sagittal_vec = sagittal_mips[0] if len(sagittal_mips) == 1 else np.max(np.stack(sagittal_mips, axis=0), axis=0)
        if sagittal_vec.ndim != 1:
            sagittal_vec = sagittal_vec.flatten()
        if sagittal_vec.shape[0] != H:
            sagittal_vec = cv2.resize(sagittal_vec[:, np.newaxis], (1, H), interpolation=cv2.INTER_AREA).squeeze()
        sagittal_mip = np.tile(sagittal_vec[:, np.newaxis], (1, W)).astype(np.float32)
        sagittal_mip = window_ct(sagittal_mip) if (modality_hint and modality_hint.upper() in ['CT','CTA']) else normalize_mr(sagittal_mip)
        assert sagittal_mip.ndim == 2, f"Sagittal MIP should be 2D, got {sagittal_mip.shape}"
    else:
        sagittal_mip = np.zeros((H, W), dtype=np.float32)
    
    # Resize to output size
    axial_mip    = cv2.resize(axial_mip,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal_mip  = cv2.resize(coronal_mip,  (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal_mip = cv2.resize(sagittal_mip, (out_size, out_size), interpolation=cv2.INTER_AREA)
    
    if total_slices > 50:
        print(f"   📐 MIP shapes before stacking:")
        print(f"      • Axial: {axial_mip.shape}")
        print(f"      • Coronal: {coronal_mip.shape}")
        print(f"      • Sagittal: {sagittal_mip.shape}")
    
    target_shape = (out_size, out_size)
    assert axial_mip.shape == target_shape
    assert coronal_mip.shape == target_shape
    assert sagittal_mip.shape == target_shape
    
    triplanar_mips = np.stack([axial_mip, coronal_mip, sagittal_mip], axis=0).astype(np.float32)
    if use_cache and Config.USE_CACHE:
        save_mip_to_cache(triplanar_mips, cache_path)
    return triplanar_mips

def make_triplanar_mips_stream(series_path: str, modality_hint: Optional[str] = None, 
                               out_size: int = 384) -> np.ndarray:
    """
    Create tri-planar MIPs by streaming slices (memory efficient).
    Ensures coronal/sagittal are always 2D even for single-slice series.
    """
    dicom_files = iter_dicom_files(series_path)
    if not dicom_files:
        raise RuntimeError(f"No DICOM files found in {series_path}")
    total_slices = len(dicom_files)
    
    if total_slices > 50:
        print(f"   📊 Processing {total_slices} DICOM slices (streaming mode - low memory)...")
    
    axial_mip = None
    coronal_rows = []   # (W,)
    sagittal_cols = []  # (H,)
    failed_decodes = 0
    successful_decodes = 0
    
    for i, file_path in enumerate(dicom_files):
        try:
            ds = pydicom.dcmread(file_path)
            img = read_pixel(ds)
            
            if hasattr(ds, 'PhotometricInterpretation'):
                photometric = ds.PhotometricInterpretation
                if photometric == 'MONOCHROME1':
                    img = np.max(img) - img
                elif photometric == 'RGB' and len(img.shape) == 3 and img.shape[-1] == 3:
                    img = np.mean(img, axis=-1)
            if img.ndim == 3:
                if img.shape[2] == 3:
                    img = np.mean(img, axis=2)
                else:
                    img = img.max(axis=2)
            
            if axial_mip is None:
                axial_mip = img.copy()
            else:
                axial_mip = np.maximum(axial_mip, img)
            
            coronal_rows.append(img.max(axis=0))   # (W,)
            sagittal_cols.append(img.max(axis=1))  # (H,)
            successful_decodes += 1
            
            if total_slices > 100 and (i + 1) % 50 == 0:
                print(f"      📈 Processed {i+1}/{total_slices} slices ({successful_decodes} successful, {failed_decodes} failed)")
        except Exception:
            failed_decodes += 1
            continue
    
    if successful_decodes == 0:
        raise RuntimeError(f"No decodable slices in {series_path} (failed={failed_decodes})")
    if failed_decodes > 0:
        print(f"   ⚠️  Warning: {failed_decodes}/{total_slices} slices failed to decode in {os.path.basename(series_path)}")
    if total_slices > 50:
        print(f"   ✅ Streaming MIP generation complete: {successful_decodes} successful, {failed_decodes} failed")
    
    # Normalize axial once
    if modality_hint and modality_hint.upper() in ['CT', 'CTA']:
        axial_mip = window_ct(axial_mip)
    else:
        axial_mip = normalize_mr(axial_mip)
    H, W = axial_mip.shape
    
    # CORONAL: 2D from 1D row
    if coronal_rows:
        coronal_vec = coronal_rows[0] if len(coronal_rows) == 1 else np.max(np.stack(coronal_rows, axis=0), axis=0)
        if coronal_vec.ndim != 1:
            coronal_vec = coronal_vec.flatten()
        if coronal_vec.shape[0] != W:
            coronal_vec = cv2.resize(coronal_vec[np.newaxis, :], (W, 1), interpolation=cv2.INTER_AREA).squeeze()
        coronal_mip = np.tile(coronal_vec[np.newaxis, :], (H, 1)).astype(np.float32)
        coronal_mip = window_ct(coronal_mip) if (modality_hint and modality_hint.upper() in ['CT','CTA']) else normalize_mr(coronal_mip)
        assert coronal_mip.ndim == 2
    else:
        coronal_mip = np.zeros((H, W), dtype=np.float32)
    
    # SAGITTAL: 2D from 1D col
    if sagittal_cols:
        sagittal_vec = sagittal_cols[0] if len(sagittal_cols) == 1 else np.max(np.stack(sagittal_cols, axis=0), axis=0)
        if sagittal_vec.ndim != 1:
            sagittal_vec = sagittal_vec.flatten()
        if sagittal_vec.shape[0] != H:
            sagittal_vec = cv2.resize(sagittal_vec[:, np.newaxis], (1, H), interpolation=cv2.INTER_AREA).squeeze()
        sagittal_mip = np.tile(sagittal_vec[:, np.newaxis], (1, W)).astype(np.float32)
        sagittal_mip = window_ct(sagittal_mip) if (modality_hint and modality_hint.upper() in ['CT','CTA']) else normalize_mr(sagittal_mip)
        assert sagittal_mip.ndim == 2
    else:
        sagittal_mip = np.zeros((H, W), dtype=np.float32)
    
    # Resize all to output size
    axial_mip    = cv2.resize(axial_mip,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal_mip  = cv2.resize(coronal_mip,  (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal_mip = cv2.resize(sagittal_mip, (out_size, out_size), interpolation=cv2.INTER_AREA)
    
    if total_slices > 50:
        print(f"   📐 MIP shapes before stacking:")
        print(f"      • Axial: {axial_mip.shape}")
        print(f"      • Coronal: {coronal_mip.shape}")
        print(f"      • Sagittal: {sagittal_mip.shape}")
    
    target_shape = (out_size, out_size)
    assert axial_mip.shape == target_shape
    assert coronal_mip.shape == target_shape
    assert sagittal_mip.shape == target_shape
    
    triplanar_mips = np.stack([axial_mip, coronal_mip, sagittal_mip], axis=0)
    return triplanar_mips.astype(np.float32)

# =============================================================================
# DATASET CLASS
# =============================================================================

class MIPDataset(Dataset):
    """Dataset class for tri-planar MIP images."""
    
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        
        missing_cols = [col for col in Config.TARGET_COLS if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Missing target columns: {missing_cols}")
    
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, str]:
        row = self.df.iloc[idx]
        series_uid = row['SeriesInstanceUID']
        
        series_path = os.path.join(self.series_dir, series_uid)
        if not os.path.exists(series_path):
            raise FileNotFoundError(f"Series folder not found: {series_path}")
        
        modality_hint = row.get('Modality', None)
        mips = make_triplanar_mips_parallel(series_path, modality_hint, self.out_size)
        x = torch.from_numpy(mips).float()
        y = torch.tensor([row[col] for col in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, series_uid

def stratified_split(df: pd.DataFrame, global_col: str = "Aneurysm Present", 
                    seed: int = 42, n_splits: int = 5) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if global_col not in df.columns:
        raise ValueError(f"Global column {global_col} not found in DataFrame")
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    for train_idx, val_idx in skf.split(df, df[global_col]):
        train_df = df.iloc[train_idx].copy()
        val_df = df.iloc[val_idx].copy()
        break
    return train_df, val_df

# =============================================================================
# MODEL DEFINITION
# =============================================================================

def build_efficientnet_b0(n_out: int = 14) -> nn.Module:
    try:
        model = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
    except AttributeError:
        model = torchvision.models.efficientnet_b0(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, n_out)
    return model

# =============================================================================
# METRICS
# =============================================================================

def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, 
                          cols: List[str]) -> Tuple[Dict[str, float], float]:
    column_aucs = {}
    valid_aucs = []
    for i, col in enumerate(cols):
        try:
            unique_classes = np.unique(y_true[:, i])
            if len(unique_classes) < 2:
                column_aucs[col] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            column_aucs[col] = auc
            valid_aucs.append(auc)
        except Exception as e:
            column_aucs[col] = np.nan
            print(f"Warning: Could not compute AUC for {col}: {e}")
    macro_auc = np.mean(valid_aucs) if valid_aucs else np.nan
    return column_aucs, macro_auc

# =============================================================================
# UTILITIES
# =============================================================================

def set_seed(seed: int, benchmark_mode: bool = False) -> None:
    make_seed(seed, benchmark_mode)

def count_params(model: nn.Module) -> int:
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def safe_series_path(series_dir: str, uid: str) -> str:
    series_path = os.path.join(series_dir, uid)
    if not os.path.exists(series_path):
        raise FileNotFoundError(f"Series path does not exist: {series_path}")
    return series_path

# =============================================================================
# TRAINING SCRIPT
# =============================================================================

def train_model(args):
    """Main training function."""
    start_time = time.time()
    print("=" * 60)
    print("🚀 STARTING RSNA IA DETECTION TRAINING")
    print("=" * 60)
    
    # Set seed and device
    print("📋 Setting up training environment...")
    benchmark_mode = getattr(args, 'benchmark_mode', False)
    set_seed(args.seed, benchmark_mode)
    device_str = device()
    print(f"✅ Device: {device_str}")
    print(f"✅ Random seed: {args.seed}")
    if not DECODERS_AVAILABLE:
        print("⚠️  Some DICOM decoders are missing - processing may be slower")
    
    # Load training data
    print(f"\n📁 Loading training data from {args.data_dir}")
    train_csv = os.path.join(args.data_dir, "train.csv")
    if not os.path.exists(train_csv):
        raise FileNotFoundError(f"❌ Training CSV not found: {train_csv}")
    
    print("📊 Reading CSV file...")
    df = pd.read_csv(train_csv)
    print(f"✅ Loaded {len(df):,} training samples")
    
    # Verify target columns exist
    print("\n🔍 Verifying target columns...")
    missing_cols = [col for col in Config.TARGET_COLS if col not in df.columns]
    if missing_cols:
        raise ValueError(f"❌ Missing target columns in CSV: {missing_cols}")
    print(f"✅ All {len(Config.TARGET_COLS)} target columns found")
    
    # Show data distribution
    print("\n📈 Data distribution:")
    aneurysm_present = df["Aneurysm Present"].sum()
    print(f"   • Aneurysm Present: {aneurysm_present:,} ({aneurysm_present/len(df)*100:.1f}%)")
    print(f"   • No Aneurysm: {len(df)-aneurysm_present:,} ({(len(df)-aneurysm_present)/len(df)*100:.1f}%)")
    
    # Create train/val split
    print("\n✂️  Creating train/validation split...")
    train_df, val_df = stratified_split(df, seed=args.seed)
    print(f"✅ Train: {len(train_df):,} samples")
    print(f"✅ Val: {len(val_df):,} samples")
    
    # Create datasets
    print(f"\n🔄 Creating datasets with image size {args.img_size}x{args.img_size}...")
    series_dir = os.path.join(args.data_dir, "series")
    print(f"📂 Series directory: {series_dir}")
    
    print("   • Building training dataset...")
    train_dataset = MIPDataset(train_df, series_dir, args.img_size)
    print("   • Building validation dataset...")
    val_dataset = MIPDataset(val_df, series_dir, args.img_size)
    print("✅ Datasets created successfully")
    
    # Create optimized data loaders
    print(f"\n📦 Creating optimized data loaders (batch_size={args.batch_size}, workers={args.num_workers})...")
    
    loader_kwargs = {
        'batch_size': args.batch_size,
        'shuffle': True,
        'num_workers': args.num_workers,
        'drop_last': True
    }
    if device_str == "cuda":
        loader_kwargs['pin_memory'] = True
    if args.num_workers > 0:
        loader_kwargs['persistent_workers'] = True
        loader_kwargs['prefetch_factor'] = Config.PREFETCH_FACTOR
    train_loader = DataLoader(train_dataset, **loader_kwargs)
    
    val_loader_kwargs = {
        'batch_size': args.batch_size,
        'shuffle': False,
        'num_workers': args.num_workers,
        'drop_last': False
    }
    if device_str == "cuda":
        val_loader_kwargs['pin_memory'] = True
    if args.num_workers > 0:
        val_loader_kwargs['persistent_workers'] = True
        val_loader_kwargs['prefetch_factor'] = Config.PREFETCH_FACTOR
    val_loader = DataLoader(val_dataset, **val_loader_kwargs)
    
    print(f"✅ Train batches: {len(train_loader)}")
    print(f"✅ Val batches: {len(val_loader)}")
    
    # Build model
    print("\n🏗️  Building model...")
    model = build_efficientnet_b0(n_out=len(Config.TARGET_COLS))
    model = model.to(device_str)
    param_count = count_params(model)
    print(f"✅ EfficientNet-B0 loaded with {param_count:,} parameters")
    print(f"✅ Model moved to {device_str}")
    
    # Positive weights
    print("\n⚖️  Computing positive weights for imbalanced classes...")
    pos_weights = []
    for i, col in enumerate(Config.TARGET_COLS):
        pos_count = train_df[col].sum()
        neg_count = len(train_df) - pos_count
        if pos_count > 0:
            pos_weight = min(max(neg_count / pos_count, 1.0), 50.0)
        else:
            pos_weight = 1.0
        pos_weights.append(pos_weight)
        if i < 5:
            print(f"   • {col}: {pos_weight:.2f}")
        elif i == 5:
            print(f"   • ... and {len(Config.TARGET_COLS)-5} more columns")
    pos_weights = torch.tensor(pos_weights, device=device_str)
    print(f"✅ Positive weights computed and applied")
    
    # Training components
    print("\n🔧 Setting up training components...")
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    optimizer = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)
    print(f"✅ Loss: BCEWithLogitsLoss with positive weighting")
    print(f"✅ Optimizer: AdamW (lr={args.lr}, weight_decay=1e-4)")
    print(f"✅ Scheduler: CosineAnnealingLR (T_max={args.epochs})")
    
    # Training loop
    best_auc = 0.0
    best_checkpoint = None
    epoch_times = []
    
    print(f"\n🎯 Starting training for {args.epochs} epochs...")
    print("=" * 60)
    
    for epoch in range(args.epochs):
        epoch_start = time.time()
        print(f"\n📅 EPOCH {epoch + 1}/{args.epochs}")
        print("-" * 40)
        
        # Training
        print("🏃 Training phase...")
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}", leave=False, ncols=100)
        for batch_idx, (x, y, _) in enumerate(train_bar):
            x, y = x.to(device_str), y.to(device_str)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            train_correct += (preds == y).sum().item()
            train_total += y.numel()
            
            avg_loss = train_loss / (batch_idx + 1)
            avg_acc = train_correct / train_total if train_total > 0 else 0
            train_bar.set_postfix({'Loss': f'{avg_loss:.4f}', 'Acc': f'{avg_acc:.3f}', 'LR': f'{scheduler.get_last_lr()[0]:.2e}'})
        
        train_loss /= len(train_loader)
        train_acc = train_correct / train_total if train_total > 0 else 0
        
        # Validation
        print("🧪 Validation phase...")
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []
        
        val_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}", leave=False, ncols=100)
        with torch.no_grad():
            for x, y, _ in val_bar:
                x, y = x.to(device_str), y.to(device_str)
                outputs = model(x)
                loss = criterion(outputs, y)
                val_loss += loss.item()
                probs = torch.sigmoid(outputs)
                all_preds.append(probs.cpu().numpy())
                all_targets.append(y.cpu().numpy())
                val_bar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        val_loss /= len(val_loader)
        print("📊 Computing metrics...")
        all_preds = np.vstack(all_preds)
        all_targets = np.vstack(all_targets)
        column_aucs, macro_auc = compute_auc_per_column(all_targets, all_preds, Config.TARGET_COLS)
        
        # Epoch results
        epoch_time = time.time() - epoch_start
        epoch_times.append(epoch_time)
        avg_epoch_time = np.mean(epoch_times)
        remaining_epochs = args.epochs - (epoch + 1)
        eta = remaining_epochs * avg_epoch_time
        
        print(f"\n📈 EPOCH {epoch + 1} RESULTS:")
        print(f"   ⏱️  Time: {epoch_time:.1f}s (Avg: {avg_epoch_time:.1f}s)")
        print(f"   🎯 Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.3f}")
        print(f"   🧪 Val Loss: {val_loss:.4f}")
        print(f"   🏆 Macro AUC: {macro_auc:.4f}")
        print(f"   ⏳ ETA: {eta/60:.1f} minutes")
        
        print(f"\n🔑 Key Column AUCs:")
        key_cols = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery", "Basilar Tip"]
        for col in key_cols:
            if col in column_aucs and not np.isnan(column_aucs[col]):
                print(f"   • {col}: {column_aucs[col]:.4f}")
        
        # Save best
        if macro_auc > best_auc:
            best_auc = macro_auc
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'auc_macro': macro_auc,
                'target_cols': Config.TARGET_COLS,
                'config': {
                    'img_size': args.img_size,
                    'batch_size': args.batch_size,
                    'lr': args.lr,
                    'seed': args.seed
                }
            }
            checkpoint_path = args.out
            torch.save(checkpoint, checkpoint_path)
            best_checkpoint = checkpoint_path
            print(f"\n💾 🆕 NEW BEST CHECKPOINT SAVED!")
            print(f"   📁 Path: {checkpoint_path}")
            print(f"   🏆 Previous Best → New Best: {macro_auc:.4f}")
        else:
            print(f"\n💾 Checkpoint not saved (current: {macro_auc:.4f} ≤ best: {best_auc:.4f})")
        
        scheduler.step()
        print("-" * 40)
    
    total_time = time.time() - start_time
    print("\n" + "=" * 60)
    print("🎉 TRAINING COMPLETED!")
    print("=" * 60)
    print(f"⏱️  Total Time: {total_time/60:.1f} minutes")
    print(f"🏆 Best Macro AUC: {best_auc:.4f}")
    print(f"💾 Best Checkpoint: {best_checkpoint}")
    print(f"📊 Final Results:")
    print(f"   • Train samples: {len(train_df):,}")
    print(f"   • Val samples: {len(val_df):,}")
    print(f"   • Model parameters: {param_count:,}")
    print(f"   • Image size: {args.img_size}x{args.img_size}")
    print(f"   • Batch size: {args.batch_size}")
    print(f"   • Learning rate: {args.lr}")
    print("=" * 60)
    
    return best_checkpoint

# =============================================================================
# JUPYTER-FRIENDLY FUNCTIONS
# =============================================================================

def train_model_jupyter(data_dir=None, img_size=None, batch_size=None, epochs=None, 
                        lr=None, num_workers=None, seed=None, out=None, use_cache=None,
                        benchmark_mode=False):
    """Jupyter-friendly training function."""
    if use_cache is not None:
        Config.USE_CACHE = use_cache
    
    class Args:
        def __init__(self):
            self.data_dir = data_dir or Config.DATA_DIR
            self.img_size = img_size or Config.IMG_SIZE
            self.batch_size = batch_size or Config.BATCH_SIZE
            self.epochs = epochs or Config.EPOCHS
            self.lr = lr or Config.LR
            self.num_workers = num_workers if num_workers is not None else Config.DEFAULT_WORKERS
            self.seed = seed or Config.SEED
            self.out = out or '/kaggle/working/best.pt'
            self.benchmark_mode = benchmark_mode
    args = Args()
    return train_model(args)

def run_inference_jupyter(ckpt, data_dir=None, uids_csv=None, out_csv=None):
    """Jupyter-friendly inference function."""
    class Args:
        def __init__(self):
            self.ckpt = ckpt
            self.data_dir = data_dir or Config.DATA_DIR
            self.uids_csv = uids_csv
            self.out_csv = out_csv or '/kaggle/working/preds.csv'
    args = Args()
    return run_inference(args)

def check_dicom_decoders():
    """Check if proper DICOM decoders are installed for optimal performance."""
    print("🔍 DICOM DECODER CHECK")
    print("=" * 40)
    try:
        import pydicom
        print(f"✅ Pydicom version: {pydicom.__version__}")
    except ImportError:
        print("❌ Pydicom not installed")
        return False
    
    decoders_installed = True
    try:
        import pylibjpeg
        print("✅ pylibjpeg: Installed")
    except ImportError:
        print("❌ pylibjpeg: NOT installed (will use slow Python fallback)")
        decoders_installed = False
    try:
        import pylibjpeg_libjpeg
        print("✅ pylibjpeg-libjpeg: Installed")
    except ImportError:
        print("❌ pylibjpeg-libjpeg: NOT installed (JPEG decoding may be slow)")
        decoders_installed = False
    try:
        import pylibjpeg_openjpeg
        print("✅ pylibjpeg-openjpeg: Installed")
    except ImportError:
        print("❌ pylibjpeg-openjpeg: NOT installed (JPEG2000 decoding may be slow)")
        decoders_installed = False
    try:
        import pylibjpeg_rle
        print("✅ pylibjpeg-rle: Installed")
    except ImportError:
        print("❌ pylibjpeg-rle: NOT installed (RLE decoding may be slow)")
        decoders_installed = False
    
    if decoders_installed:
        print("\n🎉 All DICOM decoders installed! Optimal performance expected.")
    else:
        print("\n⚠️  Some DICOM decoders missing. Install with:")
        print("   pip install pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg pylibjpeg-rle")
        print("   This will significantly improve DICOM processing speed.")
    print("=" * 40)
    return decoders_installed

def quick_test():
    """Quick test function to verify the code works in Jupyter."""
    print("🧪 QUICK TEST MODE")
    print("=" * 40)
    try:
        print("✅ Testing imports...")
        import numpy as np  # noqa
        import pandas as pd  # noqa
        import torch  # noqa
        import torchvision  # noqa
        import cv2  # noqa
        import pydicom  # noqa
        print("   • All required packages imported successfully")
        print("✅ Testing device detection...")
        device_str = device()
        print(f"   • Device: {device_str}")
        print("✅ Testing model building...")
        model = build_efficientnet_b0(n_out=14)
        param_count = count_params(model)
        print(f"   • Model built with {param_count:,} parameters")
        print("✅ Testing configuration...")
        print(f"   • Target columns: {len(Config.TARGET_COLS)}")
        print(f"   • Image size: {Config.IMG_SIZE}")
        print(f"   • Batch size: {Config.BATCH_SIZE}")
        print(f"   • Default workers: {Config.DEFAULT_WORKERS}")
        print("✅ Testing data directory...")
        data_dir = Config.DATA_DIR
        if os.path.exists(data_dir):
            print(f"   • Data directory exists: {data_dir}")
            train_csv = os.path.join(data_dir, "train.csv")
            if os.path.exists(train_csv):
                print(f"   • Training CSV found: {train_csv}")
                try:
                    df = pd.read_csv(train_csv, nrows=5)
                    print(f"   • CSV readable: {len(df)} sample rows loaded")
                except Exception as e:
                    print(f"   • CSV read error: {e}")
            else:
                print(f"   • Training CSV not found: {train_csv}")
        else:
            print(f"   • Data directory not found: {data_dir}")
            print("   • This is expected if you haven't attached the Kaggle dataset yet")
        print("\n🎉 ALL TESTS PASSED! The code is ready to use.")
    except Exception as e:
        print(f"\n❌ Test failed: {e}")
        print("Please check your installation and try again.")

def clear_mip_cache():
    """Clear the MIP cache directory."""
    print("🧹 CLEARING MIP CACHE")
    print("=" * 40)
    cache_dir = Config.CACHE_DIR
    if os.path.exists(cache_dir):
        import shutil
        try:
            shutil.rmtree(cache_dir)
            print(f"✅ Cache directory cleared: {cache_dir}")
        except Exception as e:
            print(f"❌ Failed to clear cache: {e}")
    else:
        print(f"ℹ️  Cache directory doesn't exist: {cache_dir}")
    print("=" * 40)

def clear_old_cache_versions():
    """Clear cache files from old versions to prevent silent bugs."""
    print("🔄 CLEARING OLD CACHE VERSIONS")
    print("=" * 40)
    cache_dir = Config.CACHE_DIR
    if not os.path.exists(cache_dir):
        print("ℹ️  Cache directory doesn't exist")
        return
    cleared_count = 0
    for filename in os.listdir(cache_dir):
        if filename.endswith('.npy'):
            file_path = os.path.join(cache_dir, filename)
            try:
                test_load = np.load(file_path, allow_pickle=False)
                if test_load.dtype != np.uint8:
                    os.remove(file_path)
                    cleared_count += 1
                    print(f"   • Removed old format file: {filename}")
            except Exception:
                os.remove(file_path)
                cleared_count += 1
                print(f"   • Removed corrupted/old file: {filename}")
    if cleared_count == 0:
        print("✅ No old cache files found")
    else:
        print(f"✅ Cleared {cleared_count} old cache files")
    print("=" * 40)

def get_cache_info():
    """Get information about the MIP cache."""
    print("📊 MIP CACHE INFO")
    print("=" * 40)
    cache_dir = Config.CACHE_DIR
    print(f"Cache directory: {cache_dir}")
    print(f"Cache enabled: {Config.USE_CACHE}")
    print(f"Cache version: {Config.CACHE_VERSION}")
    if os.path.exists(cache_dir):
        cache_files = [f for f in os.listdir(cache_dir) if f.endswith('.npy')]
        total_size = 0
        for f in cache_files:
            file_path = os.path.join(cache_dir, f)
            total_size += os.path.getsize(file_path)
        print(f"✅ Cache files: {len(cache_files)}")
        print(f"✅ Total size: {total_size / (1024*1024):.1f} MB")
        print(f"✅ Avg size per file: {total_size / (1024*1024) / len(cache_files):.2f} MB" if cache_files else "N/A")
        if cache_files:
            print("\nRecent cache files:")
            cache_files.sort(key=lambda x: os.path.getmtime(os.path.join(cache_dir, x)), reverse=True)
            for f in cache_files[:5]:
                file_path = os.path.join(cache_dir, f)
                mtime = time.ctime(os.path.getmtime(file_path))
                size_mb = os.path.getsize(file_path) / (1024*1024)
                print(f"   • {f}: {size_mb:.1f} MB ({mtime})")
    else:
        print("ℹ️  Cache directory doesn't exist yet")
    print("=" * 40)

def check_data_directory():
    """Check if the Kaggle data directory exists and show its contents."""
    print("📁 DATA DIRECTORY CHECK")
    print("=" * 40)
    data_dir = Config.DATA_DIR
    print(f"Expected data directory: {data_dir}")
    if os.path.exists(data_dir):
        print("✅ Data directory exists!")
        train_csv = os.path.join(data_dir, "train.csv")
        if os.path.exists(train_csv):
            print("✅ Training CSV found!")
            try:
                df = pd.read_csv(train_csv, nrows=5)
                print(f"   • Sample data shape: {df.shape}")
                print(f"   • Columns: {list(df.columns)}")
                print(f"   • Sample SeriesInstanceUID: {df['SeriesInstanceUID'].iloc[0]}")
            except Exception as e:
                print(f"   • Error reading CSV: {e}")
        else:
            print("❌ Training CSV not found")
        series_dir = os.path.join(data_dir, "series")
        if os.path.exists(series_dir):
            print("✅ Series directory found!")
            try:
                series_folders = [f for f in os.listdir(series_dir) if os.path.isdir(os.path.join(series_dir, f))]
                print(f"   • Number of series folders: {len(series_folders)}")
                if series_folders:
                    sample_series = series_folders[0]
                    sample_path = os.path.join(series_dir, sample_series)
                    sample_files = [f for f in os.listdir(sample_path) if os.path.isfile(os.path.join(sample_path, f))]
                    print(f"   • Sample series '{sample_series}' has {len(sample_files)} files")
            except Exception as e:
                print(f"   • Error checking series directory: {e}")
        else:
            print("❌ Series directory not found")
    else:
        print("❌ Data directory not found!")
        print("\n💡 To fix this:")
        print("   1. Make sure you've attached the RSNA dataset to your Kaggle notebook")
        print("   2. Check the dataset path in the notebook")
        print("   3. Verify the dataset contains 'train.csv' and 'series/' subdirectory")
        print(f"   4. Expected path: {data_dir}")
    print("=" * 40)

# =============================================================================
# INFERENCE SCRIPT
# =============================================================================

def run_inference(args):
    """Main inference function."""
    start_time = time.time()
    print("=" * 60)
    print("🔍 STARTING RSNA IA DETECTION INFERENCE")
    print("=" * 60)
    
    # Load checkpoint
    print("📁 Loading checkpoint...")
    checkpoint = torch.load(args.ckpt, map_location='cpu')
    target_cols = checkpoint['target_cols']
    config = checkpoint.get('config', {})
    
    auc_val = checkpoint.get('auc_macro', None)
    auc_str = f"{auc_val:.4f}" if isinstance(auc_val, (int, float, np.floating)) else "N/A"
    print(f"✅ Checkpoint loaded from epoch {checkpoint['epoch']}")
    print(f"✅ Target columns: {len(target_cols)}")
    print(f"✅ Checkpoint AUC: {auc_str}")
    
    if config:
        print(f"\n⚙️  Model Configuration:")
        for key, value in config.items():
            print(f"   • {key}: {value}")
    
    # Build model
    print(f"\n🏗️  Building model...")
    model = build_efficientnet_b0(n_out=len(target_cols))
    model.load_state_dict(checkpoint['model_state_dict'])
    
    device_str = device()
    model = model.to(device_str)
    model.eval()
    print(f"✅ Model loaded and moved to {device_str}")
    print(f"✅ Model parameters: {count_params(model):,}")
    
    # Load data
    print(f"\n📊 Loading data...")
    if args.uids_csv:
        print(f"📋 Loading UIDs from: {args.uids_csv}")
        uids_df = pd.read_csv(args.uids_csv)
        uids = uids_df['SeriesInstanceUID'].tolist()
        print(f"✅ Loaded {len(uids):,} UIDs from CSV")
    else:
        train_csv = os.path.join(args.data_dir, "train.csv")
        print(f"📋 Using sample from training data: {train_csv}")
        df = pd.read_csv(train_csv)
        uids = df['SeriesInstanceUID'].head(10).tolist()
        print(f"✅ Using sample of {len(uids):,} UIDs from training data")
    
    results = []
    series_dir = os.path.join(args.data_dir, "series")
    print(f"📂 Series directory: {series_dir}")
    
    print(f"\n🚀 Running inference on {len(uids):,} series...")
    print("=" * 60)
    
    successful = 0
    failed = 0
    processing_times = []
    
    for i, uid in enumerate(tqdm(uids, desc="Inference", ncols=100)):
        uid_start = time.time()
        try:
            series_path = safe_series_path(series_dir, uid)
            mips = make_triplanar_mips_parallel(series_path, out_size=config.get('img_size', 384))
            x = torch.from_numpy(mips).float().unsqueeze(0).to(device_str)
            with torch.no_grad():
                outputs = model(x)
                probs = torch.sigmoid(outputs).cpu().numpy()[0]
            result = {'SeriesInstanceUID': uid}
            for j, col in enumerate(target_cols):
                result[col] = probs[j]
            results.append(result)
            successful += 1
            if (i + 1) % 10 == 0 or len(uids) <= 20:
                elapsed = time.time() - start_time
                avg_time = elapsed / (i + 1)
                eta = avg_time * (len(uids) - i - 1)
                print(f"\n📊 Progress Update ({i+1}/{len(uids)}):")
                print(f"   • Successful: {successful}")
                print(f"   • Failed: {failed}")
                print(f"   • Elapsed: {elapsed/60:.1f} minutes")
                print(f"   • ETA: {eta/60:.1f} minutes")
                if i < 3:
                    aneurysm_prob = probs[-1]
                    print(f"   • Sample {uid[:8]}...: Aneurysm Present = {aneurysm_prob:.4f}")
        except Exception as e:
            print(f"\n❌ Error processing {uid}: {e}")
            result = {'SeriesInstanceUID': uid}
            for col in target_cols:
                result[col] = np.nan
            results.append(result)
            failed += 1
        uid_time = time.time() - uid_start
        processing_times.append(uid_time)
    
    total_time = time.time() - start_time
    avg_processing_time = np.mean(processing_times) if processing_times else 0
    
    print("\n" + "=" * 60)
    print("📊 INFERENCE COMPLETED!")
    print("=" * 60)
    print(f"⏱️  Total Time: {total_time/60:.1f} minutes")
    print(f"✅ Successful: {successful:,}")
    print(f"❌ Failed: {failed:,}")
    print(f"📈 Success Rate: {successful/(successful+failed)*100:.1f}%")
    print(f"⚡ Avg Processing Time: {avg_processing_time:.2f}s per series")
    
    print(f"\n💾 Saving results...")
    results_df = pd.DataFrame(results)
    results_df.to_csv(args.out_csv, index=False)
    print(f"✅ Results saved to: {args.out_csv}")
    print(f"📊 Output shape: {results_df.shape}")
    
    if len(results_df) > 0:
        print(f"\n🔍 Sample Results:")
        sample_cols = ["SeriesInstanceUID"] + target_cols[-3:]
        sample_df = results_df[sample_cols].head(3)
        for _, row in sample_df.iterrows():
            uid_short = row['SeriesInstanceUID'][:8] + "..."
            aneurysm_prob = row.get('Aneurysm Present', 'N/A')
            print(f"   • {uid_short}: Aneurysm Present = {aneurysm_prob}")
    print("=" * 60)
    return results_df

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main entry point."""
    print("🧠 RSNA Intracranial Aneurysm Detection")
    print("   Tri-planar MIP Baseline Implementation")
    print("   " + "=" * 50)
    
    # Check if we're running in Jupyter/IPython
    try:
        get_ipython()
        # We're in Jupyter - show usage instructions
        print("\n📚 JUPYTER NOTEBOOK MODE DETECTED!")
        print("=" * 50)
        print("To use this script in Jupyter, call the functions directly:")
        print()
        print("🎯 For Training:")
        print("   train_model_jupyter()  # Basic training")
        print("   train_model_jupyter(num_workers=2, batch_size=8)  # Optimized")
        print("   train_model_jupyter(benchmark_mode=True)  # Max speed")
        print()
        print("🔍 For Inference:")
        print("   run_inference_jupyter(ckpt='/kaggle/working/best.pt')")
        print()
        print("📋 For Testing & Cache Management:")
        print("   check_dicom_decoders()      # Check DICOM decoder performance")
        print("   quick_test()                # Test all components")
        print("   get_cache_info()            # Check MIP cache status")
        print("   clear_mip_cache()           # Clear all cached MIPs")
        print("   clear_old_cache_versions()  # Clear old version cache files")
        print("   check_data_directory()      # Verify dataset")
        print()
        print("⚡ PERFORMANCE OPTIMIZATIONS:")
        print(f"   • TRUE on-the-fly MIP reduction (no RAM buildup)")
        print(f"   • Completion-order parallel processing")
        print(f"   • Normalize once after reduction")
        print(f"   • Conservative defaults (workers={Config.DEFAULT_WORKERS})")
        print(f"   • Smart fallback for large series (>{Config.LARGE_SERIES_THRESHOLD} slices)")
        print(f"   • Fast uint8 caching ({'ON' if Config.USE_CACHE else 'OFF'})")
        print(f"   • Cache version: {Config.CACHE_VERSION}")
        print("=" * 50)
        return
    except NameError:
        # Not in Jupyter; proceed with argparse
        pass
    
    parser = argparse.ArgumentParser(description="RSNA IA Detection Training/Inference")
    subparsers = parser.add_subparsers(dest='command', help='Command to run')
    
    # Training parser
    train_parser = subparsers.add_parser('train', help='Train the model')
    train_parser.add_argument('--data_dir', type=str, default=Config.DATA_DIR, help='Data directory path')
    train_parser.add_argument('--img_size', type=int, default=Config.IMG_SIZE, help='Image size')
    train_parser.add_argument('--batch_size', type=int, default=Config.BATCH_SIZE, help='Batch size')
    train_parser.add_argument('--epochs', type=int, default=Config.EPOCHS, help='Number of epochs')
    train_parser.add_argument('--lr', type=float, default=Config.LR, help='Learning rate')
    train_parser.add_argument('--num_workers', type=int, default=Config.NUM_WORKERS, help='Number of data loader workers')
    train_parser.add_argument('--seed', type=int, default=Config.SEED, help='Random seed')
    train_parser.add_argument('--out', type=str, default='/kaggle/working/best.pt', help='Output checkpoint path')
    train_parser.add_argument('--benchmark_mode', action='store_true', help='Enable cuDNN benchmark for speed over reproducibility')
    
    # Inference parser
    infer_parser = subparsers.add_parser('infer', help='Run inference')
    infer_parser.add_argument('--ckpt', type=str, required=True, help='Checkpoint path')
    infer_parser.add_argument('--data_dir', type=str, default=Config.DATA_DIR, help='Data directory path')
    infer_parser.add_argument('--uids_csv', type=str, default=None, help='CSV with UIDs to process (optional)')
    infer_parser.add_argument('--out_csv', type=str, default='/kaggle/working/preds.csv', help='Output CSV path')
    
    args = parser.parse_args()
    if args.command == 'train':
        print(f"\n🎯 Starting TRAINING mode...")
        print(f"   • Data directory: {args.data_dir}")
        print(f"   • Image size: {args.img_size}x{args.img_size}")
        print(f"   • Batch size: {args.batch_size}")
        print(f"   • Epochs: {args.epochs}")
        print(f"   • Learning rate: {args.lr}")
        print(f"   • Workers: {args.num_workers}")
        print(f"   • Output: {args.out}")
        if getattr(args, 'benchmark_mode', False):
            print(f"   • CuDNN benchmark: ENABLED")
        print("   " + "-" * 50)
        train_model(args)
    elif args.command == 'infer':
        print(f"\n🔍 Starting INFERENCE mode...")
        print(f"   • Checkpoint: {args.ckpt}")
        print(f"   • Data directory: {args.data_dir}")
        print(f"   • Output CSV: {args.out_csv}")
        if args.uids_csv:
            print(f"   • UIDs CSV: {args.uids_csv}")
        else:
            print(f"   • UIDs: Using sample from training data")
        print("   " + "-" * 50)
        run_inference(args)
    else:
        print("\n❓ No command specified. Use --help for options.")
        parser.print_help()

if __name__ == "__main__":
    main()

train_model_jupyter(num_workers=2)

# **TEST 2** 

Fixing the **MIP**  and also lessening the jargons to see where we reach

In [ ]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Tri-planar MIP (Stable, runs on Kaggle)

What changed to fix your error:
- Replaced the "compressed header guess" with a real quick test decode:
  we try ds.pixel_array on the first slice; if it works, we keep the series.
- If filtering keeps 0 items, we FALL BACK to an unfiltered subset so training
  still runs and completes.
- Dataset __getitem__ has a last-resort fallback (returns zeros) to avoid crashes.

Usage after loading this cell:
train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True
)
"""

import os, time, random, warnings, hashlib
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    IMG_SIZE   = 288
    BATCH_SIZE = 2
    EPOCHS     = 1
    LR         = 2e-4
    NUM_WORKERS= 0
    SEED       = 42
    USE_AMP    = True

    LOG_EVERY_N = 50
    USE_CACHE = True
    CACHE_DIR = "/kaggle/working/mip_cache"
    CACHE_VERSION = "v4.2"

    # Always use streaming (low RAM)
    LARGE_SERIES_THRESHOLD = 1
    MAX_WORKERS = 1

    # Keep a subset so one epoch completes quickly. Set both to None to use ALL.
    SMOKE_TRAIN_LIMIT = 600
    SMOKE_VAL_LIMIT   = 200

    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

# =============================================================================
# DICOM I/O & MIP (streaming)
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = level - width/2.0, level + width/2.0
    img = np.clip(img, lo, hi)
    img = (img - lo) / max(1e-6, (hi - lo))
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

def _cache_path(series_path: str, out_size: int, modality_hint: Optional[str]) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{modality_hint or 'unk'}_{out_size}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"mip_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if not Config.USE_CACHE or not os.path.exists(path):
            return None
        arr = np.load(path, allow_pickle=False)
        return arr.astype(np.float32) / 255.0
    except Exception:
        return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, (arr * 255).astype(np.uint8), allow_pickle=False)
    except Exception:
        pass

def make_triplanar_mips_stream(series_path: str, modality_hint: Optional[str], out_size: int) -> np.ndarray:
    cache = _cache_path(series_path, out_size, modality_hint)
    cached = _load_cache(cache)
    if cached is not None:
        return cached

    files = iter_dicom_files(series_path)
    if not files:
        raise RuntimeError("No DICOMs in series")

    axial = None
    cor_rows = None  # (Z, W)
    sag_cols = None  # (Z, H)

    z = 0
    H = W = None
    for f in files:
        try:
            ds = pydicom.dcmread(f, force=True)
            arr = ds.pixel_array  # let pydicom decide; if it fails we skip this slice
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)

            if axial is None:
                H, W = img.shape
                axial = img.copy()
                cor_rows = np.zeros((len(files), W), dtype=np.float32)
                sag_cols = np.zeros((len(files), H), dtype=np.float32)
            else:
                np.maximum(axial, img, out=axial)

            cor_rows[z, :] = img.max(axis=0)
            sag_cols[z, :] = img.max(axis=1)
            z += 1
        except Exception:
            # skip a bad/unsupported slice
            continue

    if axial is None:
        raise RuntimeError("No decodable slices in series")

    if modality_hint and str(modality_hint).upper() in ["CT", "CTA"]:
        axial = window_ct(axial);    cor_rows = window_ct(cor_rows); sag_cols = window_ct(sag_cols)
    else:
        axial = normalize_mr(axial); cor_rows = normalize_mr(cor_rows); sag_cols = normalize_mr(sag_cols)

    axial    = cv2.resize(axial,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal  = cv2.resize(cor_rows, (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal = cv2.resize(sag_cols, (out_size, out_size), interpolation=cv2.INTER_AREA)

    mips = np.stack([axial, coronal, sagittal], axis=0).astype(np.float32)
    _save_cache(mips, cache)
    return mips

# =============================================================================
# DATASET & FILTERING
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p):
        raise FileNotFoundError(p)
    return p

def _quick_series_decode_ok(series_path: str) -> bool:
    """Robust, version-agnostic decodability test: actually try to decode ONE slice."""
    try:
        files = iter_dicom_files(series_path)
        if not files:
            return False
        ds = pydicom.dcmread(files[0], force=True)
        _ = ds.pixel_array  # raises if unsupported
        return True
    except Exception:
        return False

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        keep_mask.append(_quick_series_decode_ok(sp))
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIPDataset(Dataset):
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss:
            raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)
        modality_hint = row.get("Modality", None)

        try:
            mips = make_triplanar_mips_stream(sp, modality_hint, self.out_size)
        except Exception:
            # Last-resort fallback to keep training running
            mips = np.zeros((3, self.out_size, self.out_size), dtype=np.float32)

        x = torch.from_numpy(mips).float()
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

# =============================================================================
# MODEL / METRICS
# =============================================================================
def build_efficientnet_b0(n_out=14) -> nn.Module:
    try:
        model = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
    except Exception:
        model = torchvision.models.efficientnet_b0(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, n_out)
    return model

def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_aucs, vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_aucs[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_aucs[c] = auc; vals.append(auc)
        except Exception:
            col_aucs[c] = np.nan
    macro = float(np.mean(vals)) if vals else np.nan
    return col_aucs, macro

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# TRAIN
# =============================================================================
def train_model(args):
    print("== RSNA IA Detection: TRAIN ==")
    set_seed(args.seed, getattr(args, "benchmark_mode", False))
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP}")

    csv_path = os.path.join(args.data_dir, "train.csv")
    if not os.path.exists(csv_path):
        raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    print(f"Total rows in CSV: {len(df):,}")

    train_df, val_df = stratified_split(df, seed=args.seed)

    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    series_dir = os.path.join(args.data_dir, "series")
    print("Scanning decodability (quick test on first slice of each series)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df,   series_dir)

    # Fallback if we somehow kept nothing: use small unfiltered subset
    if len(train_keep) == 0:
        print("⚠️  Decodability filter kept 0 for train — falling back to unfiltered subset.")
        train_keep = train_df.head(64).copy()
    if len(val_keep) == 0:
        print("⚠️  Decodability filter kept 0 for val — falling back to unfiltered subset.")
        val_keep = val_df.head(32).copy()

    print(f"Train: {len(train_keep):,} | Val: {len(val_keep):,}")

    train_ds = MIPDataset(train_keep, series_dir, args.img_size)
    val_ds   = MIPDataset(val_keep,   series_dir, args.img_size)

    tkw = dict(batch_size=args.batch_size, shuffle=True,  num_workers=args.num_workers, drop_last=True)
    vkw = dict(batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    model = build_efficientnet_b0(len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    posw = []
    for c in Config.TARGET_COLS:
        pos = train_keep[c].sum()
        neg = len(train_keep) - pos
        w = min(max((neg / max(1.0, pos)), 1.0), 50.0) if pos > 0 else 1.0
        posw.append(w)
    posw = torch.tensor(posw, device=dev)

    criterion = nn.BCEWithLogitsLoss(pos_weight=posw)
    opt = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=args.epochs)

    use_amp = (dev == "cuda") and Config.USE_AMP
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    best_auc = -1.0
    best_ckpt = args.out

    for epoch in range(args.epochs):
        model.train()
        run_loss = 0.0; correct = 0; total = 0
        t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev), y.to(dev)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=use_amp):
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()

            run_loss += loss.item()
            preds = (torch.sigmoid(out) > 0.5).float()
            correct += (preds == y).sum().item(); total += y.numel()

            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[E{epoch+1} {b}/{len(train_loader)}] "
                      f"loss={run_loss/b:.4f} acc={correct/max(1,total):.3f} "
                      f"lr={sch.get_last_lr()[0]:.2e}")

        train_loss = run_loss / max(1, len(train_loader))
        train_acc  = correct / max(1, total)

        # Val
        model.eval()
        val_loss = 0.0
        preds_all, targs_all = [], []
        with torch.no_grad():
            for x, y, _ in val_loader:
                x, y = x.to(dev), y.to(dev)
                with torch.cuda.amp.autocast(enabled=use_amp):
                    out = model(x); loss = criterion(out, y)
                val_loss += loss.item()
                preds_all.append(torch.sigmoid(out).cpu().numpy())
                targs_all.append(y.cpu().numpy())

        val_loss /= max(1, len(val_loader))
        P = np.vstack(preds_all) if preds_all else np.zeros((0, len(Config.TARGET_COLS)))
        T = np.vstack(targs_all) if targs_all else np.zeros((0, len(Config.TARGET_COLS)))
        _, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)

        dt = time.time() - t0
        print(f"Epoch {epoch+1}/{args.epochs} | {dt:.1f}s  "
              f"train_loss={train_loss:.4f} train_acc={train_acc:.3f}  "
              f"val_loss={val_loss:.4f} macro_auc={macro_auc:.4f}")

        if macro_auc > best_auc:
            best_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": opt.state_dict(),
                "scheduler_state_dict": sch.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
                "config": {
                    "img_size": args.img_size,
                    "batch_size": args.batch_size,
                    "lr": args.lr,
                    "seed": args.seed,
                }
            }, best_ckpt)
            print(f"Saved new best: {best_ckpt} (macro_auc={macro_auc:.4f})")

        sch.step()

    print(f"Training complete. Best macro AUC: {best_auc:.4f} | ckpt={best_ckpt}")
    return best_ckpt

# =============================================================================
# JUPYTER HELPER
# =============================================================================
def train_model_jupyter(data_dir=None, img_size=None, batch_size=None, epochs=None,
                        lr=None, num_workers=None, seed=None, out=None, benchmark_mode=False):
    class A: pass
    a = A()
    a.data_dir = data_dir or Config.DATA_DIR
    a.img_size = img_size or Config.IMG_SIZE
    a.batch_size = batch_size or Config.BATCH_SIZE
    a.epochs = epochs or Config.EPOCHS
    a.lr = lr or Config.LR
    a.num_workers = num_workers if num_workers is not None else Config.NUM_WORKERS
    a.seed = seed or Config.SEED
    a.out = out or "/kaggle/working/best.pt"
    a.benchmark_mode = benchmark_mode
    return train_model(a)

print("Loaded stable script. Now run train_model_jupyter(...) below.")

In [ ]:
train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=288,
    batch_size=2,
    epochs=1,
    num_workers=0,
    benchmark_mode=True
)

# **TEST 3**

using a diffrent image processing for CT scans and also using a much more bigger cifusion matrixes

In [2]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Stable Baseline (Kaggle-ready)

What's in here:
- MRI/MRA  : tri-planar MIPs (axial/coronal/sagittal) → 3 channels
- CT/CTA   : multi-window axial MIP (3 windows) → 3 channels
- Robust DICOM decode (RescaleSlope/Intercept, VOI LUT), quiet & cached
- BCEWithLogits + pos_weight
- Fixed seed, no-leakage split
- Val metrics: per-label AUROC, PR-AUC; per-label confusion at tuned thresholds
- Threshold tuning on val (best F1 per label)
- Saves: /kaggle/working/val_preds.csv and checkpoint with thresholds

Usage after loading:
train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True
)
"""

import os, time, random, warnings, hashlib, math
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

# VOI LUT import (compat across pydicom versions)
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except Exception:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except Exception:
        def apply_voi_lut(x, ds): return x

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    # Kaggle dataset
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    # Train defaults
    IMG_SIZE   = 288
    BATCH_SIZE = 2
    EPOCHS     = 1
    LR         = 2e-4
    NUM_WORKERS= 0
    SEED       = 42
    USE_AMP    = True

    # Logs / cache
    LOG_EVERY_N = 50
    USE_CACHE = True
    CACHE_DIR = "/kaggle/working/mip_cache"
    CACHE_VERSION = "v5.0"

    # DICOM/MIP
    LARGE_SERIES_THRESHOLD = 1   # use streaming path for all (lowest RAM)
    MAX_WORKERS = 1              # keep it simple/stable in Kaggle

    # Multi-window CT (L, W) triplet → 3 channels
    CT_WINDOWS = [(40, 80), (200, 500), (600, 2800)]  # brain, vessel/soft, bone

    # Small subset so one run completes quickly; set both to None to use ALL
    SMOKE_TRAIN_LIMIT = 600
    SMOKE_VAL_LIMIT   = 200

    # Early stopping (optional)
    EARLY_STOP_PATIENCE = None  # set to int for longer runs

    # Labels (13 sites + global)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

# =============================================================================
# UTIL
# =============================================================================
def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# DICOM I/O & NORMALIZATION
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def read_pixel(ds) -> np.ndarray:
    """Decode with VOI LUT + RescaleSlope/Intercept → float32 (HU for CT)."""
    arr = ds.pixel_array
    try:
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            arr = apply_voi_lut(arr, ds)
    except Exception:
        pass
    if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
        arr = arr.astype(np.float32) * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
    else:
        arr = arr.astype(np.float32)
    return arr

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:  # RGB or multi-channel → grayscale
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = float(level) - float(width)/2.0, float(level) + float(width)/2.0
    img = np.clip(img, lo, hi)
    denom = max(1e-6, (hi - lo))
    img = (img - lo) / denom
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# =============================================================================
# MIP BUILDERS (streaming, quiet)
# =============================================================================
def _cache_path(series_path: str, out_size: int, key_extra: str) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{out_size}_{key_extra}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"mip_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if Config.USE_CACHE and os.path.exists(path):
            arr = np.load(path, allow_pickle=False)
            return arr.astype(np.float32) / 255.0
    except Exception:
        pass
    return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, (arr * 255).astype(np.uint8), allow_pickle=False)
    except Exception:
        pass

def make_triplanar_mips_stream(series_path: str, is_ct: bool, out_size: int) -> np.ndarray:
    """
    MRI/MRA path: tri-planar MIPs (max over Z yields axial; stack per-slice maxima for cor/sag).
    If is_ct is True, still usable (not typical); kept for completeness.
    """
    cache = _cache_path(series_path, out_size, key_extra=f"tri_isct{int(is_ct)}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial = None
    cor_rows = None  # (Z, W)
    sag_cols = None  # (Z, H)

    H = W = None
    z = 0
    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)

            if axial is None:
                H, W = img.shape
                axial    = img.copy()
                cor_rows = np.zeros((len(files), W), dtype=np.float32)
                sag_cols = np.zeros((len(files), H), dtype=np.float32)
            else:
                np.maximum(axial, img, out=axial)

            cor_rows[z, :] = img.max(axis=0)
            sag_cols[z, :] = img.max(axis=1)
            z += 1
        except Exception:
            continue

    if axial is None: raise RuntimeError("No decodable slices")

    # Normalize: CT uses CT-like windowing (but tri-planar often for MR)
    if is_ct:
        axial    = window_ct(axial, 200, 500)
        cor_rows = window_ct(cor_rows, 200, 500)
        sag_cols = window_ct(sag_cols, 200, 500)
    else:
        axial    = normalize_mr(axial)
        cor_rows = normalize_mr(cor_rows)
        sag_cols = normalize_mr(sag_cols)

    axial    = cv2.resize(axial,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal  = cv2.resize(cor_rows, (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal = cv2.resize(sag_cols, (out_size, out_size), interpolation=cv2.INTER_AREA)

    mips = np.stack([axial, coronal, sagittal], axis=0).astype(np.float32)
    _save_cache(mips, cache)
    return mips

def make_ct_multiwindow_axial_mip_stream(series_path: str, out_size: int) -> np.ndarray:
    """
    CT/CTA path: build ONE axial MIP in HU, then apply 3 windows → 3 channels.
    """
    # include window triple in cache key
    win_key = "_".join([f"{l}_{w}" for (l,w) in Config.CT_WINDOWS])
    cache = _cache_path(series_path, out_size, key_extra=f"ctmw_{win_key}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial_hu = None
    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)  # HU for CT (after rescale)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)

            if axial_hu is None:
                axial_hu = img.copy()
            else:
                np.maximum(axial_hu, img, out=axial_hu)
        except Exception:
            continue

    if axial_hu is None: raise RuntimeError("No decodable slices")

    channels = []
    for (L, W) in Config.CT_WINDOWS:
        ch = window_ct(axial_hu, level=L, width=W)
        ch = cv2.resize(ch, (out_size, out_size), interpolation=cv2.INTER_AREA)
        channels.append(ch.astype(np.float32))
    mips = np.stack(channels, axis=0)  # 3xHxW
    _save_cache(mips, cache)
    return mips

def _detect_modality_from_header(series_path: str) -> Optional[str]:
    files = iter_dicom_files(series_path)
    if not files: return None
    try:
        ds = pydicom.dcmread(files[0], stop_before_pixels=True, force=True)
        mod = str(getattr(ds, "Modality", "")).upper()
        return mod or None
    except Exception:
        return None

# =============================================================================
# DATASET & FILTERING
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p): raise FileNotFoundError(p)
    return p

def _quick_series_decode_ok(series_path: str) -> bool:
    """Try decoding ONE slice: robust and version-agnostic."""
    try:
        files = iter_dicom_files(series_path)
        if not files: return False
        ds = pydicom.dcmread(files[0], force=True)
        _ = ds.pixel_array
        return True
    except Exception:
        return False

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        keep_mask.append(_quick_series_decode_ok(sp))
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIPDataset(Dataset):
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss:
            raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)

        # modality hint from CSV if present; else detect from header
        mod_hint = row.get("Modality", None)
        if pd.isna(mod_hint) or not str(mod_hint).strip():
            mod_hint = _detect_modality_from_header(sp)
        mod_up = str(mod_hint).upper() if mod_hint is not None else ""

        is_ct = (mod_up in ["CT", "CTA"])

        try:
            if is_ct:
                mips = make_ct_multiwindow_axial_mip_stream(sp, self.out_size)
            else:
                mips = make_triplanar_mips_stream(sp, is_ct=False, out_size=self.out_size)
        except Exception:
            # Last-resort fallback to keep training running
            mips = np.zeros((3, self.out_size, self.out_size), dtype=np.float32)

        x = torch.from_numpy(mips).float()
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

# =============================================================================
# MODEL / METRICS
# =============================================================================
def build_efficientnet_b0(n_out=14) -> nn.Module:
    try:
        model = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
    except Exception:
        model = torchvision.models.efficientnet_b0(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, n_out)
    return model

def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_auc, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_auc[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_auc[c] = float(auc); macro_vals.append(float(auc))
        except Exception:
            col_auc[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_auc, macro

def compute_ap_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_ap, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_ap[c] = np.nan
                continue
            ap = average_precision_score(y_true[:, i], y_prob[:, i])
            col_ap[c] = float(ap); macro_vals.append(float(ap))
        except Exception:
            col_ap[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_ap, macro

def tune_thresholds_best_f1(y_true: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
    """
    Returns per-label thresholds (0..1) maximizing F1 on val.
    """
    n_labels = y_true.shape[1]
    thrs = np.zeros(n_labels, dtype=np.float32)
    grid = np.linspace(0.0, 1.0, 101)
    for i in range(n_labels):
        yi = y_true[:, i].astype(np.int32)
        pi = y_prob[:, i]
        if len(np.unique(yi)) < 2:
            thrs[i] = 0.5
            continue
        best_f1, best_t = -1.0, 0.5
        for t in grid:
            pred = (pi >= t).astype(np.int32)
            tp = int((pred & (yi == 1)).sum())
            fp = int((pred & (yi == 0)).sum())
            fn = int(((1 - pred) & (yi == 1)).sum())
            precision = tp / max(1, (tp + fp))
            recall    = tp / max(1, (tp + fn))
            f1 = 2*precision*recall / max(1e-8, (precision + recall))
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thrs[i] = best_t
    return thrs

def confusion_stats(y_true: np.ndarray, y_prob: np.ndarray, thrs: np.ndarray,
                    cols: List[str], top_k: int = 6) -> pd.DataFrame:
    rows = []
    for i, c in enumerate(cols):
        yi = y_true[:, i].astype(np.int32)
        pi = (y_prob[:, i] >= thrs[i]).astype(np.int32)
        tp = int((pi & (yi == 1)).sum())
        fp = int((pi & (yi == 0)).sum())
        tn = int(((1 - pi) & (yi == 0)).sum())
        fn = int(((1 - pi) & (yi == 1)).sum())
        prec = tp / max(1, (tp + fp))
        rec  = tp / max(1, (tp + fn))
        f1   = 2*prec*rec / max(1e-8, (prec + rec))
        rows.append([c, thrs[i], tp, fp, tn, fn, prec, rec, f1])
    df = pd.DataFrame(rows, columns=["label","thr","TP","FP","TN","FN","precision","recall","F1"])
    # Show a few meaningful rows first (global + common arteries)
    priority = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery",
                "Basilar Tip", "Anterior Communicating Artery"]
    order = priority + [c for c in cols if c not in priority]
    df = df.set_index("label").loc[order].reset_index()
    return df.head(top_k)

# =============================================================================
# TRAIN
# =============================================================================
def train_model(args):
    print("== RSNA IA Detection: TRAIN ==")
    set_seed(args.seed, getattr(args, "benchmark_mode", False))
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP}")

    csv_path = os.path.join(args.data_dir, "train.csv")
    if not os.path.exists(csv_path): raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    print(f"Total rows in CSV: {len(df):,}")

    # Split (no leakage)
    train_df, val_df = stratified_split(df, seed=args.seed)

    # Smoke limits (quick runs). Set both to None to use ALL.
    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df   = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    series_dir = os.path.join(args.data_dir, "series")
    print("Scanning decodability (quick test on first slice of each series)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df,   series_dir)

    # Fallback if filtering keeps 0
    if len(train_keep) == 0:
        print("⚠️  Kept 0 for train — using unfiltered subset.")
        train_keep = train_df.head(64).copy()
    if len(val_keep) == 0:
        print("⚠️  Kept 0 for val — using unfiltered subset.")
        val_keep = val_df.head(32).copy()

    print(f"Train: {len(train_keep):,} | Val: {len(val_keep):,}")

    train_ds = MIPDataset(train_keep, series_dir, args.img_size)
    val_ds   = MIPDataset(val_keep,   series_dir, args.img_size)

    tkw = dict(batch_size=args.batch_size, shuffle=True,  num_workers=args.num_workers, drop_last=True)
    vkw = dict(batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    model = build_efficientnet_b0(len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    # Class imbalance weights
    posw = []
    for c in Config.TARGET_COLS:
        pos = train_keep[c].sum()
        neg = len(train_keep) - pos
        w = min(max((neg / max(1.0, pos)), 1.0), 50.0) if pos > 0 else 1.0
        posw.append(w)
    posw = torch.tensor(posw, device=dev)

    criterion = nn.BCEWithLogitsLoss(pos_weight=posw)
    opt = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=args.epochs)

    use_amp = (dev == "cuda") and Config.USE_AMP
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    best_auc = -1.0
    best_ckpt = args.out
    patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else 1_000_000

    for epoch in range(args.epochs):
        # ----------------- Train ----------------- #
        model.train()
        run_loss = 0.0; correct = 0; total = 0
        t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev), y.to(dev)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=use_amp):
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()

            run_loss += loss.item()
            preds = (torch.sigmoid(out) > 0.5).float()
            correct += (preds == y).sum().item(); total += y.numel()

            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[E{epoch+1} {b}/{len(train_loader)}] "
                      f"loss={run_loss/b:.4f} acc={correct/max(1,total):.3f} "
                      f"lr={sch.get_last_lr()[0]:.2e}")

        train_loss = run_loss / max(1, len(train_loader))
        train_acc  = correct / max(1, total)

        # ----------------- Val ----------------- #
        model.eval()
        val_loss = 0.0
        preds_all, targs_all, uids_all = [], [], []
        with torch.no_grad():
            for x, y, uids in val_loader:
                x, y = x.to(dev), y.to(dev)
                with torch.cuda.amp.autocast(enabled=use_amp):
                    out = model(x); loss = criterion(out, y)
                val_loss += loss.item()
                preds_all.append(torch.sigmoid(out).cpu().numpy())
                targs_all.append(y.cpu().numpy())
                uids_all.extend(list(uids))

        val_loss /= max(1, len(val_loader))
        P = np.vstack(preds_all) if preds_all else np.zeros((0, len(Config.TARGET_COLS)))
        T = np.vstack(targs_all) if targs_all else np.zeros((0, len(Config.TARGET_COLS)))

        col_aucs, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P, Config.TARGET_COLS)

        dt = time.time() - t0
        print(f"Epoch {epoch+1}/{args.epochs} | {dt:.1f}s  "
              f"train_loss={train_loss:.4f} train_acc={train_acc:.3f}  "
              f"val_loss={val_loss:.4f} macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f}")

        # Threshold tuning (val only)
        thrs = tune_thresholds_best_f1(T, P)
        conf_df = confusion_stats(T, P, thrs, Config.TARGET_COLS, top_k=6)
        print("\nVal confusion summary (top labels, tuned thresholds):")
        for _, r in conf_df.iterrows():
            print(f"  {r['label'][:28]:28s} thr={r['thr']:.2f}  TP={r['TP']:3d} FP={r['FP']:3d} "
                  f"TN={r['TN']:3d} FN={r['FN']:3d}  P={r['precision']:.2f} R={r['recall']:.2f} F1={r['F1']:.2f}")
        print("")

        # Save val predictions CSV for inspection
        out_csv = "/kaggle/working/val_preds.csv"
        val_out = pd.DataFrame({"SeriesInstanceUID": uids_all})
        for i, c in enumerate(Config.TARGET_COLS):
            val_out[c + "_prob"] = P[:, i]
            val_out[c + "_true"] = T[:, i]
        val_out.to_csv(out_csv, index=False)
        print(f"Saved validation predictions to {out_csv} (shape={val_out.shape})")

        # Save best checkpoint (stores thresholds too)
        if macro_auc > best_auc:
            best_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": opt.state_dict(),
                "scheduler_state_dict": sch.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
                "thresholds": thrs,  # tuned on val
                "config": {
                    "img_size": args.img_size,
                    "batch_size": args.batch_size,
                    "lr": args.lr,
                    "seed": args.seed,
                    "ct_windows": Config.CT_WINDOWS,
                }
            }, best_ckpt)
            print(f"Saved new best: {best_ckpt} (macro_auc={macro_auc:.4f})")
            patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else patience_left
        else:
            if Config.EARLY_STOP_PATIENCE is not None:
                patience_left -= 1
                print(f"No AUC improvement. Early-stop patience left: {patience_left}")
                if patience_left <= 0:
                    print("Early stopping.")
                    break

        sch.step()

    print(f"Training complete. Best macro AUC: {best_auc:.4f} | ckpt={best_ckpt}")
    return best_ckpt

# =============================================================================
# JUPYTER HELPER
# =============================================================================
def train_model_jupyter(data_dir=None, img_size=None, batch_size=None, epochs=None,
                        lr=None, num_workers=None, seed=None, out=None, benchmark_mode=False):
    class A: pass
    a = A()
    a.data_dir = data_dir or Config.DATA_DIR
    a.img_size = img_size or Config.IMG_SIZE
    a.batch_size = batch_size or Config.BATCH_SIZE
    a.epochs = epochs or Config.EPOCHS
    a.lr = lr or Config.LR
    a.num_workers = num_workers if num_workers is not None else Config.NUM_WORKERS
    a.seed = seed or Config.SEED
    a.out = out or "/kaggle/working/best.pt"
    a.benchmark_mode = benchmark_mode
    return train_model(a)

print("Loaded improved script. Now run train_model_jupyter(...) below, e.g.:")
print('train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True)')

Loaded improved script. Now run train_model_jupyter(...) below, e.g.:
train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True)


In [ ]:
# Balanced run (recommended)
Config.SMOKE_TRAIN_LIMIT = 600   # use the 600/200 smoke split
Config.SMOKE_VAL_LIMIT   = 200
Config.IMG_SIZE          = 288   # good quality, reasonable time
Config.LOG_EVERY_N       = 50    # not too chatty

train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=Config.IMG_SIZE,
    batch_size=2,
    epochs=2,              # 2 passes gives noticeably more signal than 1
    num_workers=0,         # safest/stable on Kaggle for our MIP path
    benchmark_mode=True
)

== RSNA IA Detection: TRAIN ==
Device: cuda | AMP: True
Total rows in CSV: 4,348
Scanning decodability (quick test on first slice of each series)...
Decodable series (quick test): 600/600 kept
Decodable series (quick test): 200/200 kept
Train: 600 | Val: 200


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 144MB/s]


Model params: 4,025,482
[E1 1/300] loss=0.8897 acc=0.500 lr=2.00e-04
[E1 50/300] loss=1.5262 acc=0.763 lr=2.00e-04
[E1 100/300] loss=1.2935 acc=0.786 lr=2.00e-04
[E1 150/300] loss=1.2237 acc=0.793 lr=2.00e-04
[E1 200/300] loss=1.2101 acc=0.780 lr=2.00e-04
[E1 250/300] loss=1.2693 acc=0.756 lr=2.00e-04
[E1 300/300] loss=1.2662 acc=0.722 lr=2.00e-04


In [2]:
 # keep the same smoke limits & size
Config.SMOKE_TRAIN_LIMIT = 600
Config.SMOKE_VAL_LIMIT   = 200
Config.IMG_SIZE          = 288
Config.LOG_EVERY_N       = 50

train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=Config.IMG_SIZE,
    batch_size=2,
    epochs=5,          # bump to 4; with cache this should be quick now
    num_workers=0,
    benchmark_mode=True
)


== RSNA IA Detection: TRAIN ==
Device: cuda | AMP: True
Total rows in CSV: 4,348
Scanning decodability (quick test on first slice of each series)...
Decodable series (quick test): 600/600 kept
Decodable series (quick test): 200/200 kept
Train: 600 | Val: 200


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 131MB/s] 


Model params: 4,025,482
[E1 1/300] loss=0.8896 acc=0.500 lr=2.00e-04
[E1 50/300] loss=1.5262 acc=0.763 lr=2.00e-04
[E1 100/300] loss=1.2941 acc=0.785 lr=2.00e-04
[E1 150/300] loss=1.2252 acc=0.789 lr=2.00e-04
[E1 200/300] loss=1.2138 acc=0.770 lr=2.00e-04
[E1 250/300] loss=1.2674 acc=0.746 lr=2.00e-04
[E1 300/300] loss=1.2615 acc=0.715 lr=2.00e-04
Epoch 1/5 | 4248.5s  train_loss=1.2615 train_acc=0.715  val_loss=1.3004 macro_auc=0.5612 macro_ap=0.1007

Val confusion summary (top labels, tuned thresholds):
  Aneurysm Present             thr=0.41  TP= 96 FP= 93 TN=  9 FN=  2  P=0.51 R=0.98 F1=0.67
  Left Middle Cerebral Artery  thr=0.55  TP=  5 FP= 45 TN=145 FN=  5  P=0.10 R=0.50 F1=0.17
  Right Middle Cerebral Artery thr=0.45  TP= 17 FP=122 TN= 57 FN=  4  P=0.12 R=0.81 F1=0.21
  Basilar Tip                  thr=0.72  TP=  1 FP=  1 TN=195 FN=  3  P=0.50 R=0.25 F1=0.33
  Anterior Communicating Arter thr=0.56  TP=  5 FP= 34 TN=152 FN=  9  P=0.13 R=0.36 F1=0.19
  Left Infraclinoid Internal C

'/kaggle/working/best.pt'

In [ ]:
Config.SMOKE_TRAIN_LIMIT = None
Config.SMOKE_VAL_LIMIT   = None
Config.IMG_SIZE          = 288
Config.LOG_EVERY_N       = 200  # quieter logs

train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=Config.IMG_SIZE,
    batch_size=2,
    epochs=2,          # keep it short to avoid overfit
    num_workers=0,
    benchmark_mode=True
)

== RSNA IA Detection: TRAIN ==
Device: cuda | AMP: True
Total rows in CSV: 4,348
Scanning decodability (quick test on first slice of each series)...
Decodable series (quick test): 3478/3478 kept
Decodable series (quick test): 870/870 kept
Train: 3,478 | Val: 870


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 133MB/s] 


Model params: 4,025,482
[E1 1/1739] loss=1.0247 acc=0.464 lr=2.00e-04
[E1 200/1739] loss=1.3171 acc=0.703 lr=2.00e-04
[E1 400/1739] loss=1.3034 acc=0.608 lr=2.00e-04
[E1 600/1739] loss=1.3061 acc=0.568 lr=2.00e-04
[E1 800/1739] loss=1.2708 acc=0.574 lr=2.00e-04
[E1 1000/1739] loss=1.2677 acc=0.582 lr=2.00e-04
[E1 1200/1739] loss=1.2543 acc=0.582 lr=2.00e-04


# **TEST4**

In [4]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Stable Baseline + Bone-aware Highlight (Kaggle-ready)

- MRI/MRA  : tri-planar MIPs (axial/coronal/sagittal) → highlight stretch (all pixels)
- CT/CTA   : multi-window axial MIP (3 windows: brain/soft/bone) → bone-aware highlight
             (highlight applied everywhere EXCEPT where a bone mask says "bone")
- Robust DICOM decode (VOI LUT + RescaleSlope/Intercept), quiet & cached
- BCEWithLogits + pos_weight
- Fixed seed, no-leakage split
- Val metrics: per-label AUROC, PR-AUC; per-label confusion at tuned thresholds
- Threshold tuning on val (best F1 per label)
- Saves: /kaggle/working/val_preds.csv and best checkpoint with tuned thresholds

Run example:
train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True
)
"""

import os, time, random, warnings, hashlib
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

# VOI LUT import (compat across pydicom versions)
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except Exception:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except Exception:
        def apply_voi_lut(x, ds): return x

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    # Kaggle dataset
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    # Train defaults
    IMG_SIZE   = 288
    BATCH_SIZE = 2
    EPOCHS     = 1
    LR         = 2e-4
    NUM_WORKERS= 0
    SEED       = 42
    USE_AMP    = True

    # Logs / cache
    LOG_EVERY_N = 50
    USE_CACHE = True
    CACHE_DIR = "/kaggle/working/mip_cache"
    CACHE_VERSION = "v6.0"

    # DICOM/MIP (streaming always → lowest RAM)
    LARGE_SERIES_THRESHOLD = 1
    MAX_WORKERS = 1

    # Multi-window CT (L, W) triplet → 3 channels
    CT_WINDOWS = [(40, 80), (200, 500), (600, 2800)]  # brain, soft/vessel, bone

    # Bone-aware highlight controls for CT
    CT_BONE_MASK_HU = 700          # HU above this is considered bone
    CT_HIGHLIGHT_SKIP_BONE = True  # do not highlight pixels flagged as bone

    # Global highlight controls (applied after normalization/windowing)
    HIGHLIGHT_ENABLE     = True    # turn highlight on
    HIGHLIGHT_TOP_PCT    = 1.0     # brighten top % pixels (0..100)
    HIGHLIGHT_GAMMA      = 0.7     # <1 brightens highlights more
    HIGHLIGHT_BLEND      = 0.6     # 0=orig, 1=full highlight
    HIGHLIGHT_ONLY_NON_CT= False   # if True, never highlight CT; we keep False because we skip bone anyway

    # Small subset so one run completes quickly; set both to None to use ALL
    SMOKE_TRAIN_LIMIT = 600
    SMOKE_VAL_LIMIT   = 200

    # Early stopping (optional)
    EARLY_STOP_PATIENCE = None  # set to int for longer runs

    # Labels (13 sites + global)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

# =============================================================================
# UTIL
# =============================================================================
def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# DICOM I/O & NORMALIZATION
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def read_pixel(ds) -> np.ndarray:
    """Decode with VOI LUT + RescaleSlope/Intercept → float32 (HU for CT)."""
    arr = ds.pixel_array
    try:
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            arr = apply_voi_lut(arr, ds)
    except Exception:
        pass
    if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
        arr = arr.astype(np.float32) * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
    else:
        arr = arr.astype(np.float32)
    return arr

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:  # RGB or multi-channel → grayscale
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = float(level) - float(width)/2.0, float(level) + float(width)/2.0
    img = np.clip(img, lo, hi)
    denom = max(1e-6, (hi - lo))
    img = (img - lo) / denom
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# =============================================================================
# HIGHLIGHT STRETCH
# =============================================================================
def highlight_stretch(img01: np.ndarray,
                      pct: float = None,
                      gamma: float = None,
                      blend: float = None) -> np.ndarray:
    """
    Brighten only the top `pct` percent of pixels in [0..1], with gamma and blend.
    Below the percentile, pixels remain unchanged (smooth, one-sided stretch).
    """
    pct   = Config.HIGHLIGHT_TOP_PCT if pct is None else pct
    gamma = Config.HIGHLIGHT_GAMMA   if gamma is None else gamma
    blend = Config.HIGHLIGHT_BLEND   if blend is None else blend

    img = img01.astype(np.float32)
    img = np.clip(img, 0.0, 1.0)

    if pct <= 0.0:
        return img.copy()

    thr = np.percentile(img, 100.0 - pct)
    # If image is flat or threshold at top, do nothing
    if not np.isfinite(thr) or thr >= 0.999:
        return img.copy()

    out = img.copy()
    mask = img >= thr
    # stretch the tail to [0..1]
    denom = max(1e-6, 1.0 - thr)
    hi = (img - thr) / denom
    # gamma shaping (gamma<1 -> brighter)
    if gamma > 0:
        hi = np.power(hi, 1.0 / gamma)
    # blend tail with original (keep continuity at thr)
    out[mask] = (1.0 - blend) * img[mask] + blend * hi[mask]
    return np.clip(out, 0.0, 1.0)

# =============================================================================
# MIP BUILDERS (streaming, quiet)
# =============================================================================
def _cache_path(series_path: str, out_size: int, key_extra: str) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    hl = f"hl{int(Config.HIGHLIGHT_ENABLE)}_{Config.HIGHLIGHT_TOP_PCT:.2f}_{Config.HIGHLIGHT_GAMMA:.2f}_{Config.HIGHLIGHT_BLEND:.2f}"
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{out_size}_{key_extra}_{hl}_skipbone{int(Config.CT_HIGHLIGHT_SKIP_BONE)}_{Config.CT_BONE_MASK_HU}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"mip_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if Config.USE_CACHE and os.path.exists(path):
            arr = np.load(path, allow_pickle=False)
            return arr.astype(np.float32) / 255.0
    except Exception:
        pass
    return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, (arr * 255).astype(np.uint8), allow_pickle=False)
    except Exception:
        pass

def make_triplanar_mips_stream(series_path: str, is_ct: bool, out_size: int) -> np.ndarray:
    """
    MRI/MRA path: tri-planar MIPs; apply highlight stretch on each plane (no bone mask).
    If is_ct is True, still usable; not the default for CT.
    """
    cache = _cache_path(series_path, out_size, key_extra=f"tri_isct{int(is_ct)}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial = None
    cor_rows = None  # (Z, W)
    sag_cols = None  # (Z, H)

    H = W = None
    z = 0
    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)

            if axial is None:
                H, W = img.shape
                axial    = img.copy()
                cor_rows = np.zeros((len(files), W), dtype=np.float32)
                sag_cols = np.zeros((len(files), H), dtype=np.float32)
            else:
                np.maximum(axial, img, out=axial)

            cor_rows[z, :] = img.max(axis=0)
            sag_cols[z, :] = img.max(axis=1)
            z += 1
        except Exception:
            continue

    if axial is None: raise RuntimeError("No decodable slices")

    # Normalize
    if is_ct:
        axial    = window_ct(axial, 200, 500)
        cor_rows = window_ct(cor_rows, 200, 500)
        sag_cols = window_ct(sag_cols, 200, 500)
    else:
        axial    = normalize_mr(axial)
        cor_rows = normalize_mr(cor_rows)
        sag_cols = normalize_mr(sag_cols)

    # Resize
    axial    = cv2.resize(axial,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal  = cv2.resize(cor_rows, (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal = cv2.resize(sag_cols, (out_size, out_size), interpolation=cv2.INTER_AREA)

    # Highlight (MRI/MRA: no bone mask)
    do_highlight = Config.HIGHLIGHT_ENABLE and (not (Config.HIGHLIGHT_ONLY_NON_CT and is_ct))
    if do_highlight:
        axial    = highlight_stretch(axial)
        coronal  = highlight_stretch(coronal)
        sagittal = highlight_stretch(sagittal)

    mips = np.stack([axial, coronal, sagittal], axis=0).astype(np.float32)
    _save_cache(mips, cache)
    return mips

def make_ct_multiwindow_axial_mip_stream(series_path: str, out_size: int) -> np.ndarray:
    """
    CT/CTA path:
      1) Build axial MIP in HU
      2) Build axial *bone mask* MIP from HU threshold (CT_BONE_MASK_HU)
      3) Apply three CT windows to axial HU → 3 channels
      4) Apply highlight on each channel, but SKIP pixels where bone mask==1
    """
    win_key = "_".join([f"{l}_{w}" for (l,w) in Config.CT_WINDOWS])
    cache = _cache_path(series_path, out_size, key_extra=f"ctmw_{win_key}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial_hu = None
    axial_bone = None  # bone mask MIP

    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)  # after rescale → HU
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)  # float HU

            # Build intensity axial HU MIP
            if axial_hu is None:
                axial_hu = img.copy()
            else:
                np.maximum(axial_hu, img, out=axial_hu)

            # Build bone MIP from per-slice threshold
            bone_slice = (img > float(Config.CT_BONE_MASK_HU)).astype(np.uint8)
            if axial_bone is None:
                axial_bone = bone_slice.copy()
            else:
                np.maximum(axial_bone, bone_slice, out=axial_bone)
        except Exception:
            continue

    if axial_hu is None: raise RuntimeError("No decodable slices")

    # Resize bone mask to output size (nearest preserves binary)
    bone_mip = cv2.resize(axial_bone, (out_size, out_size), interpolation=cv2.INTER_NEAREST)

    # Create 3 windowed channels in [0..1]
    channels = []
    for (L, W) in Config.CT_WINDOWS:
        ch = window_ct(axial_hu, level=L, width=W)  # 0..1
        ch = cv2.resize(ch, (out_size, out_size), interpolation=cv2.INTER_AREA)
        channels.append(ch.astype(np.float32))

    # Highlight handling for CT
    do_highlight = Config.HIGHLIGHT_ENABLE and (not Config.HIGHLIGHT_ONLY_NON_CT)
    if do_highlight:
        if Config.CT_HIGHLIGHT_SKIP_BONE:
            # Apply highlight only where bone_mip == 0
            inv_bone = (bone_mip == 0)
            for i in range(len(channels)):
                hi = highlight_stretch(channels[i])
                # keep original in bone pixels, apply highlight elsewhere
                channels[i] = np.where(inv_bone, hi, channels[i])
        else:
            # Highlight everywhere (may brighten bone too)
            for i in range(len(channels)):
                channels[i] = highlight_stretch(channels[i])

    mips = np.stack(channels, axis=0)  # 3xHxW
    _save_cache(mips, cache)
    return mips

def _detect_modality_from_header(series_path: str) -> Optional[str]:
    files = iter_dicom_files(series_path)
    if not files: return None
    try:
        ds = pydicom.dcmread(files[0], stop_before_pixels=True, force=True)
        mod = str(getattr(ds, "Modality", "")).upper()
        return mod or None
    except Exception:
        return None

# =============================================================================
# DATASET & FILTERING
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p): raise FileNotFoundError(p)
    return p

def _quick_series_decode_ok(series_path: str) -> bool:
    """Try decoding ONE slice: robust and version-agnostic."""
    try:
        files = iter_dicom_files(series_path)
        if not files: return False
        ds = pydicom.dcmread(files[0], force=True)
        _ = ds.pixel_array
        return True
    except Exception:
        return False

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        keep_mask.append(_quick_series_decode_ok(sp))
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIPDataset(Dataset):
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss:
            raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)

        # modality hint from CSV if present; else detect from header
        mod_hint = row.get("Modality", None)
        if pd.isna(mod_hint) or not str(mod_hint).strip():
            mod_hint = _detect_modality_from_header(sp)
        mod_up = str(mod_hint).upper() if mod_hint is not None else ""

        is_ct = (mod_up in ["CT", "CTA"])

        try:
            if is_ct:
                mips = make_ct_multiwindow_axial_mip_stream(sp, self.out_size)
            else:
                mips = make_triplanar_mips_stream(sp, is_ct=False, out_size=self.out_size)
        except Exception:
            # Last-resort fallback to keep training running
            mips = np.zeros((3, self.out_size, self.out_size), dtype=np.float32)

        x = torch.from_numpy(mips).float()
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

# =============================================================================
# MODEL / METRICS
# =============================================================================
def build_efficientnet_b0(n_out=14) -> nn.Module:
    try:
        model = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
    except Exception:
        model = torchvision.models.efficientnet_b0(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, n_out)
    return model

def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_auc, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_auc[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_auc[c] = float(auc); macro_vals.append(float(auc))
        except Exception:
            col_auc[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_auc, macro

def compute_ap_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_ap, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_ap[c] = np.nan
                continue
            ap = average_precision_score(y_true[:, i], y_prob[:, i])
            col_ap[c] = float(ap); macro_vals.append(float(ap))
        except Exception:
            col_ap[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_ap, macro

def tune_thresholds_best_f1(y_true: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
    """
    Returns per-label thresholds (0..1) maximizing F1 on val.
    """
    n_labels = y_true.shape[1]
    thrs = np.zeros(n_labels, dtype=np.float32)
    grid = np.linspace(0.0, 1.0, 101)
    for i in range(n_labels):
        yi = y_true[:, i].astype(np.int32)
        pi = y_prob[:, i]
        if len(np.unique(yi)) < 2:
            thrs[i] = 0.5
            continue
        best_f1, best_t = -1.0, 0.5
        for t in grid:
            pred = (pi >= t).astype(np.int32)
            tp = int((pred & (yi == 1)).sum())
            fp = int((pred & (yi == 0)).sum())
            fn = int(((1 - pred) & (yi == 1)).sum())
            precision = tp / max(1, (tp + fp))
            recall    = tp / max(1, (tp + fn))
            f1 = 2*precision*recall / max(1e-8, (precision + recall))
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thrs[i] = best_t
    return thrs

def confusion_stats(y_true: np.ndarray, y_prob: np.ndarray, thrs: np.ndarray,
                    cols: List[str], top_k: int = 6) -> pd.DataFrame:
    rows = []
    for i, c in enumerate(cols):
        yi = y_true[:, i].astype(np.int32)
        pi = (y_prob[:, i] >= thrs[i]).astype(np.int32)
        tp = int((pi & (yi == 1)).sum())
        fp = int((pi & (yi == 0)).sum())
        tn = int(((1 - pi) & (yi == 0)).sum())
        fn = int(((1 - pi) & (yi == 1)).sum())
        prec = tp / max(1, (tp + fp))
        rec  = tp / max(1, (tp + fn))
        f1   = 2*prec*rec / max(1e-8, (prec + rec))
        rows.append([c, thrs[i], tp, fp, tn, fn, prec, rec, f1])
    df = pd.DataFrame(rows, columns=["label","thr","TP","FP","TN","FN","precision","recall","F1"])
    priority = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery",
                "Basilar Tip", "Anterior Communicating Artery"]
    order = priority + [c for c in cols if c not in priority]
    df = df.set_index("label").loc[order].reset_index()
    return df.head(top_k)

# =============================================================================
# TRAIN
# =============================================================================
def train_model(args):
    print("== RSNA IA Detection: TRAIN ==")
    set_seed(args.seed, getattr(args, "benchmark_mode", False))
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP}")

    csv_path = os.path.join(args.data_dir, "train.csv")
    if not os.path.exists(csv_path): raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    print(f"Total rows in CSV: {len(df):,}")

    # Split (no leakage)
    train_df, val_df = stratified_split(df, seed=args.seed)

    # Smoke limits (quick runs). Set both to None to use ALL.
    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df   = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    series_dir = os.path.join(args.data_dir, "series")
    print("Scanning decodability (quick test on first slice of each series)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df,   series_dir)

    # Fallback if filtering keeps 0
    if len(train_keep) == 0:
        print("⚠️  Kept 0 for train — using unfiltered subset.")
        train_keep = train_df.head(64).copy()
    if len(val_keep) == 0:
        print("⚠️  Kept 0 for val — using unfiltered subset.")
        val_keep = val_df.head(32).copy()

    print(f"Train: {len(train_keep):,} | Val: {len(val_keep):,}")

    train_ds = MIPDataset(train_keep, series_dir, args.img_size)
    val_ds   = MIPDataset(val_keep,   series_dir, args.img_size)

    tkw = dict(batch_size=args.batch_size, shuffle=True,  num_workers=args.num_workers, drop_last=True)
    vkw = dict(batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    model = build_efficientnet_b0(len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    # Class imbalance weights
    posw = []
    for c in Config.TARGET_COLS:
        pos = train_keep[c].sum()
        neg = len(train_keep) - pos
        w = min(max((neg / max(1.0, pos)), 1.0), 50.0) if pos > 0 else 1.0
        posw.append(w)
    posw = torch.tensor(posw, device=dev)

    criterion = nn.BCEWithLogitsLoss(pos_weight=posw)
    opt = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=args.epochs)

    use_amp = (dev == "cuda") and Config.USE_AMP
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    best_auc = -1.0
    best_ckpt = args.out
    patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else 1_000_000

    for epoch in range(args.epochs):
        # ----------------- Train ----------------- #
        model.train()
        run_loss = 0.0; correct = 0; total = 0
        t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev), y.to(dev)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=use_amp):
                out = model(x)
                loss = criterion(out, y)
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()

            run_loss += loss.item()
            preds = (torch.sigmoid(out) > 0.5).float()
            correct += (preds == y).sum().item(); total += y.numel()

            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[E{epoch+1} {b}/{len(train_loader)}] "
                      f"loss={run_loss/b:.4f} acc={correct/max(1,total):.3f} "
                      f"lr={sch.get_last_lr()[0]:.2e}")

        train_loss = run_loss / max(1, len(train_loader))
        train_acc  = correct / max(1, total)

        # ----------------- Val ----------------- #
        model.eval()
        val_loss = 0.0
        preds_all, targs_all, uids_all = [], [], []
        with torch.no_grad():
            for x, y, uids in val_loader:
                x, y = x.to(dev), y.to(dev)
                with torch.cuda.amp.autocast(enabled=use_amp):
                    out = model(x); loss = criterion(out, y)
                val_loss += loss.item()
                preds_all.append(torch.sigmoid(out).cpu().numpy())
                targs_all.append(y.cpu().numpy())
                uids_all.extend(list(uids))

        val_loss /= max(1, len(val_loader))
        P = np.vstack(preds_all) if preds_all else np.zeros((0, len(Config.TARGET_COLS)))
        T = np.vstack(targs_all) if targs_all else np.zeros((0, len(Config.TARGET_COLS)))

        col_aucs, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P, Config.TARGET_COLS)

        dt = time.time() - t0
        print(f"Epoch {epoch+1}/{args.epochs} | {dt:.1f}s  "
              f"train_loss={train_loss:.4f} train_acc={train_acc:.3f}  "
              f"val_loss={val_loss:.4f} macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f}")

        # Threshold tuning (val only)
        thrs = tune_thresholds_best_f1(T, P)
        conf_df = confusion_stats(T, P, thrs, Config.TARGET_COLS, top_k=6)
        print("\nVal confusion summary (top labels, tuned thresholds):")
        for _, r in conf_df.iterrows():
            print(f"  {r['label'][:28]:28s} thr={r['thr']:.2f}  TP={r['TP']:3d} FP={r['FP']:3d} "
                  f"TN={r['TN']:3d} FN={r['FN']:3d}  P={r['precision']:.2f} R={r['recall']:.2f} F1={r['F1']:.2f}")
        print("")

        # Save val predictions CSV for inspection
        out_csv = "/kaggle/working/val_preds.csv"
        val_out = pd.DataFrame({"SeriesInstanceUID": uids_all})
        for i, c in enumerate(Config.TARGET_COLS):
            val_out[c + "_prob"] = P[:, i]
            val_out[c + "_true"] = T[:, i]
        val_out.to_csv(out_csv, index=False)
        print(f"Saved validation predictions to {out_csv} (shape={val_out.shape})")

        # Save best checkpoint (stores thresholds too)
        if macro_auc > best_auc:
            best_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": opt.state_dict(),
                "scheduler_state_dict": sch.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
                "thresholds": thrs,  # tuned on val
                "config": {
                    "img_size": args.img_size,
                    "batch_size": args.batch_size,
                    "lr": args.lr,
                    "seed": args.seed,
                    "ct_windows": Config.CT_WINDOWS,
                    "highlight": {
                        "enable": Config.HIGHLIGHT_ENABLE,
                        "pct": Config.HIGHLIGHT_TOP_PCT,
                        "gamma": Config.HIGHLIGHT_GAMMA,
                        "blend": Config.HIGHLIGHT_BLEND,
                        "ct_skip_bone": Config.CT_HIGHLIGHT_SKIP_BONE,
                        "bone_hu": Config.CT_BONE_MASK_HU,
                    }
                }
            }, best_ckpt)
            print(f"Saved new best: {best_ckpt} (macro_auc={macro_auc:.4f})")
            patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else patience_left
        else:
            if Config.EARLY_STOP_PATIENCE is not None:
                patience_left -= 1
                print(f"No AUC improvement. Early-stop patience left: {patience_left}")
                if patience_left <= 0:
                    print("Early stopping.")
                    break

        sch.step()

    print(f"Training complete. Best macro AUC: {best_auc:.4f} | ckpt={best_ckpt}")
    return best_ckpt

# =============================================================================
# JUPYTER HELPER
# =============================================================================
def train_model_jupyter(data_dir=None, img_size=None, batch_size=None, epochs=None,
                        lr=None, num_workers=None, seed=None, out=None, benchmark_mode=False):
    class A: pass
    a = A()
    a.data_dir = data_dir or Config.DATA_DIR
    a.img_size = img_size or Config.IMG_SIZE
    a.batch_size = batch_size or Config.BATCH_SIZE
    a.epochs = epochs or Config.EPOCHS
    a.lr = lr or Config.LR
    a.num_workers = num_workers if num_workers is not None else Config.NUM_WORKERS
    a.seed = seed or Config.SEED
    a.out = out or "/kaggle/working/best.pt"
    a.benchmark_mode = benchmark_mode
    return train_model(a)

print("Loaded improved script (bone-aware highlight). Example:")
print('train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True)')

Loaded improved script (bone-aware highlight). Example:
train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=1, num_workers=0, benchmark_mode=True)


In [5]:
# --- Recommended baseline ---
Config.SMOKE_TRAIN_LIMIT = 600
Config.SMOKE_VAL_LIMIT   = 200
Config.IMG_SIZE          = 288
Config.LOG_EVERY_N       = 50
Config.USE_CACHE         = True

# bone-aware highlight (ON)
Config.HIGHLIGHT_CT      = True
Config.BONE_SUPPRESS     = True
Config.BONE_THRESH_HU    = 300
Config.BONE_MORPH_K      = 3
Config.HIGHLIGHT_MRI     = True

train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=Config.IMG_SIZE,
    batch_size=2,
    epochs=2,        # you can use 2–3 once stable
    num_workers=0,
    benchmark_mode=True
)

== RSNA IA Detection: TRAIN ==
Device: cuda | AMP: True
Total rows in CSV: 4,348
Scanning decodability (quick test on first slice of each series)...
Decodable series (quick test): 600/600 kept
Decodable series (quick test): 200/200 kept
Train: 600 | Val: 200


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 141MB/s]


Model params: 4,025,482
[E1 1/300] loss=0.8897 acc=0.500 lr=2.00e-04
[E1 50/300] loss=1.5267 acc=0.761 lr=2.00e-04
[E1 100/300] loss=1.2954 acc=0.790 lr=2.00e-04
[E1 150/300] loss=1.2250 acc=0.790 lr=2.00e-04
[E1 200/300] loss=1.2158 acc=0.764 lr=2.00e-04
[E1 250/300] loss=1.2630 acc=0.744 lr=2.00e-04
[E1 300/300] loss=1.2614 acc=0.710 lr=2.00e-04
Epoch 1/2 | 4563.1s  train_loss=1.2614 train_acc=0.710  val_loss=1.3187 macro_auc=0.5604 macro_ap=0.1066

Val confusion summary (top labels, tuned thresholds):
  Aneurysm Present             thr=0.38  TP= 98 FP= 98 TN=  4 FN=  0  P=0.50 R=1.00 F1=0.67
  Left Middle Cerebral Artery  thr=0.52  TP=  9 FP=134 TN= 56 FN=  1  P=0.06 R=0.90 F1=0.12
  Right Middle Cerebral Artery thr=0.40  TP= 20 FP=159 TN= 20 FN=  1  P=0.11 R=0.95 F1=0.20
  Basilar Tip                  thr=0.67  TP=  2 FP=  6 TN=190 FN=  2  P=0.25 R=0.50 F1=0.33
  Anterior Communicating Arter thr=0.58  TP=  3 FP=  6 TN=180 FN= 11  P=0.33 R=0.21 F1=0.26
  Left Infraclinoid Internal C

'/kaggle/working/best.pt'

In [1]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Baseline (patched)
- MRI/MRA  : tri-planar MIPs (axial/coronal/sagittal) → highlight stretch
- CT/CTA   : multi-window axial MIP (brain/soft/bone) → bone-aware highlight with morphology
- DICOM decode (VOI LUT + Rescale) robust, quiet & cached
- Loss: BCEWithLogits + Focal (γ=2) with per-class pos_weight
- Class-balanced sampler (rare positives appear regularly)
- Optional consistency penalty: locations ≤ "Aneurysm Present"
- Metrics: per-label AUC/AP, macro AUC/AP, micro AP
- Per-label threshold tuning (max F1), confusion summary
- Saves: /kaggle/working/val_preds.csv and best checkpoint

Example:
train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=288, batch_size=2, epochs=2, num_workers=0, benchmark_mode=True
)
"""

import os, time, random, warnings, hashlib
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

# VOI LUT import (compat across pydicom versions)
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except Exception:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except Exception:
        def apply_voi_lut(x, ds): return x

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    # Kaggle dataset
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    # Train defaults
    IMG_SIZE   = 288
    BATCH_SIZE = 2
    EPOCHS     = 1
    LR         = 2e-4
    NUM_WORKERS= 0
    SEED       = 42
    USE_AMP    = True

    # Logs / cache
    LOG_EVERY_N = 50
    USE_CACHE = True
    CACHE_DIR = "/kaggle/working/mip_cache"
    CACHE_VERSION = "v7.0"

    # DICOM/MIP (streaming always → lowest RAM)
    LARGE_SERIES_THRESHOLD = 1
    MAX_WORKERS = 1

    # Multi-window CT (L, W) triplet → 3 channels
    CT_WINDOWS = [(40, 80), (200, 500), (600, 2800)]  # brain, soft/vessel, bone

    # Highlight master switches
    HIGHLIGHT_ENABLE      = True
    HIGHLIGHT_ONLY_NON_CT = False  # keep False; we skip bone anyway
    HIGHLIGHT_TOP_PCT     = 1.0
    HIGHLIGHT_GAMMA       = 0.7
    HIGHLIGHT_BLEND       = 0.6

    # CT bone-aware highlight
    CT_HIGHLIGHT_SKIP_BONE = True
    CT_BONE_MASK_HU        = 300      # intended lower threshold
    CT_BONE_MORPH_K        = 3        # morphology kernel (0 disables)

    # Small subset so one run completes quickly; set both to None to use ALL
    SMOKE_TRAIN_LIMIT = 600
    SMOKE_VAL_LIMIT   = 200

    # Early stopping (optional)
    EARLY_STOP_PATIENCE = None  # set to int for longer runs

    # Consistency penalty (locations shouldn't exceed "Present")
    CONSISTENCY_W = 0.2  # set 0.0 to disable

    # Labels (13 sites + global)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

# ImageNet normalization (works fine even though channels ≠ RGB semantics)
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)

# =============================================================================
# UTIL
# =============================================================================
def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# DICOM I/O & NORMALIZATION
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def read_pixel(ds) -> np.ndarray:
    """Decode with VOI LUT + RescaleSlope/Intercept → float32 (HU for CT)."""
    arr = ds.pixel_array
    try:
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            arr = apply_voi_lut(arr, ds)
    except Exception:
        pass
    if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
        arr = arr.astype(np.float32) * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
    else:
        arr = arr.astype(np.float32)
    return arr

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:  # RGB or multi-channel → grayscale
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = float(level) - float(width)/2.0, float(level) + float(width)/2.0
    img = np.clip(img, lo, hi)
    denom = max(1e-6, (hi - lo))
    img = (img - lo) / denom
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# =============================================================================
# HIGHLIGHT STRETCH
# =============================================================================
def highlight_stretch(img01: np.ndarray,
                      pct: float = None,
                      gamma: float = None,
                      blend: float = None) -> np.ndarray:
    """
    Brighten only the top `pct` percent of pixels in [0..1], with gamma and blend.
    Below the percentile, pixels remain unchanged (smooth, one-sided stretch).
    """
    pct   = Config.HIGHLIGHT_TOP_PCT if pct is None else pct
    gamma = Config.HIGHLIGHT_GAMMA   if gamma is None else gamma
    blend = Config.HIGHLIGHT_BLEND   if blend is None else blend

    img = img01.astype(np.float32)
    img = np.clip(img, 0.0, 1.0)

    if pct <= 0.0:
        return img.copy()

    thr = np.percentile(img, 100.0 - pct)
    # If image is flat or threshold at top, do nothing
    if not np.isfinite(thr) or thr >= 0.999:
        return img.copy()

    out = img.copy()
    mask = img >= thr
    # stretch the tail to [0..1]
    denom = max(1e-6, 1.0 - thr)
    hi = (img - thr) / denom
    # gamma shaping (gamma<1 -> brighter)
    if gamma > 0:
        hi = np.power(hi, 1.0 / gamma)
    # blend tail with original (keep continuity at thr)
    out[mask] = (1.0 - blend) * img[mask] + blend * hi[mask]
    return np.clip(out, 0.0, 1.0)

# =============================================================================
# MIP BUILDERS (streaming, quiet)
# =============================================================================
def _cache_path(series_path: str, out_size: int, key_extra: str) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    hl = f"hl{int(Config.HIGHLIGHT_ENABLE)}_{Config.HIGHLIGHT_TOP_PCT:.2f}_{Config.HIGHLIGHT_GAMMA:.2f}_{Config.HIGHLIGHT_BLEND:.2f}"
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{out_size}_{key_extra}_{hl}_skipbone{int(Config.CT_HIGHLIGHT_SKIP_BONE)}_{Config.CT_BONE_MASK_HU}_morph{Config.CT_BONE_MORPH_K}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"mip_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if Config.USE_CACHE and os.path.exists(path):
            arr = np.load(path, allow_pickle=False)
            return arr.astype(np.float32) / 255.0
    except Exception:
        pass
    return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, (arr * 255).astype(np.uint8), allow_pickle=False)
    except Exception:
        pass

def make_triplanar_mips_stream(series_path: str, is_ct: bool, out_size: int) -> np.ndarray:
    """
    MRI/MRA path: tri-planar MIPs; apply highlight stretch on each plane (no bone mask).
    If is_ct is True, still usable; not the default for CT.
    """
    cache = _cache_path(series_path, out_size, key_extra=f"tri_isct{int(is_ct)}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial = None
    cor_rows = None  # (Z, W)
    sag_cols = None  # (Z, H)

    H = W = None
    z = 0
    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)

            if axial is None:
                H, W = img.shape
                axial    = img.copy()
                cor_rows = np.zeros((len(files), W), dtype=np.float32)
                sag_cols = np.zeros((len(files), H), dtype=np.float32)
            else:
                np.maximum(axial, img, out=axial)

            cor_rows[z, :] = img.max(axis=0)
            sag_cols[z, :] = img.max(axis=1)
            z += 1
        except Exception:
            continue

    if axial is None: raise RuntimeError("No decodable slices")

    # Normalize
    if is_ct:
        axial    = window_ct(axial, 200, 500)
        cor_rows = window_ct(cor_rows, 200, 500)
        sag_cols = window_ct(sag_cols, 200, 500)
    else:
        axial    = normalize_mr(axial)
        cor_rows = normalize_mr(cor_rows)
        sag_cols = normalize_mr(sag_cols)

    # Resize
    axial    = cv2.resize(axial,    (out_size, out_size), interpolation=cv2.INTER_AREA)
    coronal  = cv2.resize(cor_rows, (out_size, out_size), interpolation=cv2.INTER_AREA)
    sagittal = cv2.resize(sag_cols, (out_size, out_size), interpolation=cv2.INTER_AREA)

    # Highlight (MRI/MRA: no bone mask)
    do_highlight = Config.HIGHLIGHT_ENABLE and (not (Config.HIGHLIGHT_ONLY_NON_CT and is_ct))
    if do_highlight:
        axial    = highlight_stretch(axial)
        coronal  = highlight_stretch(coronal)
        sagittal = highlight_stretch(sagittal)

    mips = np.stack([axial, coronal, sagittal], axis=0).astype(np.float32)
    _save_cache(mips, cache)
    return mips

def _morph_clean(mask: np.ndarray, k: int) -> np.ndarray:
    if k is None or k <= 0: return mask
    kernel = np.ones((k,k), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel)
    return mask

def make_ct_multiwindow_axial_mip_stream(series_path: str, out_size: int) -> np.ndarray:
    """
    CT/CTA path:
      1) Build axial MIP in HU
      2) Build axial *bone mask* MIP from HU threshold (CT_BONE_MASK_HU) + morphology
      3) Apply three CT windows to axial HU → 3 channels
      4) Apply highlight on each channel, but SKIP pixels where bone mask==1
    """
    win_key = "_".join([f"{l}_{w}" for (l,w) in Config.CT_WINDOWS])
    cache = _cache_path(series_path, out_size, key_extra=f"ctmw_{win_key}")
    cached = _load_cache(cache)
    if cached is not None: return cached

    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")

    axial_hu = None
    axial_bone = None  # bone mask MIP

    for f in files:
        try:
            ds  = pydicom.dcmread(f, force=True)
            arr = read_pixel(ds)  # after rescale → HU
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            img = _ensure_gray2d(arr)  # float HU

            # Build intensity axial HU MIP
            if axial_hu is None:
                axial_hu = img.copy()
            else:
                np.maximum(axial_hu, img, out=axial_hu)

            # Build bone MIP from per-slice threshold
            bone_slice = (img > float(Config.CT_BONE_MASK_HU)).astype(np.uint8)
            if axial_bone is None:
                axial_bone = bone_slice.copy()
            else:
                np.maximum(axial_bone, bone_slice, out=axial_bone)
        except Exception:
            continue

    if axial_hu is None: raise RuntimeError("No decodable slices")

    # Resize bone mask to output size and clean
    bone_mip = cv2.resize(axial_bone, (out_size, out_size), interpolation=cv2.INTER_NEAREST)
    bone_mip = _morph_clean(bone_mip.astype(np.uint8), Config.CT_BONE_MORPH_K)

    # Create 3 windowed channels in [0..1]
    channels = []
    for (L, W) in Config.CT_WINDOWS:
        ch = window_ct(axial_hu, level=L, width=W)  # 0..1
        ch = cv2.resize(ch, (out_size, out_size), interpolation=cv2.INTER_AREA)
        channels.append(ch.astype(np.float32))

    # Highlight handling for CT
    do_highlight = Config.HIGHLIGHT_ENABLE and (not Config.HIGHLIGHT_ONLY_NON_CT)
    if do_highlight:
        if Config.CT_HIGHLIGHT_SKIP_BONE:
            inv_bone = (bone_mip == 0)
            for i in range(len(channels)):
                hi = highlight_stretch(channels[i])
                channels[i] = np.where(inv_bone, hi, channels[i])
        else:
            for i in range(len(channels)):
                channels[i] = highlight_stretch(channels[i])

    mips = np.stack(channels, axis=0)  # 3xHxW
    _save_cache(mips, cache)
    return mips

def _detect_modality_from_header(series_path: str) -> Optional[str]:
    files = iter_dicom_files(series_path)
    if not files: return None
    try:
        ds = pydicom.dcmread(files[0], stop_before_pixels=True, force=True)
        mod = str(getattr(ds, "Modality", "")).upper()
        return mod or None
    except Exception:
        return None

# =============================================================================
# DATASET & FILTERING
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p): raise FileNotFoundError(p)
    return p

def _quick_series_decode_ok(series_path: str) -> bool:
    """Try decoding ONE slice: robust and version-agnostic."""
    try:
        files = iter_dicom_files(series_path)
        if not files: return False
        ds = pydicom.dcmread(files[0], force=True)
        _ = ds.pixel_array
        return True
    except Exception:
        return False

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        keep_mask.append(_quick_series_decode_ok(sp))
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIPDataset(Dataset):
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss:
            raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)

        # modality hint from CSV if present; else detect from header
        mod_hint = row.get("Modality", None)
        if pd.isna(mod_hint) or not str(mod_hint).strip():
            mod_hint = _detect_modality_from_header(sp)
        mod_up = str(mod_hint).upper() if mod_hint is not None else ""

        is_ct = (mod_up in ["CT", "CTA"])

        try:
            if is_ct:
                mips = make_ct_multiwindow_axial_mip_stream(sp, self.out_size)
            else:
                mips = make_triplanar_mips_stream(sp, is_ct=False, out_size=self.out_size)
        except Exception:
            # Last-resort fallback to keep training running
            mips = np.zeros((3, self.out_size, self.out_size), dtype=np.float32)

        x = torch.from_numpy(mips).float()       # [3,H,W] in [0,1]
        # ImageNet normalization
        x = (x - IMAGENET_MEAN) / IMAGENET_STD
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

def build_class_balanced_sampler(df: pd.DataFrame, target_cols: List[str]) -> WeightedRandomSampler:
    pos = df[target_cols].sum(0).values
    neg = len(df) - pos
    cls_w = np.where(pos > 0, neg / np.maximum(1, pos), 1.0)
    cls_w = np.clip(cls_w, 1.0, 50.0)
    Y = df[target_cols].values.astype(np.float32)
    w = 1.0 + (Y * cls_w).sum(1)
    w = torch.as_tensor(w, dtype=torch.double)
    sampler = WeightedRandomSampler(weights=w, num_samples=len(df), replacement=True)
    return sampler

# =============================================================================
# MODEL / METRICS
# =============================================================================
def build_efficientnet_b0(n_out=14) -> nn.Module:
    try:
        model = torchvision.models.efficientnet_b0(
            weights=torchvision.models.EfficientNet_B0_Weights.IMAGENET1K_V1
        )
    except Exception:
        model = torchvision.models.efficientnet_b0(pretrained=True)
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, n_out)
    return model

class BCEWithLogitsFocal(nn.Module):
    def __init__(self, pos_weight=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction="none")
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, logits, targets):
        bce = self.bce(logits, targets)              # [B,C]
        p = torch.sigmoid(logits)
        pt = p*targets + (1-p)*(1-targets)
        focal = (1.0 - pt).clamp_min(1e-6).pow(self.gamma)
        loss = focal * bce
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        return loss

def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_auc, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_auc[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_auc[c] = float(auc); macro_vals.append(float(auc))
        except Exception:
            col_auc[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_auc, macro

def compute_ap_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_ap, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_ap[c] = np.nan
                continue
            ap = average_precision_score(y_true[:, i], y_prob[:, i])
            col_ap[c] = float(ap); macro_vals.append(float(ap))
        except Exception:
            col_ap[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_ap, macro

def compute_micro_ap(y_true: np.ndarray, y_prob: np.ndarray):
    mask = (y_true.sum(0) > 0) & ((1 - y_true).sum(0) > 0)
    if mask.sum() == 0: return np.nan
    yt = y_true[:, mask].ravel()
    yp = y_prob[:, mask].ravel()
    return average_precision_score(yt, yp)

def tune_thresholds_best_f1(y_true: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
    """
    Returns per-label thresholds (0..1) maximizing F1 on val.
    """
    n_labels = y_true.shape[1]
    thrs = np.zeros(n_labels, dtype=np.float32)
    grid = np.linspace(0.0, 1.0, 101)
    for i in range(n_labels):
        yi = y_true[:, i].astype(np.int32)
        pi = y_prob[:, i]
        if len(np.unique(yi)) < 2:
            thrs[i] = 0.5
            continue
        best_f1, best_t = -1.0, 0.5
        for t in grid:
            pred = (pi >= t).astype(np.int32)
            tp = int((pred & (yi == 1)).sum())
            fp = int((pred & (yi == 0)).sum())
            fn = int(((1 - pred) & (yi == 1)).sum())
            precision = tp / max(1, (tp + fp))
            recall    = tp / max(1, (tp + fn))
            f1 = 2*precision*recall / max(1e-8, (precision + recall))
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thrs[i] = best_t
    return thrs

def confusion_stats(y_true: np.ndarray, y_prob: np.ndarray, thrs: np.ndarray,
                    cols: List[str], top_k: int = 6) -> pd.DataFrame:
    rows = []
    for i, c in enumerate(cols):
        yi = y_true[:, i].astype(np.int32)
        pi = (y_prob[:, i] >= thrs[i]).astype(np.int32)
        tp = int((pi & (yi == 1)).sum())
        fp = int((pi & (yi == 0)).sum())
        tn = int(((1 - pi) & (yi == 0)).sum())
        fn = int(((1 - pi) & (yi == 1)).sum())
        prec = tp / max(1, (tp + fp))
        rec  = tp / max(1, (tp + fn))
        f1   = 2*prec*rec / max(1e-8, (prec + rec))
        rows.append([c, thrs[i], tp, fp, tn, fn, prec, rec, f1])
    df = pd.DataFrame(rows, columns=["label","thr","TP","FP","TN","FN","precision","recall","F1"])
    priority = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery",
                "Basilar Tip", "Anterior Communicating Artery"]
    order = priority + [c for c in cols if c not in priority]
    df = df.set_index("label").loc[order].reset_index()
    return df.head(top_k)

# =============================================================================
# TRAIN
# =============================================================================
def train_model(args):
    print("== RSNA IA Detection: TRAIN ==")
    set_seed(args.seed, getattr(args, "benchmark_mode", False))
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP}")

    csv_path = os.path.join(args.data_dir, "train.csv")
    if not os.path.exists(csv_path): raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    print(f"Total rows in CSV: {len(df):,}")

    # Split (no leakage)
    train_df, val_df = stratified_split(df, seed=args.seed)

    # Smoke limits (quick runs). Set both to None to use ALL.
    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df   = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    series_dir = os.path.join(args.data_dir, "series")
    print("Scanning decodability (quick test on first slice of each series)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df,   series_dir)

    # Fallback if filtering keeps 0
    if len(train_keep) == 0:
        print("⚠️  Kept 0 for train — using unfiltered subset.")
        train_keep = train_df.head(64).copy()
    if len(val_keep) == 0:
        print("⚠️  Kept 0 for val — using unfiltered subset.")
        val_keep = val_df.head(32).copy()

    print(f"Train: {len(train_keep):,} | Val: {len(val_keep):,}")

    train_ds = MIPDataset(train_keep, series_dir, args.img_size)
    val_ds   = MIPDataset(val_keep,   series_dir, args.img_size)

    # Class-balanced sampler for train
    train_sampler = build_class_balanced_sampler(train_keep, Config.TARGET_COLS)

    tkw = dict(batch_size=args.batch_size, sampler=train_sampler, num_workers=args.num_workers, drop_last=True)
    vkw = dict(batch_size=args.batch_size, shuffle=False,     num_workers=args.num_workers, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    model = build_efficientnet_b0(len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    # Class imbalance weights (per label)
    posw = []
    for c in Config.TARGET_COLS:
        pos = train_keep[c].sum()
        neg = len(train_keep) - pos
        w = (neg / max(1.0, pos)) if pos > 0 else 1.0
        w = float(np.clip(w, 1.0, 50.0))
        posw.append(w)
    posw = torch.tensor(posw, device=dev)

    criterion = BCEWithLogitsFocal(pos_weight=posw, gamma=2.0)
    opt = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=args.epochs)

    use_amp = (dev == "cuda") and Config.USE_AMP
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

    present_idx = Config.TARGET_COLS.index("Aneurysm Present")
    loc_idx = [i for i,_c in enumerate(Config.TARGET_COLS) if i != present_idx]

    best_auc = -1.0
    best_ckpt = args.out
    patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else 1_000_000

    for epoch in range(args.epochs):
        # ----------------- Train ----------------- #
        model.train()
        run_loss = 0.0
        t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=use_amp):
                out = model(x)
                loss = criterion(out, y)

                # Consistency penalty (locations shouldn't exceed "Present")
                if Config.CONSISTENCY_W > 0:
                    probs = torch.sigmoid(out)
                    present = probs[:, present_idx:present_idx+1]  # [B,1]
                    loc_probs = probs[:, loc_idx]                  # [B,L]
                    consistency = torch.clamp(loc_probs - present, min=0.0).pow(2).mean()
                    loss = loss + Config.CONSISTENCY_W * consistency

            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()

            run_loss += loss.item()

            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[E{epoch+1} {b}/{len(train_loader)}] loss={run_loss/b:.4f} lr={sch.get_last_lr()[0]:.2e}")

        train_loss = run_loss / max(1, len(train_loader))

        # ----------------- Val ----------------- #
        model.eval()
        val_loss = 0.0
        preds_all, targs_all, uids_all = [], [], []
        with torch.no_grad():
            for x, y, uids in val_loader:
                x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=use_amp):
                    out = model(x); loss = criterion(out, y)
                val_loss += loss.item()
                preds_all.append(torch.sigmoid(out).cpu().numpy())
                targs_all.append(y.cpu().numpy())
                uids_all.extend(list(uids))

        val_loss /= max(1, len(val_loader))
        P = np.vstack(preds_all) if preds_all else np.zeros((0, len(Config.TARGET_COLS)))
        T = np.vstack(targs_all) if targs_all else np.zeros((0, len(Config.TARGET_COLS)))

        col_aucs, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P, Config.TARGET_COLS)
        micro_ap = compute_micro_ap(T, P)

        dt = time.time() - t0
        print(f"Epoch {epoch+1}/{args.epochs} | {dt:.1f}s  "
              f"train_loss={train_loss:.4f}  val_loss={val_loss:.4f}  "
              f"macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f} micro_ap={micro_ap:.4f}")

        # Threshold tuning (val only)
        thrs = tune_thresholds_best_f1(T, P)
        conf_df = confusion_stats(T, P, thrs, Config.TARGET_COLS, top_k=6)
        print("\nVal confusion summary (top labels, tuned thresholds):")
        for _, r in conf_df.iterrows():
            print(f"  {r['label'][:28]:28s} thr={r['thr']:.2f}  TP={r['TP']:3d} FP={r['FP']:3d} "
                  f"TN={r['TN']:3d} FN={r['FN']:3d}  P={r['precision']:.2f} R={r['recall']:.2f} F1={r['F1']:.2f}")
        print("")

        # Save val predictions CSV for inspection
        out_csv = "/kaggle/working/val_preds.csv"
        val_out = pd.DataFrame({"SeriesInstanceUID": uids_all})
        for i, c in enumerate(Config.TARGET_COLS):
            val_out[c + "_prob"] = P[:, i]
            val_out[c + "_true"] = T[:, i]
            val_out[c + "_thr"]  = thrs[i]
        val_out.to_csv(out_csv, index=False)
        print(f"Saved validation predictions to {out_csv} (shape={val_out.shape})")

        # Save best checkpoint (stores thresholds too)
        if macro_auc > best_auc:
            best_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": opt.state_dict(),
                "scheduler_state_dict": sch.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
                "thresholds": thrs,  # tuned on val
                "config": {
                    "img_size": args.img_size,
                    "batch_size": args.batch_size,
                    "lr": args.lr,
                    "seed": args.seed,
                    "ct_windows": Config.CT_WINDOWS,
                    "highlight": {
                        "enable": Config.HIGHLIGHT_ENABLE,
                        "pct": Config.HIGHLIGHT_TOP_PCT,
                        "gamma": Config.HIGHLIGHT_GAMMA,
                        "blend": Config.HIGHLIGHT_BLEND,
                        "ct_skip_bone": Config.CT_HIGHLIGHT_SKIP_BONE,
                        "bone_hu": Config.CT_BONE_MASK_HU,
                        "morph_k": Config.CT_BONE_MORPH_K,
                    },
                    "consistency_w": Config.CONSISTENCY_W,
                }
            }, best_ckpt)
            print(f"Saved new best: {best_ckpt} (macro_auc={macro_auc:.4f})")
            patience_left = Config.EARLY_STOP_PATIENCE if Config.EARLY_STOP_PATIENCE is not None else patience_left
        else:
            if Config.EARLY_STOP_PATIENCE is not None:
                patience_left -= 1
                print(f"No AUC improvement. Early-stop patience left: {patience_left}")
                if patience_left <= 0:
                    print("Early stopping.")
                    break

        sch.step()

    print(f"Training complete. Best macro AUC: {best_auc:.4f} | ckpt={best_ckpt}")
    return best_ckpt

# =============================================================================
# JUPYTER HELPER
# =============================================================================
def train_model_jupyter(data_dir=None, img_size=None, batch_size=None, epochs=None,
                        lr=None, num_workers=None, seed=None, out=None, benchmark_mode=False):
    class A: pass
    a = A()
    a.data_dir = data_dir or Config.DATA_DIR
    a.img_size = img_size or Config.IMG_SIZE
    a.batch_size = batch_size or Config.BATCH_SIZE
    a.epochs = epochs or Config.EPOCHS
    a.lr = lr or Config.LR
    a.num_workers = num_workers if num_workers is not None else Config.NUM_WORKERS
    a.seed = seed or Config.SEED
    a.out = out or "/kaggle/working/best.pt"
    a.benchmark_mode = benchmark_mode
    return train_model(a)

print("Loaded script (normalized, focal, sampler, morphology). Example:")
print('train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=2, num_workers=0, benchmark_mode=True)')

Loaded script (normalized, focal, sampler, morphology). Example:
train_model_jupyter(data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection", img_size=288, batch_size=2, epochs=2, num_workers=0, benchmark_mode=True)


In [2]:

# --- Recommended baseline run (smoke) ---
Config.SMOKE_TRAIN_LIMIT = 600
Config.SMOKE_VAL_LIMIT   = 200
Config.IMG_SIZE          = 288
Config.USE_CACHE         = True

# Bone-aware highlight settings you intended
Config.CT_BONE_MASK_HU   = 300
Config.CT_BONE_MORPH_K   = 3
Config.HIGHLIGHT_ENABLE  = True
Config.CT_HIGHLIGHT_SKIP_BONE = True

# Optional: ease off consistency if you want pure baseline
Config.CONSISTENCY_W = 0.2  # set to 0.0 to disable

train_model_jupyter(
    data_dir="/kaggle/input/rsna-intracranial-aneurysm-detection",
    img_size=Config.IMG_SIZE,
    batch_size=2,
    epochs=2,
    num_workers=0,
    benchmark_mode=True
)

== RSNA IA Detection: TRAIN ==
Device: cuda | AMP: True
Total rows in CSV: 4,348
Scanning decodability (quick test on first slice of each series)...
Decodable series (quick test): 600/600 kept
Decodable series (quick test): 200/200 kept
Train: 600 | Val: 200


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 146MB/s]


Model params: 4,025,482
[E1 1/300] loss=0.7246 lr=2.00e-04
[E1 50/300] loss=0.6062 lr=2.00e-04
[E1 100/300] loss=0.6490 lr=2.00e-04
[E1 150/300] loss=0.6121 lr=2.00e-04
[E1 200/300] loss=0.5900 lr=2.00e-04
[E1 250/300] loss=0.5596 lr=2.00e-04
[E1 300/300] loss=0.5391 lr=2.00e-04
Epoch 1/2 | 2317.8s  train_loss=0.5391  val_loss=0.3466  macro_auc=0.5837 macro_ap=0.1102 micro_ap=0.1348

Val confusion summary (top labels, tuned thresholds):
  Aneurysm Present             thr=0.45  TP= 95 FP= 91 TN= 11 FN=  3  P=0.51 R=0.97 F1=0.67
  Left Middle Cerebral Artery  thr=0.54  TP=  9 FP= 74 TN=116 FN=  1  P=0.11 R=0.90 F1=0.19
  Right Middle Cerebral Artery thr=0.60  TP=  5 FP= 10 TN=169 FN= 16  P=0.33 R=0.24 F1=0.28
  Basilar Tip                  thr=0.50  TP=  3 FP= 83 TN=113 FN=  1  P=0.03 R=0.75 F1=0.07
  Anterior Communicating Arter thr=0.42  TP= 14 FP=180 TN=  6 FN=  0  P=0.07 R=1.00 F1=0.13
  Left Infraclinoid Internal C thr=0.58  TP=  1 FP= 96 TN=103 FN=  0  P=0.01 R=1.00 F1=0.02

Saved 

'/kaggle/working/best.pt'

# **TEST5**

In [ ]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection - Modality-Split + 2.5D MIL + ASL + Temp Scaling

What this script does:
- Splits the dataset by modality and trains two separate models: CTA/CT and MR/MRA.
- Default representation: 2.5D Attention-MIL (top-K axial slices per study → features → attention pooling).
- Optional: 3D crop around Circle of Willis (enable via Config.MODE_3D = True).
- CTA pipeline: multi-window HU, bone mask + morphology, Frangi vesselness (with fallback), vessel-weighted highlight.
- MR pipeline: per-slice percentile normalization; tri-planar MIPs NOT used (we use axial slices for MIL).
- Loss: Asymmetric Loss (ASL) by default; BCE+Focal available.
- Hierarchical heads: Present + 13 Locations. Locations are gated at inference by Present^alpha. Consistency penalty in training.
- Temperature scaling (per label) on validation logits; thresholds then chosen:
    - Present: smallest t with precision >= TARGET_PRECISION_PRESENT (default 0.80)
    - Locations: per-label F1-tuned (you can change to target precision too)
- Metrics: per-label AUC/AP, macro AUC/AP, micro AP; confusion at chosen thresholds.
- Outputs (per modality): /kaggle/working/{mod}_val_preds.csv and {mod}_best.pt

Quick start (same dataset you used):
  - Just run the cell. To try 3D later: set Config.MODE_3D = True and keep batch_size small (e.g., 1).
"""

import os, time, random, warnings, hashlib, math
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

# Optional vesselness (Frangi); we ship a safe fallback if skimage is missing
try:
    from skimage.filters import frangi as frangi2d
    _HAS_FRANGI = True
except Exception:
    _HAS_FRANGI = False

from scipy import ndimage as ndi

# VOI LUT import (compat across pydicom versions)
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except Exception:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except Exception:
        def apply_voi_lut(x, ds): return x

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
from torchvision.models.video import r3d_18

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    # Kaggle dataset
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    # Train defaults (2.5D MIL)
    IMG_SIZE            = 320      # 288–384 is fine for MIL
    BATCH_SIZE          = 2
    EPOCHS              = 8
    LR                  = 3e-4
    NUM_WORKERS         = 0
    SEED                = 42
    USE_AMP             = True

    # 3D mode (optional). If True, we switch the model and dataset to 3D crops.
    MODE_3D             = False
    VOL_CROP_SIZE       = 96       # cubic crop edge (e.g., 96 or 112)
    VOL_ISO_SPACING_MM  = 1.2

    # MIL slice selection
    K_SLICES            = 16       # top-K axial slices per study
    VESSELNESS_BLEND    = 0.35     # CTA only: boost channels by (1 + blend*vesselness)

    # Loss
    LOSS_NAME           = "ASL"    # "ASL" or "FOCAL_BCE"
    FOCAL_GAMMA         = 2.0      # if using FOCAL_BCE

    # Logs / cache
    LOG_EVERY_N         = 50
    USE_CACHE           = True
    CACHE_DIR           = "/kaggle/working/mip_cache"
    CACHE_VERSION       = "v8.0"

    # CT windows (L, W)
    CT_WINDOWS          = [(40, 80), (200, 500), (600, 2800)]  # brain, soft/vessel, bone

    # Bone-aware highlight (CTA)
    HIGHLIGHT_ENABLE     = True
    HIGHLIGHT_TOP_PCT    = 1.0
    HIGHLIGHT_GAMMA      = 0.7
    HIGHLIGHT_BLEND      = 0.6
    CT_HIGHLIGHT_SKIP_BONE = True
    CT_BONE_MASK_HU      = 300
    CT_BONE_MORPH_K      = 3

    # Temperature scaling & thresholds
    DO_TEMP_SCALING      = True
    TARGET_PRECISION_PRESENT = 0.80
    GATE_ALPHA           = 1.0     # locations *= present_prob ** alpha

    # Consistency penalty
    CONSISTENCY_W        = 0.2

    # Small smoke subset for quick tests (set both None to use ALL)
    SMOKE_TRAIN_LIMIT    = None
    SMOKE_VAL_LIMIT      = None

    # Labels (13 sites + global)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

# ImageNet normalization (works fine even though channels ≠ RGB semantics)
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)

def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# DICOM I/O & NORMALIZATION
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def read_pixel(ds) -> np.ndarray:
    arr = ds.pixel_array
    try:
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            arr = apply_voi_lut(arr, ds)
    except Exception:
        pass
    if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
        arr = arr.astype(np.float32) * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
    else:
        arr = arr.astype(np.float32)
    return arr

def read_volume_hu(series_path: str) -> Tuple[np.ndarray, Tuple[float,float,float]]:
    """Returns volume in HU [Z,H,W] and voxel spacing (z,y,x)."""
    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")
    slices, zs = [], []
    spacing_yx = (1.0, 1.0)
    for fp in files:
        try:
            ds = pydicom.dcmread(fp, force=True)
            arr = read_pixel(ds)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            if hasattr(ds, "PixelSpacing"):
                spacing_yx = (float(ds.PixelSpacing[0]), float(ds.PixelSpacing[1]))
            st = float(getattr(ds, "SliceThickness", 1.0))
            zs.append(st)
            slices.append(arr.astype(np.float32))
        except Exception:
            continue
    if not slices: raise RuntimeError("No decodable slices")
    vol = np.stack(slices, axis=0)   # [Z,H,W]
    spacing = (np.median(zs) if zs else 1.0, spacing_yx[0], spacing_yx[1])
    return vol, spacing

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = float(level) - float(width)/2.0, float(level) + float(width)/2.0
    img = np.clip(img, lo, hi)
    denom = max(1e-6, (hi - lo))
    img = (img - lo) / denom
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# =============================================================================
# CTA vesselness & highlight
# =============================================================================
def vesselness_2d_soft(img01: np.ndarray) -> np.ndarray:
    """
    Returns vesselness in [0,1] for a 2D slice (CTA soft-tissue window).
    Uses skimage.frangi if available; otherwise a safe LoG-based fallback.
    """
    img01 = np.clip(img01.astype(np.float32), 0, 1)
    if _HAS_FRANGI:
        try:
            v = frangi2d(img01, sigmas=(1, 2, 3), alpha=0.5, beta=0.5, gamma=15.0, black_ridges=False)
            v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)
            v = (v - v.min()) / max(1e-6, (v.max() - v.min()))
            return v.astype(np.float32)
        except Exception:
            pass
    # Fallback: multi-sigma Laplacian-of-Gaussian magnitude (proxy)
    sigmas = [1.0, 2.0, 3.0]
    acc = np.zeros_like(img01, dtype=np.float32)
    for s in sigmas:
        g = ndi.gaussian_laplace(img01, sigma=s)
        acc = np.maximum(acc, -g)  # vessels bright → negative LoG
    acc = (acc - acc.min()) / max(1e-6, (acc.max() - acc.min()))
    return acc

def highlight_stretch(img01: np.ndarray, pct=None, gamma=None, blend=None) -> np.ndarray:
    pct   = Config.HIGHLIGHT_TOP_PCT if pct is None else pct
    gamma = Config.HIGHLIGHT_GAMMA   if gamma is None else gamma
    blend = Config.HIGHLIGHT_BLEND   if blend is None else blend
    img = img01.astype(np.float32); img = np.clip(img, 0.0, 1.0)
    if pct <= 0.0: return img.copy()
    thr = np.percentile(img, 100.0 - pct)
    if not np.isfinite(thr) or thr >= 0.999: return img.copy()
    out = img.copy(); mask = img >= thr
    denom = max(1e-6, 1.0 - thr)
    hi = (img - thr) / denom
    if gamma > 0: hi = np.power(hi, 1.0 / gamma)
    out[mask] = (1.0 - blend) * img[mask] + blend * hi[mask]
    return np.clip(out, 0.0, 1.0)

def morph_clean(mask: np.ndarray, k: int) -> np.ndarray:
    if k is None or k <= 0: return mask
    kernel = np.ones((k,k), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel)
    return mask

# =============================================================================
# CACHE KEYS
# =============================================================================
def _cache_path(series_path: str, out_size: int, key_extra: str) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{out_size}_{key_extra}_bone{Config.CT_BONE_MASK_HU}_m{Config.CT_BONE_MORPH_K}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"buf_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if Config.USE_CACHE and os.path.exists(path):
            arr = np.load(path, allow_pickle=False)
            return arr
    except Exception:
        pass
    return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, arr, allow_pickle=False)
    except Exception:
        pass

# =============================================================================
# 2.5D MIL DATASET (CTA & MR)
# =============================================================================
def pick_topk_slices_cta(vol_hu: np.ndarray, k: int, out_size: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Returns [K,3,H,W] CTA slices (3 CT windows) and a vesselness map per slice [K,H,W] for optional weighting.
    """
    Z, H, W = vol_hu.shape
    # Build windowed axial stacks
    wins = []
    soft = None
    for (L, Wd) in Config.CT_WINDOWS:
        ch = []
        for z in range(Z):
            ch.append(window_ct(vol_hu[z], level=L, width=Wd))
        ch = np.stack(ch, 0)  # [Z,H,W]
        wins.append(ch)
        if (L, Wd) == (200, 500): soft = ch
    if soft is None: soft = wins[1]

    # Vesselness per slice on soft-tissue window
    vness = []
    for z in range(Z):
        v = vesselness_2d_soft(soft[z])
        vness.append(v)
    vness = np.stack(vness, 0)  # [Z,H,W]

    # Score slices by mean vesselness near center (robust)
    cy0, cx0, r = H//2, W//2, min(H, W)//4
    yy, xx = np.ogrid[:H, :W]
    circ = ((yy-cy0)**2 + (xx-cx0)**2) <= r*r
    slice_scores = (vness * circ).sum((1,2)) / max(1, circ.sum())

    idx = np.argsort(-slice_scores)[:k]
    idx = np.sort(idx)
    # assemble channels, resize
    imgs = []
    vnks = []
    for z in idx:
        chans = []
        for ch in wins:
            im = cv2.resize(ch[z], (out_size, out_size), interpolation=cv2.INTER_AREA)
            chans.append(im.astype(np.float32))
        v = cv2.resize(vness[z], (out_size, out_size), interpolation=cv2.INTER_AREA).astype(np.float32)
        # Optional vesselness-weighted boost of each channel
        if Config.VESSELNESS_BLEND > 0:
            boost = (1.0 + Config.VESSELNESS_BLEND * v)
            chans = [np.clip(c * boost, 0, 1) for c in chans]

        # Optional highlight, skip bone areas
        if Config.HIGHLIGHT_ENABLE:
            # Bone mask from bone window: threshold BEFORE highlight
            bone_mip = (cv2.resize(wins[2][z], (out_size, out_size), interpolation=cv2.INTER_AREA) > 
                        (Config.CT_BONE_MASK_HU - (-1024)) / (2800))  # rough; safe fallback
            bone_mip = morph_clean(bone_mip.astype(np.uint8), Config.CT_BONE_MORPH_K).astype(bool)
            for i in range(len(chans)):
                hi = highlight_stretch(chans[i])
                if Config.CT_HIGHLIGHT_SKIP_BONE:
                    chans[i] = np.where(~bone_mip, hi, chans[i]).astype(np.float32)
                else:
                    chans[i] = hi

        imgs.append(np.stack(chans, 0))   # [3,H,W]
        vnks.append(v)
    return np.stack(imgs, 0), np.stack(vnks, 0)  # [K,3,H,W], [K,H,W]

def pick_topk_slices_mr(vol: np.ndarray, k: int, out_size: int) -> np.ndarray:
    """
    Returns [K,3,H,W] MR slices (replicate normalized slice into 3 channels).
    We rank slices by their 99th percentile intensity (brighter vessels).
    """
    Z, H, W = vol.shape
    norm = []
    scores = []
    for z in range(Z):
        im = normalize_mr(vol[z])
        scores.append(np.percentile(im, 99))
        im = cv2.resize(im, (out_size, out_size), interpolation=cv2.INTER_AREA)
        # gentle highlight
        if Config.HIGHLIGHT_ENABLE:
            im = highlight_stretch(im)
        norm.append(im.astype(np.float32))
    idx = np.argsort(-np.array(scores))[:k]
    idx = np.sort(idx)
    out = []
    for z in idx:
        im = norm[z]
        out.append(np.stack([im, im, im], 0))
    return np.stack(out, 0)  # [K,3,H,W]

def build_2d_mil_buffers(series_path: str, is_ct: bool, out_size: int, k_slices: int):
    cache = _cache_path(series_path, out_size, key_extra=f"mil2d_ct{int(is_ct)}_k{k_slices}")
    cached = _load_cache(cache)
    if cached is not None:
        return cached
    vol, _ = read_volume_hu(series_path) if is_ct else read_volume_hu(series_path)  # MR read same but not HU-scale
    if is_ct:
        imgs, _v = pick_topk_slices_cta(vol, k_slices, out_size)   # [K,3,H,W]
        buf = imgs
    else:
        imgs = pick_topk_slices_mr(vol, k_slices, out_size)
        buf = imgs
    _save_cache(buf, cache)
    return buf

# =============================================================================
# 3D CROP DATASET (optional)
# =============================================================================
def resample_iso(vol: np.ndarray, spacing: Tuple[float,float,float], iso=1.2) -> np.ndarray:
    """Resample volume [Z,H,W] to isotropic spacing ~iso (mm)."""
    z,y,x = spacing
    zoom = (z/iso, y/iso, x/iso)
    out = ndi.zoom(vol.astype(np.float32), zoom=zoom, order=1)
    return out

def find_cof_bbox_cta(vol_hu_iso: np.ndarray, out_edge: int) -> Tuple[int,int,int]:
    """Very simple: use vesselness on soft window to get a coarse COM, then center a cube."""
    # Build soft window stack in [0,1]
    Z,H,W = vol_hu_iso.shape
    soft = np.stack([window_ct(vol_hu_iso[z], 200, 500) for z in range(Z)], 0)
    # Vesselness per slice and combine
    v3 = np.zeros_like(soft)
    for z in range(Z):
        v3[z] = vesselness_2d_soft(soft[z])
    v3 = ndi.gaussian_filter(v3, sigma=1.0)
    # Center of mass
    m = v3 / max(1e-6, v3.sum())
    zz, yy, xx = np.indices(v3.shape)
    cz = int(np.clip((zz*m).sum(), 0, Z-1))
    cy = int(np.clip((yy*m).sum(), 0, H-1))
    cx = int(np.clip((xx*m).sum(), 0, W-1))
    return cz, cy, cx

def crop_cube(vol: np.ndarray, center: Tuple[int,int,int], edge: int) -> np.ndarray:
    cz, cy, cx = center
    Z,H,W = vol.shape
    r = edge//2
    z0, z1 = max(0, cz-r), min(Z, cz+r)
    y0, y1 = max(0, cy-r), min(H, cy+r)
    x0, x1 = max(0, cx-r), min(W, cx+r)
    cube = np.zeros((edge, edge, edge), np.float32)
    cz0 = (edge - (z1 - z0))//2
    cy0 = (edge - (y1 - y0))//2
    cx0 = (edge - (x1 - x0))//2
    cube[cz0:cz0+(z1-z0), cy0:cy0+(y1-y0), cx0:cx0+(x1-x0)] = vol[z0:z1, y0:y1, x0:x1]
    return cube

# =============================================================================
# DATASET & SAMPLER
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p): raise FileNotFoundError(p)
    return p

def _detect_modality_from_header(series_path: str) -> Optional[str]:
    files = iter_dicom_files(series_path)
    if not files: return None
    try:
        ds = pydicom.dcmread(files[0], stop_before_pixels=True, force=True)
        return str(getattr(ds, "Modality", "")).upper() or None
    except Exception:
        return None

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        ok = False
        try:
            files = iter_dicom_files(sp)
            if files:
                ds = pydicom.dcmread(files[0], force=True)
                _ = ds.pixel_array
                ok = True
        except Exception:
            ok = False
        keep_mask.append(ok)
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIL2DDataset(Dataset):
    """Returns one study as K slices [K,3,H,W], normalized for ImageNet, and label vector [C]."""
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int, k_slices: int, modality: str):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        self.k_slices = k_slices
        self.modality = modality  # "CT" or "MR"
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss: raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)
        if self.modality == "CT":
            buf = build_2d_mil_buffers(sp, is_ct=True, out_size=self.out_size, k_slices=self.k_slices)  # [K,3,H,W]
        else:
            buf = build_2d_mil_buffers(sp, is_ct=False, out_size=self.out_size, k_slices=self.k_slices)
        x = torch.from_numpy(buf).float()         # [K,3,H,W] in [0,1]
        # ImageNet normalization per-slice
        x = (x - IMAGENET_MEAN) / IMAGENET_STD
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

class Vol3DDataset(Dataset):
    """3D crop dataset: returns [3,D,H,W] by stacking same HU volume into 3 channels, normalized ~ImageNet."""
    def __init__(self, df: pd.DataFrame, series_dir: str, crop_edge: int, iso: float, modality: str):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.edge = crop_edge
        self.iso = iso
        self.modality = modality
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss: raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)
        vol_hu, spacing = read_volume_hu(sp)
        vol_iso = resample_iso(vol_hu, spacing, iso=self.iso)  # [Z,H,W]
        if self.modality == "CT":
            cz, cy, cx = find_cof_bbox_cta(vol_iso, self.edge)
        else:
            # For MR, use 99th-percentile intensity COM
            v = np.clip(vol_iso, np.percentile(vol_iso, 1), np.percentile(vol_iso, 99))
            v = (v - v.min()) / max(1e-6, (v.max() - v.min()))
            m = v / max(1e-6, v.sum())
            zz, yy, xx = np.indices(v.shape)
            cz = int(np.clip((zz*m).sum(), 0, v.shape[0]-1))
            cy = int(np.clip((yy*m).sum(), 0, v.shape[1]-1))
            cx = int(np.clip((xx*m).sum(), 0, v.shape[2]-1))
        cube = crop_cube(vol_iso, (cz,cy,cx), self.edge)  # [D,H,W]
        # Window for CT; MR percentile normalize
        if self.modality == "CT":
            # Use soft-tissue window as base, but replicate into 3 channels
            D,H,W = cube.shape
            ch = np.stack([window_ct(cube[d], 200, 500) for d in range(D)], 0)
        else:
            v = np.clip(cube, np.percentile(cube, 1), np.percentile(cube, 99))
            ch = (v - v.min()) / max(1e-6, (v.max()-v.min()))
        # Stack to 3 channels for 3D Net
        vol3 = np.stack([ch, ch, ch], 0)  # [3,D,H,W]
        x = torch.from_numpy(vol3).float()
        # crude "ImageNet-like" normalization channel-wise
        for c in range(3):
            x[c] = (x[c] - IMAGENET_MEAN[c]) / IMAGENET_STD[c]
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

def build_class_balanced_sampler(df: pd.DataFrame, target_cols: List[str]) -> WeightedRandomSampler:
    pos = df[target_cols].sum(0).values
    neg = len(df) - pos
    cls_w = np.where(pos > 0, neg / np.maximum(1, pos), 1.0)
    cls_w = np.clip(cls_w, 1.0, 50.0)
    Y = df[target_cols].values.astype(np.float32)
    w = 1.0 + (Y * cls_w).sum(1)
    w = torch.as_tensor(w, dtype=torch.double)
    return WeightedRandomSampler(weights=w, num_samples=len(df), replacement=True)

# =============================================================================
# MODELS
# =============================================================================
def efficientnet_b2_features():
    # torchvision EfficientNet-B2 returns features via model.features
    m = torchvision.models.efficientnet_b2(weights=torchvision.models.EfficientNet_B2_Weights.IMAGENET1K_V1)
    feat_dim = m.classifier[1].in_features
    backbone = nn.Sequential(m.features, nn.AdaptiveAvgPool2d(1), nn.Flatten())  # -> [B, feat_dim]
    return backbone, feat_dim

class AttnMIL(nn.Module):
    """Processes K slices with a 2D backbone and attention-pools to a study embedding."""
    def __init__(self, feat_dim, hidden=512):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(feat_dim, hidden), nn.ReLU(inplace=True),
            nn.Linear(hidden, 1)
        )
    def forward(self, F):  # F: [B,K,D]
        A = self.attn(F).squeeze(-1)           # [B,K]
        A = torch.softmax(A, dim=1)            # weights sum to 1
        M = torch.sum(F * A.unsqueeze(-1), 1)  # [B,D]
        return M, A

class HierHead(nn.Module):
    """Two heads: Present (1) + Locations (13)."""
    def __init__(self, in_dim, n_locs=13):
        super().__init__()
        self.present = nn.Linear(in_dim, 1)
        self.locs    = nn.Linear(in_dim, n_locs)
    def forward(self, z):
        return self.present(z).squeeze(-1), self.locs(z)

class MIL2DModel(nn.Module):
    def __init__(self, n_out=14):
        super().__init__()
        self.backbone, feat_dim = efficientnet_b2_features()
        self.mil = AttnMIL(feat_dim)
        self.head = HierHead(feat_dim, n_locs=n_out-1)
    def forward(self, x):  # x: [B,K,3,H,W]
        B,K,C,H,W = x.shape
        x = x.view(B*K, C, H, W)
        f = self.backbone(x)                 # [B*K, D]
        f = f.view(B, K, -1)                 # [B,K,D]
        z, att = self.mil(f)                 # [B,D], [B,K]
        present, locs = self.head(z)         # [B], [B,13]
        logits = torch.cat([locs, present.unsqueeze(-1)], dim=1)  # [B,14] (locations first)
        return logits, att

class Res3DModel(nn.Module):
    def __init__(self, n_out=14):
        super().__init__()
        m = r3d_18(weights=None)
        m.fc = nn.Linear(m.fc.in_features, 512)
        self.backbone = m
        self.head = HierHead(512, n_locs=n_out-1)
    def forward(self, x):  # [B,3,D,H,W] → r3d expects [B,3,T,H,W]
        z = self.backbone(x)                 # [B,512]
        present, locs = self.head(z)
        logits = torch.cat([locs, present.unsqueeze(-1)], dim=1)
        return logits, None

# =============================================================================
# LOSSES
# =============================================================================
class AsymmetricLoss(nn.Module):
    """
    Asymmetric Loss (ASL) for multi-label classification.
    Params near standard: gamma_neg=4, gamma_pos=0, clip=0.05 (margin), disable_torch_grad_focal
    """
    def __init__(self, gamma_pos=0, gamma_neg=4, clip=0.05, eps=1e-8):
        super().__init__()
        self.gp = gamma_pos
        self.gn = gamma_neg
        self.clip = clip
        self.eps = eps

    def forward(self, logits, targets):
        x_sigmoid = torch.sigmoid(logits)
        xs_pos = x_sigmoid
        xs_neg = 1.0 - x_sigmoid
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)
        loss_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        loss_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        loss = loss_pos + loss_neg
        # Asymmetric focusing
        if self.gp > 0 or self.gn > 0:
            pt = x_sigmoid * targets + (1 - x_sigmoid) * (1 - targets)
            one_sided_gamma = self.gp * targets + self.gn * (1 - targets)
            focal_weight = (1 - pt) ** one_sided_gamma
            loss *= focal_weight
        return -loss.mean()

class BCEWithLogitsFocal(nn.Module):
    def __init__(self, pos_weight=None, gamma=2.0):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction="none")
        self.gamma = gamma
    def forward(self, logits, targets):
        bce = self.bce(logits, targets)
        p = torch.sigmoid(logits)
        pt = p*targets + (1-p)*(1-targets)
        focal = (1.0 - pt).clamp_min(1e-6).pow(self.gamma)
        return (focal * bce).mean()

# =============================================================================
# METRICS & THRESHOLDS
# =============================================================================
def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_auc, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_auc[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_auc[c] = float(auc); macro_vals.append(float(auc))
        except Exception:
            col_auc[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_auc, macro

def compute_ap_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_ap, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_ap[c] = np.nan
                continue
            ap = average_precision_score(y_true[:, i], y_prob[:, i])
            col_ap[c] = float(ap); macro_vals.append(float(ap))
        except Exception:
            col_ap[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_ap, macro

def compute_micro_ap(y_true: np.ndarray, y_prob: np.ndarray):
    mask = (y_true.sum(0) > 0) & ((1 - y_true).sum(0) > 0)
    if mask.sum() == 0: return np.nan
    return average_precision_score(y_true[:, mask].ravel(), y_prob[:, mask].ravel())

def tune_thresholds_best_f1(y_true: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
    n_labels = y_true.shape[1]
    thrs = np.zeros(n_labels, dtype=np.float32)
    grid = np.linspace(0.0, 1.0, 1001)
    for i in range(n_labels):
        yi = y_true[:, i].astype(np.int32)
        pi = y_prob[:, i]
        if len(np.unique(yi)) < 2: thrs[i] = 0.5; continue
        best_f1, best_t = -1.0, 0.5
        for t in grid:
            pred = (pi >= t).astype(np.int32)
            tp = int((pred & (yi == 1)).sum()); fp = int((pred & (yi == 0)).sum())
            fn = int(((1 - pred) & (yi == 1)).sum())
            precision = tp / max(1, (tp + fp)); recall = tp / max(1, (tp + fn))
            f1 = 2*precision*recall / max(1e-8, (precision + recall))
            if f1 > best_f1: best_f1, best_t = f1, t
        thrs[i] = best_t
    return thrs

def smallest_t_for_precision(y_true: np.ndarray, y_prob: np.ndarray, target_p=0.8) -> float:
    order = np.argsort(-y_prob)
    y = y_true[order]; p = y_prob[order]
    tp=0; fp=0
    best_t = 1.0
    for i in range(len(y)):
        if y[i] == 1: tp += 1
        else: fp += 1
        prec = tp / max(1, tp+fp)
        if prec >= target_p:
            best_t = p[i]
            break
    return float(best_t)

def confusion_stats(y_true: np.ndarray, y_prob: np.ndarray, thrs: np.ndarray,
                    cols: List[str], top_k: int = 6) -> pd.DataFrame:
    rows = []
    for i, c in enumerate(cols):
        yi = y_true[:, i].astype(np.int32)
        pi = (y_prob[:, i] >= thrs[i]).astype(np.int32)
        tp = int((pi & (yi == 1)).sum())
        fp = int((pi & (yi == 0)).sum())
        tn = int(((1 - pi) & (yi == 0)).sum())
        fn = int(((1 - pi) & (yi == 1)).sum())
        prec = tp / max(1, (tp + fp))
        rec  = tp / max(1, (tp + fn))
        f1   = 2*prec*rec / max(1e-8, (prec + rec))
        rows.append([c, thrs[i], tp, fp, tn, fn, prec, rec, f1])
    df = pd.DataFrame(rows, columns=["label","thr","TP","FP","TN","FN","precision","recall","F1"])
    priority = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery",
                "Basilar Tip", "Anterior Communicating Artery"]
    order = priority + [c for c in cols if c not in priority]
    df = df.set_index("label").loc[order].reset_index()
    return df.head(top_k)

# =============================================================================
# TEMPERATURE SCALING (per label)
# =============================================================================
@torch.no_grad()
def collect_logits(model, loader, dev, mode_3d=False):
    model.eval()
    logits_list, targets_list, uids = [], [], []
    for batch in loader:
        x, y, idlist = batch
        x = x.to(dev, non_blocking=True)
        y = y.to(dev, non_blocking=True)
        if mode_3d:
            out, _ = model(x)              # [B,14]
        else:
            out, _ = model(x)              # [B,14]
        logits_list.append(out.detach().cpu())
        targets_list.append(y.detach().cpu())
        uids.extend(list(idlist))
    L = torch.cat(logits_list, 0).numpy()
    T = torch.cat(targets_list, 0).numpy()
    return L, T, uids

def fit_temperature_per_label(logits: np.ndarray, targets: np.ndarray, max_iter=200, lr=0.05) -> np.ndarray:
    """
    Fit a temperature per column minimizing BCEWithLogits on val.
    Returns temperatures shape [C], >= 1e-3.
    """
    L = torch.tensor(logits, dtype=torch.float32)
    Y = torch.tensor(targets, dtype=torch.float32)
    C = L.shape[1]
    temps = torch.ones(C, dtype=torch.float32, requires_grad=True)
    optim_t = torch.optim.Adam([temps], lr=lr)
    for _ in range(max_iter):
        optim_t.zero_grad()
        scaled = L / temps.clamp_min(1e-3)
        loss = nn.BCEWithLogitsLoss()(scaled, Y)
        loss.backward()
        optim_t.step()
    return temps.detach().clamp_min(1e-3).cpu().numpy()

# =============================================================================
# TRAIN / EVAL LOOP (one modality)
# =============================================================================
def run_one_modality(modality_name: str, full_df: pd.DataFrame, series_dir: str, dev: str):
    """
    modality_name: "CT"  -> use CTA/CT subset
                   "MR"  -> use MR/MRA subset
    """
    is_ct = (modality_name == "CT")
    print(f"\n================= TRAINING MODALITY: {modality_name} =================")

    # Subset df by modality
    df = full_df.copy()
    # Prefer CSV Modality, else read header
    mod = df.get("Modality")
    if mod is None:
        df["Modality"] = ""
    if "Modality" in df.columns:
        df["Modality"] = df["Modality"].fillna("")
    modes = []
    for _, r in df.iterrows():
        m = str(r.get("Modality", "")).upper()
        if not m:
            try:
                m = _detect_modality_from_header(os.path.join(series_dir, str(r["SeriesInstanceUID"]))) or ""
            except Exception:
                m = ""
        modes.append(m)
    df["__mod"] = modes
    if is_ct:
        sub = df[df["__mod"].isin(["CT","CTA"])].reset_index(drop=True)
    else:
        sub = df[df["__mod"].isin(["MR","MRA"])].reset_index(drop=True)

    if len(sub) == 0:
        print(f"No {modality_name} studies found. Skipping.")
        return None

    # Split stratified by "Aneurysm Present"
    train_df, val_df = stratified_split(sub, seed=Config.SEED)

    # Optional smoke limits
    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df   = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    # Filter decodable
    print("Scanning decodability (quick test)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df, series_dir)
    print(f"{modality_name} Train: {len(train_keep)} | Val: {len(val_keep)}")

    # Build datasets/loaders
    if Config.MODE_3D:
        train_ds = Vol3DDataset(train_keep, series_dir, Config.VOL_CROP_SIZE, Config.VOL_ISO_SPACING_MM, modality_name)
        val_ds   = Vol3DDataset(val_keep,   series_dir, Config.VOL_CROP_SIZE, Config.VOL_ISO_SPACING_MM, modality_name)
    else:
        train_ds = MIL2DDataset(train_keep, series_dir, Config.IMG_SIZE, Config.K_SLICES, modality_name)
        val_ds   = MIL2DDataset(val_keep,   series_dir, Config.IMG_SIZE, Config.K_SLICES, modality_name)

    train_sampler = build_class_balanced_sampler(train_keep, Config.TARGET_COLS)
    tkw = dict(batch_size=Config.BATCH_SIZE, sampler=train_sampler, num_workers=Config.NUM_WORKERS, drop_last=True)
    vkw = dict(batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=Config.NUM_WORKERS, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    # Model
    if Config.MODE_3D:
        model = Res3DModel(n_out=len(Config.TARGET_COLS)).to(dev)
    else:
        model = MIL2DModel(n_out=len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    # Loss
    present_idx = Config.TARGET_COLS.index("Aneurysm Present")
    if Config.LOSS_NAME.upper() == "ASL":
        criterion = AsymmetricLoss(gamma_pos=0, gamma_neg=4, clip=0.05)
    else:
        # compute pos_weight per label from training subset
        posw = []
        for c in Config.TARGET_COLS:
            pos = train_keep[c].sum(); neg = len(train_keep) - pos
            w = (neg / max(1.0, pos)) if pos > 0 else 1.0
            posw.append(float(np.clip(w, 1.0, 50.0)))
        criterion = BCEWithLogitsFocal(pos_weight=torch.tensor(posw, device=dev), gamma=Config.FOCAL_GAMMA)

    opt = optim.AdamW(model.parameters(), lr=Config.LR, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=Config.EPOCHS)
    scaler = torch.cuda.amp.GradScaler(enabled=(dev=="cuda" and Config.USE_AMP))

    best_macro_auc = -1.0
    best_ckpt = f"/kaggle/working/{modality_name.lower()}_best.pt"

    # ------------- Train -------------
    for epoch in range(Config.EPOCHS):
        model.train()
        run_loss = 0.0; t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(dev=="cuda" and Config.USE_AMP)):
                logits, _att = model(x)                # [B,14]
                loss = criterion(logits, y)
                # consistency: locations <= present
                if Config.CONSISTENCY_W > 0:
                    probs = torch.sigmoid(logits)
                    present = probs[:, present_idx:present_idx+1]
                    locs   = probs[:, :present_idx]   # [B,13]
                    consist = torch.clamp(locs - present, min=0.0).pow(2).mean()
                    loss = loss + Config.CONSISTENCY_W * consist
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()
            run_loss += loss.item()
            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[{modality_name} E{epoch+1} {b}/{len(train_loader)}] loss={run_loss/b:.4f} lr={sch.get_last_lr()[0]:.2e}")
        sch.step()

        # ------------- Val -------------
        model.eval()
        val_loss = 0.0
        logits_all, probs_all, targs_all, uids_all = [], [], [], []
        with torch.no_grad():
            for x, y, uids in val_loader:
                x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(dev=="cuda" and Config.USE_AMP)):
                    out, _att = model(x)
                    ls = nn.BCEWithLogitsLoss(reduction="mean")(out, y)
                val_loss += ls.item()
                logits_all.append(out.detach().cpu().numpy())
                probs_all.append(torch.sigmoid(out).detach().cpu().numpy())
                targs_all.append(y.detach().cpu().numpy())
                uids_all.extend(list(uids))

        L = np.vstack(logits_all)
        P = np.vstack(probs_all)
        T = np.vstack(targs_all)
        col_aucs, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P, Config.TARGET_COLS)
        micro_ap = compute_micro_ap(T, P)
        dt = time.time() - t0
        print(f"{modality_name} Epoch {epoch+1}/{Config.EPOCHS} | {dt:.1f}s  val_loss={val_loss/max(1,len(val_loader)):.4f}  "
              f"macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f} micro_ap={micro_ap:.4f}")

        # Save best
        if macro_auc > best_macro_auc:
            best_macro_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
                "config": vars(Config),
            }, best_ckpt)
            print(f"Saved new best for {modality_name}: {best_ckpt} (macro_auc={macro_auc:.4f})")

    # -------- Post-hoc calibration (temperature scaling) on final val --------
    # Re-run val once to collect logits/targets cleanly
    L, T, uids_all = collect_logits(model, val_loader, dev, mode_3d=Config.MODE_3D)
    if Config.DO_TEMP_SCALING:
        temps = fit_temperature_per_label(L, T, max_iter=200, lr=0.05)  # [C]
    else:
        temps = np.ones(L.shape[1], dtype=np.float32)
    Ls = L / np.clip(temps, 1e-3, None)
    P_cal = 1.0 / (1.0 + np.exp(-Ls))

    # Gating: locations *= present^alpha
    present_idx = Config.TARGET_COLS.index("Aneurysm Present")
    loc_idx = [i for i in range(len(Config.TARGET_COLS)) if i != present_idx]
    if Config.GATE_ALPHA > 0:
        pr = np.clip(P_cal[:, present_idx:present_idx+1], 0, 1) ** Config.GATE_ALPHA
        P_cal[:, loc_idx] = P_cal[:, loc_idx] * pr

    # Thresholds:
    thrs = np.zeros(len(Config.TARGET_COLS), dtype=np.float32)
    # 1) Present: smallest t with precision >= target
    thrs[present_idx] = smallest_t_for_precision(T[:, present_idx], P_cal[:, present_idx], target_p=Config.TARGET_PRECISION_PRESENT)
    # 2) Locations: F1-tuned (you can switch to target precision similarly)
    thrs[loc_idx] = tune_thresholds_best_f1(T[:, loc_idx], P_cal[:, loc_idx])

    # Confusion summary
    conf_df = confusion_stats(T, P_cal, thrs, Config.TARGET_COLS, top_k=6)
    print(f"\n{modality_name} (Calibrated) confusion summary @ thresholds:")
    for _, r in conf_df.iterrows():
        print(f"  {r['label'][:28]:28s} thr={r['thr']:.2f}  TP={r['TP']:3d} FP={r['FP']:3d} "
              f"TN={r['TN']:3d} FN={r['FN']:3d}  P={r['precision']:.2f} R={r['recall']:.2f} F1={r['F1']:.2f}")

    # Metrics (calibrated)
    col_aucs, macro_auc = compute_auc_per_column(T, P_cal, Config.TARGET_COLS)
    col_aps , macro_ap  = compute_ap_per_column(T, P_cal, Config.TARGET_COLS)
    micro_ap = compute_micro_ap(T, P_cal)
    print(f"\n{modality_name} (Calibrated) macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f} micro_ap={micro_ap:.4f}")

    # Save per-modality val preds CSV
    out_csv = f"/kaggle/working/{modality_name.lower()}_val_preds.csv"
    df_out = pd.DataFrame({"SeriesInstanceUID": uids_all})
    for i, c in enumerate(Config.TARGET_COLS):
        df_out[c + "_prob"] = P_cal[:, i]
        df_out[c + "_true"] = T[:, i]
        df_out[c + "_thr"]  = thrs[i]
    df_out.to_csv(out_csv, index=False)
    print(f"Saved {modality_name} validation predictions to {out_csv} (shape={df_out.shape})")

    # Save final checkpoint with temps + thresholds
    torch.save({
        "epoch": Config.EPOCHS,
        "model_state_dict": model.state_dict(),
        "target_cols": Config.TARGET_COLS,
        "temps": temps.astype(np.float32),
        "thresholds": thrs.astype(np.float32),
        "config": vars(Config),
        "modality": modality_name,
    }, f"/kaggle/working/{modality_name.lower()}_best.pt")
    print(f"Saved {modality_name} calibrated model with temps+thresholds: /kaggle/working/{modality_name.lower()}_best.pt")

    return True

# =============================================================================
# MAIN
# =============================================================================
def main():
    print("== RSNA IA Detection: Modality-Split MIL (ASL, TempScaling, Gated Heads) ==")
    set_seed(Config.SEED, benchmark_mode=True if device()=="cuda" else False)
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP} | 3D Mode: {Config.MODE_3D}")

    csv_path = os.path.join(Config.DATA_DIR, "train.csv")
    if not os.path.exists(csv_path): raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    series_dir = os.path.join(Config.DATA_DIR, "series")
    print(f"Rows in CSV: {len(df):,}")

    # Train per-modality
    run_one_modality("CT", df, series_dir, dev)
    run_one_modality("MR", df, series_dir, dev)
    print("All done.")

# Jupyter helper
def train_model_jupyter(**kwargs):
    # Allow quick overrides
    for k,v in kwargs.items():
        if hasattr(Config, k):
            setattr(Config, k, v)
    main()

if __name__ == "__main__":
    main()

print("Loaded script. Example:")
print('train_model_jupyter(DATA_DIR="/kaggle/input/rsna-intracranial-aneurysm-detection", IMG_SIZE=320, K_SLICES=16, EPOCHS=8)')

== RSNA IA Detection: Modality-Split MIL (ASL, TempScaling, Gated Heads) ==
Device: cuda | AMP: True | 3D Mode: False
Rows in CSV: 4,348

================= TRAINING MODALITY: CT =================
Scanning decodability (quick test)...
Decodable series (quick test): 1446/1446 kept
Decodable series (quick test): 362/362 kept
CT Train: 1446 | Val: 362


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 66.3MB/s]


Model params: 8,442,641
[CT E1 1/723] loss=0.1303 lr=3.00e-04


# **TEST2**

In [ ]:
#!/usr/bin/env python3
"""
RSNA Intracranial Aneurysm Detection
Modality-Split (CT vs MR) + 2.5D MIL + Asymmetric Loss + Temp Scaling
(with Kaggle-friendly speed-ups)

What this script does
- Trains two separate models: CT/CTA and MR/MRA.
- 2.5D Attention-MIL: top-K axial slices per study → 2D backbone → attention pooling.
- CTA: multi-window HU, HU bone mask + morphology, optional vesselness (Frangi) ranking/boost.
- MR: percentile normalization; axial slices for MIL.
- Loss: Asymmetric Loss (ASL) by default (BCE+Focal available).
- Heads: 'Present' + 13 locations; during inference, location probs are gated by Present^alpha.
- Temperature scaling (per-label) on validation logits; thresholds then chosen:
    * Present: smallest t with precision ≥ TARGET_PRECISION_PRESENT (default 0.80)
    * Locations: per-label F1 tuned
- Outputs (per modality): /kaggle/working/{mod}_val_preds.csv and {mod}_best.pt

Default settings here favor **speed for a first pass**:
- CT slice ranking uses fast p99 (no vesselness), K=8, 1–2 num_workers.
- After the first run (cache built), bump K_SLICES to 16 and (optionally) enable vesselness.

Quick start:
    # just run the cell; or call:
    train_model_jupyter(
        DATA_DIR="/kaggle/input/rsna-intracranial-aneurysm-detection",
        IMG_SIZE=320, K_SLICES=8, EPOCHS=4
    )
"""

import os, time, random, warnings, hashlib, math
from pathlib import Path
from typing import List, Tuple, Dict, Optional

import numpy as np
import pandas as pd
import cv2
import pydicom

# Optional vesselness (Frangi); safe fallback if skimage isn't present
try:
    from skimage.filters import frangi as frangi2d
    _HAS_FRANGI = True
except Exception:
    _HAS_FRANGI = False

from scipy import ndimage as ndi

# VOI LUT import (compat across pydicom versions)
try:
    from pydicom.pixel_data_handlers import apply_voi_lut
except Exception:
    try:
        from pydicom.pixel_data_handlers.util import apply_voi_lut
    except Exception:
        def apply_voi_lut(x, ds): return x

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision
from torchvision.models.video import r3d_18

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score

warnings.filterwarnings("ignore")

# =============================================================================
# CONFIG
# =============================================================================
class Config:
    # Kaggle dataset
    DATA_DIR   = "/kaggle/input/rsna-intracranial-aneurysm-detection"
    SERIES_DIR = f"{DATA_DIR}/series"
    TRAIN_CSV  = f"{DATA_DIR}/train.csv"

    # Train defaults (2.5D MIL)
    IMG_SIZE            = 320
    BATCH_SIZE          = 2
    EPOCHS              = 4
    LR                  = 3e-4
    # Use 1–(cpu-1) workers on Kaggle (usually 2 vCPUs); overlap CPU prep with GPU
    NUM_WORKERS         = max(1, (os.cpu_count() or 2) - 1)
    SEED                = 42
    USE_AMP             = True

    # 3D mode (optional). If True, we switch to a 3D backbone on cropped cubes.
    MODE_3D             = False
    VOL_CROP_SIZE       = 96
    VOL_ISO_SPACING_MM  = 1.2

    # MIL slice selection
    K_SLICES            = 8                 # fast first pass; bump to 16 later
    CTA_RANKING         = "p99"             # "p99" (fast) or "vesselness"
    VESSELNESS_BLEND    = 0.0               # set 0.35 for slight boost (CTA), adds CPU

    # Loss
    LOSS_NAME           = "ASL"             # "ASL" or "FOCAL_BCE"
    FOCAL_GAMMA         = 2.0               # if using FOCAL_BCE

    # Logs / cache
    LOG_EVERY_N         = 10
    USE_CACHE           = True
    CACHE_DIR           = "/kaggle/working/mip_cache"
    CACHE_VERSION       = "v8.1"

    # CT windows (L, W)
    CT_WINDOWS          = [(40, 80), (200, 500), (600, 2800)]  # brain, soft/vessel, bone

    # Bone-aware highlight (CTA)
    HIGHLIGHT_ENABLE       = True
    HIGHLIGHT_TOP_PCT      = 1.0
    HIGHLIGHT_GAMMA        = 0.7
    HIGHLIGHT_BLEND        = 0.6
    CT_HIGHLIGHT_SKIP_BONE = True
    CT_BONE_MASK_HU        = 300
    CT_BONE_MORPH_K        = 3

    # Temperature scaling & thresholds
    DO_TEMP_SCALING      = True
    TARGET_PRECISION_PRESENT = 0.80
    GATE_ALPHA           = 1.0     # locations *= present_prob ** alpha

    # Consistency penalty
    CONSISTENCY_W        = 0.2

    # Small smoke subset for quick tests (set both None to use ALL)
    SMOKE_TRAIN_LIMIT    = None
    SMOKE_VAL_LIMIT      = None

    # Labels (13 sites + global)
    TARGET_COLS = [
        "Left Infraclinoid Internal Carotid Artery",
        "Right Infraclinoid Internal Carotid Artery",
        "Left Supraclinoid Internal Carotid Artery",
        "Right Supraclinoid Internal Carotid Artery",
        "Left Middle Cerebral Artery",
        "Right Middle Cerebral Artery",
        "Anterior Communicating Artery",
        "Left Anterior Cerebral Artery",
        "Right Anterior Cerebral Artery",
        "Left Posterior Communicating Artery",
        "Right Posterior Communicating Artery",
        "Basilar Tip",
        "Other Posterior Circulation",
        "Aneurysm Present",
    ]

# ImageNet normalization (works even though channels ≠ RGB semantics)
IMAGENET_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
IMAGENET_STD  = torch.tensor([0.229, 0.224, 0.225]).view(3,1,1)

# =============================================================================
# UTIL
# =============================================================================
def set_seed(seed=42, benchmark_mode=False):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = not benchmark_mode
    torch.backends.cudnn.benchmark     = benchmark_mode

def device() -> str:
    return "cuda" if torch.cuda.is_available() else "cpu"

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

# =============================================================================
# DICOM I/O & NORMALIZATION
# =============================================================================
def iter_dicom_files(series_path: str) -> List[str]:
    sp = Path(series_path)
    dcm_files = list(sp.glob("*.dcm"))
    if not dcm_files:
        dcm_files = [f for f in sp.iterdir() if f.is_file() and not f.suffix]
    if not dcm_files:
        return []
    infos = []
    for f in dcm_files:
        try:
            ds = pydicom.dcmread(str(f), stop_before_pixels=True, force=True)
            ins = getattr(ds, "InstanceNumber", 0)
        except Exception:
            ins = 0
        infos.append((ins, str(f)))
    infos.sort(key=lambda x: (x[0], x[1]))
    return [p for _, p in infos]

def read_pixel(ds) -> np.ndarray:
    arr = ds.pixel_array
    try:
        if hasattr(ds, 'VOILUTSequence') or hasattr(ds, 'WindowCenter'):
            arr = apply_voi_lut(arr, ds)
    except Exception:
        pass
    if hasattr(ds, "RescaleSlope") and hasattr(ds, "RescaleIntercept"):
        arr = arr.astype(np.float32) * float(ds.RescaleSlope) + float(ds.RescaleIntercept)
    else:
        arr = arr.astype(np.float32)
    return arr

def read_volume_hu(series_path: str) -> Tuple[np.ndarray, Tuple[float,float,float]]:
    """Returns volume in HU [Z,H,W] and voxel spacing (z,y,x). Works for MR too (value scale just not HU)."""
    files = iter_dicom_files(series_path)
    if not files: raise RuntimeError("No DICOMs in series")
    slices, zs = [], []
    spacing_yx = (1.0, 1.0)
    for fp in files:
        try:
            ds = pydicom.dcmread(fp, force=True)
            arr = read_pixel(ds)
            if getattr(ds, "PhotometricInterpretation", "MONOCHROME2") == "MONOCHROME1":
                arr = np.max(arr) - arr
            if hasattr(ds, "PixelSpacing"):
                spacing_yx = (float(ds.PixelSpacing[0]), float(ds.PixelSpacing[1]))
            st = float(getattr(ds, "SliceThickness", 1.0))
            zs.append(st)
            slices.append(arr.astype(np.float32))
        except Exception:
            continue
    if not slices: raise RuntimeError("No decodable slices")
    vol = np.stack(slices, axis=0)   # [Z,H,W]
    spacing = (np.median(zs) if zs else 1.0, spacing_yx[0], spacing_yx[1])
    return vol, spacing

def _ensure_gray2d(img: np.ndarray) -> np.ndarray:
    if img.ndim == 3:
        if img.shape[-1] == 3:
            img = np.mean(img, axis=-1)
        else:
            img = img.max(axis=-1)
    return img.astype(np.float32)

def window_ct(img: np.ndarray, level=200, width=500) -> np.ndarray:
    img = img.astype(np.float32)
    lo, hi = float(level) - float(width)/2.0, float(level) + float(width)/2.0
    img = np.clip(img, lo, hi)
    denom = max(1e-6, (hi - lo))
    img = (img - lo) / denom
    return np.clip(img, 0, 1)

def normalize_mr(img: np.ndarray) -> np.ndarray:
    img = img.astype(np.float32)
    p1, p99 = np.percentile(img, [1, 99])
    img = np.clip(img, p1, p99)
    if p99 > p1:
        img = (img - p1) / (p99 - p1)
    return np.clip(img, 0, 1)

# =============================================================================
# CTA vesselness & highlight
# =============================================================================
def vesselness_2d_soft(img01: np.ndarray) -> np.ndarray:
    """
    Returns vesselness in [0,1] for a 2D slice (CTA soft-tissue window).
    Uses skimage.frangi if available; otherwise a safe LoG-based fallback.
    """
    img01 = np.clip(img01.astype(np.float32), 0, 1)
    if _HAS_FRANGI:
        try:
            v = frangi2d(img01, sigmas=(1, 2, 3), alpha=0.5, beta=0.5, gamma=15.0, black_ridges=False)
            v = np.nan_to_num(v, nan=0.0, posinf=0.0, neginf=0.0)
            v = (v - v.min()) / max(1e-6, (v.max() - v.min()))
            return v.astype(np.float32)
        except Exception:
            pass
    # Fallback: multi-sigma Laplacian-of-Gaussian magnitude (proxy)
    sigmas = [1.0, 2.0, 3.0]
    acc = np.zeros_like(img01, dtype=np.float32)
    for s in sigmas:
        g = ndi.gaussian_laplace(img01, sigma=s)
        acc = np.maximum(acc, -g)  # vessels bright → negative LoG
    acc = (acc - acc.min()) / max(1e-6, (acc.max() - acc.min()))
    return acc

def highlight_stretch(img01: np.ndarray, pct=None, gamma=None, blend=None) -> np.ndarray:
    pct   = Config.HIGHLIGHT_TOP_PCT if pct is None else pct
    gamma = Config.HIGHLIGHT_GAMMA   if gamma is None else gamma
    blend = Config.HIGHLIGHT_BLEND   if blend is None else blend
    img = img01.astype(np.float32); img = np.clip(img, 0.0, 1.0)
    if pct <= 0.0: return img.copy()
    thr = np.percentile(img, 100.0 - pct)
    if not np.isfinite(thr) or thr >= 0.999: return img.copy()
    out = img.copy(); mask = img >= thr
    denom = max(1e-6, 1.0 - thr)
    hi = (img - thr) / denom
    if gamma > 0: hi = np.power(hi, 1.0 / gamma)
    out[mask] = (1.0 - blend) * img[mask] + blend * hi[mask]
    return np.clip(out, 0.0, 1.0)

def morph_clean(mask: np.ndarray, k: int) -> np.ndarray:
    if k is None or k <= 0: return mask
    kernel = np.ones((k,k), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN,  kernel)
    return mask

# =============================================================================
# CACHE KEYS
# =============================================================================
def _cache_path(series_path: str, out_size: int, key_extra: str) -> str:
    os.makedirs(Config.CACHE_DIR, exist_ok=True)
    key = f"{Config.CACHE_VERSION}_{os.path.basename(series_path)}_{out_size}_{key_extra}_k{Config.K_SLICES}_rank{Config.CTA_RANKING}_vbl{Config.VESSELNESS_BLEND}_bone{Config.CT_BONE_MASK_HU}_m{Config.CT_BONE_MORPH_K}"
    h = hashlib.md5(key.encode()).hexdigest()[:16]
    return os.path.join(Config.CACHE_DIR, f"buf_{h}.npy")

def _load_cache(path: str) -> Optional[np.ndarray]:
    try:
        if Config.USE_CACHE and os.path.exists(path):
            arr = np.load(path, allow_pickle=False)
            return arr
    except Exception:
        pass
    return None

def _save_cache(arr: np.ndarray, path: str):
    if not Config.USE_CACHE: return
    try:
        np.save(path, arr, allow_pickle=False)
    except Exception:
        pass

# =============================================================================
# 2.5D MIL DATASET (CTA & MR)
# =============================================================================
def pick_topk_slices_cta_fast(vol_hu: np.ndarray, k: int, out_size: int) -> np.ndarray:
    """
    Fast CTA path:
      - Build CT windows
      - Rank slices by 99th percentile on soft-tissue window (200/500)
      - Bone mask in HU (> Config.CT_BONE_MASK_HU), then highlight (skip bone)
    Returns [K,3,H,W]
    """
    Z, H, W = vol_hu.shape
    # Build window stacks once
    wins = []
    for (L, Wd) in Config.CT_WINDOWS:
        ch = np.stack([window_ct(vol_hu[z], level=L, width=Wd) for z in range(Z)], 0).astype(np.float32)
        wins.append(ch)
    soft = wins[1]  # (200,500)

    # Rank by 99th percentile (fast)
    p99 = np.array([np.percentile(soft[z], 99) for z in range(Z)], dtype=np.float32)
    idx = np.sort(np.argsort(-p99)[:k])

    # Assemble channels & highlight
    imgs = []
    for z in idx:
        chans = [cv2.resize(ch[z], (out_size, out_size), interpolation=cv2.INTER_AREA) for ch in wins]

        # Bone mask directly from HU threshold (then resize & morph clean)
        bone_mask = (vol_hu[z] > float(Config.CT_BONE_MASK_HU)).astype(np.uint8)
        bone_mask = cv2.resize(bone_mask, (out_size, out_size), interpolation=cv2.INTER_NEAREST).astype(bool)
        bone_mask = morph_clean(bone_mask.astype(np.uint8), Config.CT_BONE_MORPH_K).astype(bool)

        if Config.HIGHLIGHT_ENABLE:
            for i in range(len(chans)):
                hi = highlight_stretch(chans[i])
                if Config.CT_HIGHLIGHT_SKIP_BONE:
                    chans[i] = np.where(~bone_mask, hi, chans[i]).astype(np.float32)
                else:
                    chans[i] = hi

        imgs.append(np.stack(chans, 0))  # [3,H,W]
    return np.stack(imgs, 0)            # [K,3,H,W]

def pick_topk_slices_cta_vessel(vol_hu: np.ndarray, k: int, out_size: int) -> np.ndarray:
    """
    Vesselness-based CTA path (heavier CPU):
      - Frangi/LoG vesselness on soft window for ranking
      - Optional vesselness boost per channel
      - HU bone mask + highlight
    Returns [K,3,H,W]
    """
    Z, H, W = vol_hu.shape
    wins = []
    for (L, Wd) in Config.CT_WINDOWS:
        ch = np.stack([window_ct(vol_hu[z], level=L, width=Wd) for z in range(Z)], 0).astype(np.float32)
        wins.append(ch)
    soft = wins[1]  # (200,500)

    vness = np.stack([vesselness_2d_soft(soft[z]) for z in range(Z)], 0)  # [Z,H,W]
    # Rank by center vesselness for robustness
    cy0, cx0, r = H//2, W//2, min(H, W)//4
    yy, xx = np.ogrid[:H, :W]
    circ = ((yy-cy0)**2 + (xx-cx0)**2) <= r*r
    scores = (vness * circ).sum((1,2)) / max(1, circ.sum())
    idx = np.sort(np.argsort(-scores)[:k])

    imgs = []
    for z in idx:
        chans = [cv2.resize(ch[z], (out_size, out_size), interpolation=cv2.INTER_AREA) for ch in wins]
        v = cv2.resize(vness[z], (out_size, out_size), interpolation=cv2.INTER_AREA).astype(np.float32)
        if Config.VESSELNESS_BLEND > 0:
            boost = (1.0 + Config.VESSELNESS_BLEND * v)
            chans = [np.clip(c * boost, 0, 1).astype(np.float32) for c in chans]

        bone_mask = (vol_hu[z] > float(Config.CT_BONE_MASK_HU)).astype(np.uint8)
        bone_mask = cv2.resize(bone_mask, (out_size, out_size), interpolation=cv2.INTER_NEAREST).astype(bool)
        bone_mask = morph_clean(bone_mask.astype(np.uint8), Config.CT_BONE_MORPH_K).astype(bool)

        if Config.HIGHLIGHT_ENABLE:
            for i in range(len(chans)):
                hi = highlight_stretch(chans[i])
                if Config.CT_HIGHLIGHT_SKIP_BONE:
                    chans[i] = np.where(~bone_mask, hi, chans[i]).astype(np.float32)
                else:
                    chans[i] = hi
        imgs.append(np.stack(chans, 0))
    return np.stack(imgs, 0)

def pick_topk_slices_mr(vol: np.ndarray, k: int, out_size: int) -> np.ndarray:
    """
    MR path:
      - Per-slice percentile normalize; rank by 99th percentile (bright vessels)
      - Gentle highlight; replicate into 3 channels
    Returns [K,3,H,W]
    """
    Z, H, W = vol.shape
    norm = []
    scores = []
    for z in range(Z):
        im = normalize_mr(vol[z])
        scores.append(np.percentile(im, 99))
        im = cv2.resize(im, (out_size, out_size), interpolation=cv2.INTER_AREA)
        if Config.HIGHLIGHT_ENABLE:
            im = highlight_stretch(im)
        norm.append(im.astype(np.float32))
    idx = np.sort(np.argsort(-np.array(scores))[:k])
    out = [np.stack([norm[z], norm[z], norm[z]], 0) for z in idx]
    return np.stack(out, 0)

def build_2d_mil_buffers(series_path: str, is_ct: bool, out_size: int, k_slices: int):
    key_extra = f"mil2d_ct{int(is_ct)}"
    cache = _cache_path(series_path, out_size, key_extra=key_extra)
    cached = _load_cache(cache)
    if cached is not None:
        return cached
    vol, _ = read_volume_hu(series_path)
    if is_ct:
        if Config.CTA_RANKING.lower() == "vesselness":
            buf = pick_topk_slices_cta_vessel(vol, k_slices, out_size)
        else:
            buf = pick_topk_slices_cta_fast(vol, k_slices, out_size)
    else:
        buf = pick_topk_slices_mr(vol, k_slices, out_size)
    _save_cache(buf, cache)
    return buf

# =============================================================================
# 3D CROP DATASET (optional)
# =============================================================================
def resample_iso(vol: np.ndarray, spacing: Tuple[float,float,float], iso=1.2) -> np.ndarray:
    z,y,x = spacing
    zoom = (z/iso, y/iso, x/iso)
    return ndi.zoom(vol.astype(np.float32), zoom=zoom, order=1)

def find_cof_bbox_cta(vol_hu_iso: np.ndarray, out_edge: int) -> Tuple[int,int,int]:
    Z,H,W = vol_hu_iso.shape
    soft = np.stack([window_ct(vol_hu_iso[z], 200, 500) for z in range(Z)], 0)
    v3 = np.stack([vesselness_2d_soft(soft[z]) for z in range(Z)], 0)
    v3 = ndi.gaussian_filter(v3, sigma=1.0)
    m = v3 / max(1e-6, v3.sum())
    zz, yy, xx = np.indices(v3.shape)
    cz = int(np.clip((zz*m).sum(), 0, Z-1))
    cy = int(np.clip((yy*m).sum(), 0, H-1))
    cx = int(np.clip((xx*m).sum(), 0, W-1))
    return cz, cy, cx

def crop_cube(vol: np.ndarray, center: Tuple[int,int,int], edge: int) -> np.ndarray:
    cz, cy, cx = center
    Z,H,W = vol.shape
    r = edge//2
    z0, z1 = max(0, cz-r), min(Z, cz+r)
    y0, y1 = max(0, cy-r), min(H, cy+r)
    x0, x1 = max(0, cx-r), min(W, cx+r)
    cube = np.zeros((edge, edge, edge), np.float32)
    cz0 = (edge - (z1 - z0))//2
    cy0 = (edge - (y1 - y0))//2
    cx0 = (edge - (x1 - x0))//2
    cube[cz0:cz0+(z1-z0), cy0:cy0+(y1-y0), cx0:cx0+(x1-x0)] = vol[z0:z1, y0:y1, x0:x1]
    return cube

# =============================================================================
# DATASET & SAMPLER
# =============================================================================
def stratified_split(df: pd.DataFrame, global_col="Aneurysm Present", seed=42, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_idx, va_idx = next(skf.split(df, df[global_col]))
    return df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

def safe_series_path(series_dir: str, uid: str) -> str:
    p = os.path.join(series_dir, str(uid))
    if not os.path.exists(p): raise FileNotFoundError(p)
    return p

def _detect_modality_from_header(series_path: str) -> Optional[str]:
    files = iter_dicom_files(series_path)
    if not files: return None
    try:
        ds = pydicom.dcmread(files[0], stop_before_pixels=True, force=True)
        return str(getattr(ds, "Modality", "")).upper() or None
    except Exception:
        return None

def filter_decodable_df(df: pd.DataFrame, series_dir: str) -> pd.DataFrame:
    uids = df["SeriesInstanceUID"].astype(str).tolist()
    keep_mask = []
    for uid in uids:
        sp = os.path.join(series_dir, uid)
        ok = False
        try:
            files = iter_dicom_files(sp)
            if files:
                ds = pydicom.dcmread(files[0], force=True)
                _ = ds.pixel_array
                ok = True
        except Exception:
            ok = False
        keep_mask.append(ok)
    kept = df[keep_mask].reset_index(drop=True)
    print(f"Decodable series (quick test): {len(kept)}/{len(df)} kept")
    return kept

class MIL2DDataset(Dataset):
    """Returns one study as K slices [K,3,H,W], normalized for ImageNet, and label vector [C]."""
    def __init__(self, df: pd.DataFrame, series_dir: str, out_size: int, k_slices: int, modality: str):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.out_size = out_size
        self.k_slices = k_slices
        self.modality = modality  # "CT" or "MR"
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss: raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)
        if self.modality == "CT":
            buf = build_2d_mil_buffers(sp, is_ct=True, out_size=self.out_size, k_slices=self.k_slices)  # [K,3,H,W]
        else:
            buf = build_2d_mil_buffers(sp, is_ct=False, out_size=self.out_size, k_slices=self.k_slices)
        x = torch.from_numpy(buf).float()         # [K,3,H,W] in [0,1]
        # ImageNet normalization per-slice
        x = (x - IMAGENET_MEAN) / IMAGENET_STD
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

class Vol3DDataset(Dataset):
    """3D crop dataset: returns [3,D,H,W] by stacking same volume into 3 channels, normalized ~ImageNet."""
    def __init__(self, df: pd.DataFrame, series_dir: str, crop_edge: int, iso: float, modality: str):
        self.df = df.reset_index(drop=True)
        self.series_dir = series_dir
        self.edge = crop_edge
        self.iso = iso
        self.modality = modality
        miss = [c for c in Config.TARGET_COLS if c not in df.columns]
        if miss: raise ValueError(f"Missing target cols: {miss}")

    def __len__(self): return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        uid = str(row["SeriesInstanceUID"])
        sp = safe_series_path(self.series_dir, uid)
        vol_hu, spacing = read_volume_hu(sp)
        vol_iso = resample_iso(vol_hu, spacing, iso=self.iso)  # [Z,H,W]
        if self.modality == "CT":
            cz, cy, cx = find_cof_bbox_cta(vol_iso, self.edge)
        else:
            # For MR, use 99th-percentile intensity COM
            v = np.clip(vol_iso, np.percentile(vol_iso, 1), np.percentile(vol_iso, 99))
            v = (v - v.min()) / max(1e-6, (v.max() - v.min()))
            m = v / max(1e-6, v.sum())
            zz, yy, xx = np.indices(v.shape)
            cz = int(np.clip((zz*m).sum(), 0, v.shape[0]-1))
            cy = int(np.clip((yy*m).sum(), 0, v.shape[1]-1))
            cx = int(np.clip((xx*m).sum(), 0, v.shape[2]-1))
        cube = crop_cube(vol_iso, (cz,cy,cx), self.edge)  # [D,H,W]
        if self.modality == "CT":
            D,H,W = cube.shape
            ch = np.stack([window_ct(cube[d], 200, 500) for d in range(D)], 0)
        else:
            v = np.clip(cube, np.percentile(cube, 1), np.percentile(cube, 99))
            ch = (v - v.min()) / max(1e-6, (v.max()-v.min()))
        vol3 = np.stack([ch, ch, ch], 0)  # [3,D,H,W]
        x = torch.from_numpy(vol3).float()
        for c in range(3):
            x[c] = (x[c] - IMAGENET_MEAN[c]) / IMAGENET_STD[c]
        y = torch.tensor([row[c] for c in Config.TARGET_COLS], dtype=torch.float32)
        return x, y, uid

def build_class_balanced_sampler(df: pd.DataFrame, target_cols: List[str]) -> WeightedRandomSampler:
    pos = df[target_cols].sum(0).values
    neg = len(df) - pos
    cls_w = np.where(pos > 0, neg / np.maximum(1, pos), 1.0)
    cls_w = np.clip(cls_w, 1.0, 50.0)
    Y = df[target_cols].values.astype(np.float32)
    w = 1.0 + (Y * cls_w).sum(1)
    w = torch.as_tensor(w, dtype=torch.double)
    return WeightedRandomSampler(weights=w, num_samples=len(df), replacement=True)

# =============================================================================
# MODELS
# =============================================================================
def efficientnet_b2_features():
    m = torchvision.models.efficientnet_b2(weights=torchvision.models.EfficientNet_B2_Weights.IMAGENET1K_V1)
    feat_dim = m.classifier[1].in_features
    backbone = nn.Sequential(m.features, nn.AdaptiveAvgPool2d(1), nn.Flatten())  # -> [B, feat_dim]
    return backbone, feat_dim

class AttnMIL(nn.Module):
    """Processes K slices with a 2D backbone and attention-pools to a study embedding."""
    def __init__(self, feat_dim, hidden=512):
        super().__init__()
        self.attn = nn.Sequential(
            nn.Linear(feat_dim, hidden), nn.ReLU(inplace=True),
            nn.Linear(hidden, 1)
        )
    def forward(self, F):  # F: [B,K,D]
        A = self.attn(F).squeeze(-1)           # [B,K]
        A = torch.softmax(A, dim=1)            # weights sum to 1
        M = torch.sum(F * A.unsqueeze(-1), 1)  # [B,D]
        return M, A

class HierHead(nn.Module):
    """Two heads: Present (1) + Locations (13)."""
    def __init__(self, in_dim, n_locs=13):
        super().__init__()
        self.present = nn.Linear(in_dim, 1)
        self.locs    = nn.Linear(in_dim, n_locs)
    def forward(self, z):
        return self.present(z).squeeze(-1), self.locs(z)

class MIL2DModel(nn.Module):
    def __init__(self, n_out=14):
        super().__init__()
        self.backbone, feat_dim = efficientnet_b2_features()
        self.mil = AttnMIL(feat_dim)
        self.head = HierHead(feat_dim, n_locs=n_out-1)
    def forward(self, x):  # x: [B,K,3,H,W]
        B,K,C,H,W = x.shape
        x = x.view(B*K, C, H, W)
        f = self.backbone(x)                 # [B*K, D]
        f = f.view(B, K, -1)                 # [B,K,D]
        z, att = self.mil(f)                 # [B,D], [B,K]
        present, locs = self.head(z)         # [B], [B,13]
        logits = torch.cat([locs, present.unsqueeze(-1)], dim=1)  # [B,14] (locations first)
        return logits, att

class Res3DModel(nn.Module):
    def __init__(self, n_out=14):
        super().__init__()
        m = r3d_18(weights=None)
        m.fc = nn.Linear(m.fc.in_features, 512)
        self.backbone = m
        self.head = HierHead(512, n_locs=n_out-1)
    def forward(self, x):  # [B,3,D,H,W]
        z = self.backbone(x)                 # [B,512]
        present, locs = self.head(z)
        logits = torch.cat([locs, present.unsqueeze(-1)], dim=1)
        return logits, None

# =============================================================================
# LOSSES
# =============================================================================
class AsymmetricLoss(nn.Module):
    """
    ASL for multi-label classification.
    Default: gamma_neg=4, gamma_pos=0, clip=0.05
    """
    def __init__(self, gamma_pos=0, gamma_neg=4, clip=0.05, eps=1e-8):
        super().__init__()
        self.gp = gamma_pos
        self.gn = gamma_neg
        self.clip = clip
        self.eps = eps

    def forward(self, logits, targets):
        x_sigmoid = torch.sigmoid(logits)
        xs_pos = x_sigmoid
        xs_neg = 1.0 - x_sigmoid
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)
        loss_pos = targets * torch.log(xs_pos.clamp(min=self.eps))
        loss_neg = (1 - targets) * torch.log(xs_neg.clamp(min=self.eps))
        loss = loss_pos + loss_neg
        if self.gp > 0 or self.gn > 0:
            pt = x_sigmoid * targets + (1 - x_sigmoid) * (1 - targets)
            one_sided_gamma = self.gp * targets + self.gn * (1 - targets)
            focal_weight = (1 - pt) ** one_sided_gamma
            loss *= focal_weight
        return -loss.mean()

class BCEWithLogitsFocal(nn.Module):
    def __init__(self, pos_weight=None, gamma=2.0):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction="none")
        self.gamma = gamma
    def forward(self, logits, targets):
        bce = self.bce(logits, targets)
        p = torch.sigmoid(logits)
        pt = p*targets + (1-p)*(1-targets)
        focal = (1.0 - pt).clamp_min(1e-6).pow(self.gamma)
        return (focal * bce).mean()

# =============================================================================
# METRICS & THRESHOLDS
# =============================================================================
def compute_auc_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_auc, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_auc[c] = np.nan
                continue
            auc = roc_auc_score(y_true[:, i], y_prob[:, i])
            col_auc[c] = float(auc); macro_vals.append(float(auc))
        except Exception:
            col_auc[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_auc, macro

def compute_ap_per_column(y_true: np.ndarray, y_prob: np.ndarray, cols: List[str]):
    col_ap, macro_vals = {}, []
    for i, c in enumerate(cols):
        try:
            if len(np.unique(y_true[:, i])) < 2:
                col_ap[c] = np.nan
                continue
            ap = average_precision_score(y_true[:, i], y_prob[:, i])
            col_ap[c] = float(ap); macro_vals.append(float(ap))
        except Exception:
            col_ap[c] = np.nan
    macro = float(np.mean(macro_vals)) if macro_vals else np.nan
    return col_ap, macro

def compute_micro_ap(y_true: np.ndarray, y_prob: np.ndarray):
    mask = (y_true.sum(0) > 0) & ((1 - y_true).sum(0) > 0)
    if mask.sum() == 0: return np.nan
    return average_precision_score(y_true[:, mask].ravel(), y_prob[:, mask].ravel())

def tune_thresholds_best_f1(y_true: np.ndarray, y_prob: np.ndarray) -> np.ndarray:
    n_labels = y_true.shape[1]
    thrs = np.zeros(n_labels, dtype=np.float32)
    grid = np.linspace(0.0, 1.0, 1001)
    for i in range(n_labels):
        yi = y_true[:, i].astype(np.int32)
        pi = y_prob[:, i]
        if len(np.unique(yi)) < 2: thrs[i] = 0.5; continue
        best_f1, best_t = -1.0, 0.5
        for t in grid:
            pred = (pi >= t).astype(np.int32)
            tp = int((pred & (yi == 1)).sum()); fp = int((pred & (yi == 0)).sum())
            fn = int(((1 - pred) & (yi == 1)).sum())
            precision = tp / max(1, (tp + fp)); recall = tp / max(1, (tp + fn))
            f1 = 2*precision*recall / max(1e-8, (precision + recall))
            if f1 > best_f1: best_f1, best_t = f1, t
        thrs[i] = best_t
    return thrs

def smallest_t_for_precision(y_true: np.ndarray, y_prob: np.ndarray, target_p=0.8) -> float:
    if len(y_true) == 0: return 0.5
    order = np.argsort(-y_prob)
    y = y_true[order]; p = y_prob[order]
    tp=0; fp=0; best_t = 1.0
    for i in range(len(y)):
        if y[i] == 1: tp += 1
        else: fp += 1
        prec = tp / max(1, tp+fp)
        if prec >= target_p:
            best_t = p[i]; break
    # If never reach target precision, return a conservative high threshold
    return float(best_t)

def confusion_stats(y_true: np.ndarray, y_prob: np.ndarray, thrs: np.ndarray,
                    cols: List[str], top_k: int = 6) -> pd.DataFrame:
    rows = []
    for i, c in enumerate(cols):
        yi = y_true[:, i].astype(np.int32)
        pi = (y_prob[:, i] >= thrs[i]).astype(np.int32)
        tp = int((pi & (yi == 1)).sum())
        fp = int((pi & (yi == 0)).sum())
        tn = int(((1 - pi) & (yi == 0)).sum())
        fn = int(((1 - pi) & (yi == 1)).sum())
        prec = tp / max(1, (tp + fp))
        rec  = tp / max(1, (tp + fn))
        f1   = 2*prec*rec / max(1e-8, (prec + rec))
        rows.append([c, thrs[i], tp, fp, tn, fn, prec, rec, f1])
    df = pd.DataFrame(rows, columns=["label","thr","TP","FP","TN","FN","precision","recall","F1"])
    priority = ["Aneurysm Present", "Left Middle Cerebral Artery", "Right Middle Cerebral Artery",
                "Basilar Tip", "Anterior Communicating Artery"]
    order = priority + [c for c in cols if c not in priority]
    df = df.set_index("label").loc[order].reset_index()
    return df.head(top_k)

# =============================================================================
# TEMPERATURE SCALING (per label)
# =============================================================================
@torch.no_grad()
def collect_logits(model, loader, dev):
    model.eval()
    logits_list, targets_list, uids = [], [], []
    for x, y, idlist in loader:
        x = x.to(dev, non_blocking=True)
        y = y.to(dev, non_blocking=True)
        out, _ = model(x)              # [B,14]
        logits_list.append(out.detach().cpu())
        targets_list.append(y.detach().cpu())
        uids.extend(list(idlist))
    L = torch.cat(logits_list, 0).numpy()
    T = torch.cat(targets_list, 0).numpy()
    return L, T, uids

def fit_temperature_per_label(logits: np.ndarray, targets: np.ndarray, max_iter=200, lr=0.05) -> np.ndarray:
    """
    Fit a temperature per column minimizing BCEWithLogits on val.
    Returns temperatures shape [C], >= 1e-3.
    """
    if logits.size == 0:
        return np.ones((len(Config.TARGET_COLS),), dtype=np.float32)
    L = torch.tensor(logits, dtype=torch.float32)
    Y = torch.tensor(targets, dtype=torch.float32)
    C = L.shape[1]
    temps = torch.ones(C, dtype=torch.float32, requires_grad=True)
    optim_t = torch.optim.Adam([temps], lr=lr)
    for _ in range(max_iter):
        optim_t.zero_grad()
        scaled = L / temps.clamp_min(1e-3)
        loss = nn.BCEWithLogitsLoss()(scaled, Y)
        loss.backward()
        optim_t.step()
    return temps.detach().clamp_min(1e-3).cpu().numpy()

# =============================================================================
# TRAIN / EVAL LOOP (one modality)
# =============================================================================
def run_one_modality(modality_name: str, full_df: pd.DataFrame, series_dir: str, dev: str):
    """
    modality_name: "CT" -> CT/CTA subset, "MR" -> MR/MRA subset
    """
    is_ct = (modality_name == "CT")
    print(f"\n================= TRAINING MODALITY: {modality_name} =================")
    print(f"Config: NUM_WORKERS={Config.NUM_WORKERS} | K_SLICES={Config.K_SLICES} | CTA_RANKING={Config.CTA_RANKING} | VBLEND={Config.VESSELNESS_BLEND}")

    # Subset df by modality (prefer CSV, fallback to header)
    df = full_df.copy()
    df["Modality"] = df.get("Modality", "").fillna("")
    modes = []
    for _, r in df.iterrows():
        m = str(r.get("Modality", "")).upper()
        if not m:
            try:
                m = _detect_modality_from_header(os.path.join(series_dir, str(r["SeriesInstanceUID"]))) or ""
            except Exception:
                m = ""
        modes.append(m)
    df["__mod"] = modes
    if is_ct:
        sub = df[df["__mod"].isin(["CT","CTA"])].reset_index(drop=True)
    else:
        sub = df[df["__mod"].isin(["MR","MRA"])].reset_index(drop=True)

    if len(sub) == 0:
        print(f"No {modality_name} studies found. Skipping.")
        return None

    # Split stratified by "Aneurysm Present"
    train_df, val_df = stratified_split(sub, seed=Config.SEED)

    # Optional smoke limits
    if Config.SMOKE_TRAIN_LIMIT is not None:
        train_df = train_df.head(Config.SMOKE_TRAIN_LIMIT).copy()
    if Config.SMOKE_VAL_LIMIT is not None:
        val_df   = val_df.head(Config.SMOKE_VAL_LIMIT).copy()

    # Filter decodable
    print("Scanning decodability (quick test)...")
    train_keep = filter_decodable_df(train_df, series_dir)
    val_keep   = filter_decodable_df(val_df, series_dir)
    print(f"{modality_name} Train: {len(train_keep)} | Val: {len(val_keep)}")

    # Build datasets/loaders
    if Config.MODE_3D:
        train_ds = Vol3DDataset(train_keep, series_dir, Config.VOL_CROP_SIZE, Config.VOL_ISO_SPACING_MM, modality_name)
        val_ds   = Vol3DDataset(val_keep,   series_dir, Config.VOL_CROP_SIZE, Config.VOL_ISO_SPACING_MM, modality_name)
    else:
        train_ds = MIL2DDataset(train_keep, series_dir, Config.IMG_SIZE, Config.K_SLICES, modality_name)
        val_ds   = MIL2DDataset(val_keep,   series_dir, Config.IMG_SIZE, Config.K_SLICES, modality_name)

    train_sampler = build_class_balanced_sampler(train_keep, Config.TARGET_COLS)
    tkw = dict(batch_size=Config.BATCH_SIZE, sampler=train_sampler, num_workers=Config.NUM_WORKERS, drop_last=True)
    vkw = dict(batch_size=Config.BATCH_SIZE, shuffle=False, num_workers=Config.NUM_WORKERS, drop_last=False)
    if dev == "cuda":
        tkw["pin_memory"] = True; vkw["pin_memory"] = True
    train_loader = DataLoader(train_ds, **tkw)
    val_loader   = DataLoader(val_ds, **vkw)

    # Model
    if Config.MODE_3D:
        model = Res3DModel(n_out=len(Config.TARGET_COLS)).to(dev)
    else:
        model = MIL2DModel(n_out=len(Config.TARGET_COLS)).to(dev)
    print(f"Model params: {count_params(model):,}")

    # Loss
    present_idx = Config.TARGET_COLS.index("Aneurysm Present")
    if Config.LOSS_NAME.upper() == "ASL":
        criterion = AsymmetricLoss(gamma_pos=0, gamma_neg=4, clip=0.05)
    else:
        # compute pos_weight per label from training subset
        posw = []
        for c in Config.TARGET_COLS:
            pos = train_keep[c].sum(); neg = len(train_keep) - pos
            w = (neg / max(1.0, pos)) if pos > 0 else 1.0
            posw.append(float(np.clip(w, 1.0, 50.0)))
        criterion = BCEWithLogitsFocal(pos_weight=torch.tensor(posw, device=dev), gamma=Config.FOCAL_GAMMA)

    opt = optim.AdamW(model.parameters(), lr=Config.LR, weight_decay=1e-4)
    sch = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=Config.EPOCHS)
    scaler = torch.cuda.amp.GradScaler(enabled=(dev=="cuda" and Config.USE_AMP))

    best_macro_auc = -1.0
    best_ckpt = f"/kaggle/working/{modality_name.lower()}_best.pt"

    # ------------- Train -------------
    for epoch in range(Config.EPOCHS):
        model.train()
        run_loss = 0.0; t0 = time.time()
        for b, (x, y, _) in enumerate(train_loader, 1):
            x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=(dev=="cuda" and Config.USE_AMP)):
                logits, _att = model(x)                # [B,14]
                loss = criterion(logits, y)
                # consistency: locations <= present
                if Config.CONSISTENCY_W > 0:
                    probs = torch.sigmoid(logits)
                    present = probs[:, present_idx:present_idx+1]
                    locs   = probs[:, :present_idx]   # [B,13]
                    consist = torch.clamp(locs - present, min=0.0).pow(2).mean()
                    loss = loss + Config.CONSISTENCY_W * consist
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()
            run_loss += loss.item()
            if b % Config.LOG_EVERY_N == 0 or b == 1:
                print(f"[{modality_name} E{epoch+1} {b}/{len(train_loader)}] loss={run_loss/b:.4f} lr={sch.get_last_lr()[0]:.2e}")
        sch.step()

        # ------------- Val -------------
        model.eval()
        val_loss = 0.0
        logits_all, probs_all, targs_all, uids_all = [], [], [], []
        with torch.no_grad():
            for x, y, uids in val_loader:
                x, y = x.to(dev, non_blocking=True), y.to(dev, non_blocking=True)
                with torch.cuda.amp.autocast(enabled=(dev=="cuda" and Config.USE_AMP)):
                    out, _att = model(x)
                    ls = nn.BCEWithLogitsLoss(reduction="mean")(out, y)
                val_loss += ls.item()
                logits_all.append(out.detach().cpu().numpy())
                probs_all.append(torch.sigmoid(out).detach().cpu().numpy())
                targs_all.append(y.detach().cpu().numpy())
                uids_all.extend(list(uids))

        L = np.vstack(logits_all) if logits_all else np.zeros((0, len(Config.TARGET_COLS)))
        P = np.vstack(probs_all)  if probs_all  else np.zeros((0, len(Config.TARGET_COLS)))
        T = np.vstack(targs_all)  if targs_all  else np.zeros((0, len(Config.TARGET_COLS)))
        col_aucs, macro_auc = compute_auc_per_column(T, P, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P, Config.TARGET_COLS)
        micro_ap = compute_micro_ap(T, P)
        dt = time.time() - t0
        print(f"{modality_name} Epoch {epoch+1}/{Config.EPOCHS} | {dt:.1f}s  val_loss={val_loss/max(1,len(val_loader)):.4f}  "
              f"macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f} micro_ap={micro_ap:.4f}")

        if macro_auc > best_macro_auc:
            best_macro_auc = macro_auc
            torch.save({
                "epoch": epoch+1,
                "model_state_dict": model.state_dict(),
                "auc_macro": macro_auc,
                "target_cols": Config.TARGET_COLS,
            }, best_ckpt)
            print(f"Saved new best for {modality_name}: {best_ckpt} (macro_auc={macro_auc:.4f})")

    # -------- Post-hoc calibration (temperature scaling) on final val --------
    L, T, uids_all = collect_logits(model, val_loader, dev)
    if Config.DO_TEMP_SCALING:
        temps = fit_temperature_per_label(L, T, max_iter=200, lr=0.05)  # [C]
    else:
        temps = np.ones(L.shape[1] if L.size else len(Config.TARGET_COLS), dtype=np.float32)
    Ls = L / np.clip(temps, 1e-3, None)
    P_cal = 1.0 / (1.0 + np.exp(-Ls))

    # Gating: locations *= present^alpha
    present_idx = Config.TARGET_COLS.index("Aneurysm Present")
    loc_idx = [i for i in range(len(Config.TARGET_COLS)) if i != present_idx]
    if Config.GATE_ALPHA > 0 and P_cal.size:
        pr = np.clip(P_cal[:, present_idx:present_idx+1], 0, 1) ** Config.GATE_ALPHA
        P_cal[:, loc_idx] = P_cal[:, loc_idx] * pr

    # Thresholds:
    thrs = np.zeros(len(Config.TARGET_COLS), dtype=np.float32)
    if P_cal.size:
        # 1) Present: smallest t with precision >= target
        thrs[present_idx] = smallest_t_for_precision(T[:, present_idx], P_cal[:, present_idx], target_p=Config.TARGET_PRECISION_PRESENT)
        # 2) Locations: F1-tuned
        thrs[loc_idx] = tune_thresholds_best_f1(T[:, loc_idx], P_cal[:, loc_idx])
    else:
        thrs[:] = 0.5

    # Confusion summary
    if P_cal.size:
        conf_df = confusion_stats(T, P_cal, thrs, Config.TARGET_COLS, top_k=6)
        print(f"\n{modality_name} (Calibrated) confusion summary @ thresholds:")
        for _, r in conf_df.iterrows():
            print(f"  {r['label'][:28]:28s} thr={r['thr']:.2f}  TP={r['TP']:3d} FP={r['FP']:3d} "
                  f"TN={r['TN']:3d} FN={r['FN']:3d}  P={r['precision']:.2f} R={r['recall']:.2f} F1={r['F1']:.2f}")

        # Metrics (calibrated)
        col_aucs, macro_auc = compute_auc_per_column(T, P_cal, Config.TARGET_COLS)
        col_aps , macro_ap  = compute_ap_per_column(T, P_cal, Config.TARGET_COLS)
        micro_ap = compute_micro_ap(T, P_cal)
        print(f"\n{modality_name} (Calibrated) macro_auc={macro_auc:.4f} macro_ap={macro_ap:.4f} micro_ap={micro_ap:.4f}")

    # Save per-modality val preds CSV
    out_csv = f"/kaggle/working/{modality_name.lower()}_val_preds.csv"
    df_out = pd.DataFrame({"SeriesInstanceUID": uids_all})
    if P_cal.size:
        for i, c in enumerate(Config.TARGET_COLS):
            df_out[c + "_prob"] = P_cal[:, i]
            df_out[c + "_true"] = T[:, i]
            df_out[c + "_thr"]  = thrs[i]
    df_out.to_csv(out_csv, index=False)
    print(f"Saved {modality_name} validation predictions to {out_csv} (shape={df_out.shape})")

    # Save final checkpoint with temps + thresholds
    torch.save({
        "epoch": Config.EPOCHS,
        "model_state_dict": model.state_dict(),
        "target_cols": Config.TARGET_COLS,
        "temps": temps.astype(np.float32),
        "thresholds": thrs.astype(np.float32),
        "modality": modality_name,
        "config_snapshot": {
            "IMG_SIZE": Config.IMG_SIZE,
            "K_SLICES": Config.K_SLICES,
            "CTA_RANKING": Config.CTA_RANKING,
            "VESSELNESS_BLEND": Config.VESSELNESS_BLEND,
            "LOSS_NAME": Config.LOSS_NAME,
            "GATE_ALPHA": Config.GATE_ALPHA,
            "CONSISTENCY_W": Config.CONSISTENCY_W,
        }
    }, f"/kaggle/working/{modality_name.lower()}_best.pt")
    print(f"Saved {modality_name} calibrated model with temps+thresholds: /kaggle/working/{modality_name.lower()}_best.pt")

    return True

# =============================================================================
# MAIN
# =============================================================================
def main():
    print("== RSNA IA Detection: Modality-Split MIL (ASL, TempScaling, Gated Heads) ==")
    set_seed(Config.SEED, benchmark_mode=True if device()=="cuda" else False)
    dev = device()
    print(f"Device: {dev} | AMP: {Config.USE_AMP} | 3D Mode: {Config.MODE_3D}")
    print(f"CPU cores: {os.cpu_count()} | DataLoader workers: {Config.NUM_WORKERS}")

    csv_path = os.path.join(Config.DATA_DIR, "train.csv")
    if not os.path.exists(csv_path): raise FileNotFoundError(csv_path)
    df = pd.read_csv(csv_path)
    series_dir = os.path.join(Config.DATA_DIR, "series")
    print(f"Rows in CSV: {len(df):,}")

    # Train per-modality
    run_one_modality("CT", df, series_dir, dev)
    run_one_modality("MR", df, series_dir, dev)
    print("All done.")

# Jupyter helper
def train_model_jupyter(**kwargs):
    # Allow quick overrides
    for k,v in kwargs.items():
        if hasattr(Config, k):
            setattr(Config, k, v)
    main()

if __name__ == "__main__":
    main()

print("Loaded script. Example:")
print('train_model_jupyter(DATA_DIR="/kaggle/input/rsna-intracranial-aneurysm-detection", IMG_SIZE=320, K_SLICES=8, EPOCHS=4)')